In [11]:
import numpy as np
import os
import pandas as pd
from arcpy import env
from arcpy import TableToTable_conversion
import matplotlib.ticker as mtick

In [12]:
dirWork = os.getcwd()
dirData = os.path.join(dirWork, r'data')
dirTDM  = os.path.join(dirData, r'tdm\1_WF')
dirFactors   = os.path.join(dirWork, r'data-processed\factors')
dirResults   = os.path.join(dirWork, r'results')

csv_FacGeoFunGroupAreaTypeVolume_ToStationGroup = os.path.join(dirFactors, r'facgeo_fungroup_areatype_volume_to_station_group.csv')
csv_SegmentsMatchedToFactorGroups = os.path.join(dirFactors,'SegmentsMatchedToFactorGroups.csv')
csv_SeasonFactors = os.path.join(dirFactors, r'SeasonFactors.csv')
csv_SeasonFactors = os.path.join(dirFactors, r'SeasonFactors.csv')
csv_DOWFactors = os.path.join(dirFactors, r'DOWFactors.csv')
csv_TODFactors = os.path.join(dirFactors, r'TODFactors.csv')


shp_Forecasts = r"data\forecasts\Traffic-Volume-Historic-and-Forecast.shp"

In [13]:
df_SegmentsMatchedToFactorGroups = pd.read_csv(csv_SegmentsMatchedToFactorGroups)
display(df_SegmentsMatchedToFactorGroups)

,SEGID,AADT2019,ATYPE,FTGROUP,FACGEO,STATIONGROUP
0,0006_141.0,1517.0,1,Arterial,Statewide,COP
1,0006_146.9,1517.0,1,Arterial,Statewide,COP
2,0006_149.9,2441.0,1,Arterial,Statewide,COP
3,0006_150.6,2441.0,1,Arterial,Statewide,COP
4,0006_152.6,2417.0,1,Arterial,Statewide,COP
...,...,...,...,...,...,...
5344,0089_326.5,13767.0,3,Arterial,Statewide,COT
5345,2849_001.5,1096.0,2,Arterial,Statewide,COS
5346,MAG_6856,0.0,3,Arterial,Statewide,COT
5347,2845_001.0,3546.0,3,Arterial,Statewide,COT


# Run Settings

In [14]:
#set to true for first time to create CSVs, false to speed up for subsequent times
#process_TDMdbftocsv = True

#export results
export_results = True

# Define Scenarios

In [15]:
df_Scenarios = pd.DataFrame()

#define AdjCode for pivot when model is not related to forecast
#AADT will be adjusted based on difference between and forecast
#if ForecastScenarioID is note defined, then the associated forecast volume will be used for that year
#DBF don't include filename extension

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'Base19',
                                    'ScenarioName'   : '2019 Base',
                                    'ScenarioYear'   :  2019,
                                    'ScenarioGroup'  : 'Base',
                                    'SegSummaryCSV'  : 'v901_SE19_Net19_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'Base23',
                                    'ScenarioName'   : '2023 Base',
                                    'ScenarioYear'   :  2023,
                                    'ScenarioGroup'  : 'Base',
                                    'SegSummaryCSV'  : 'v901_SE23_Net23_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'Base19'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'TIP28',
                                    'ScenarioName'   : '2028 TIP Projects',
                                    'ScenarioYear'   :  2028,
                                    'ScenarioGroup'  : 'TIP',
                                    'SegSummaryCSV'  : 'v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'Base23'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'RTP32',
                                    'ScenarioName'   : '2032 Fiscally-Constrained',
                                    'ScenarioYear'   :  2032,
                                    'ScenarioGroup'  : 'RTP',
                                    'SegSummaryCSV'  : 'v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP28',
                                    'AdjScenarioCode': None
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'RTP42',
                                    'ScenarioName'   : '2042 Fiscally-Constrained
                                    'ScenarioYear'   :  2042,
                                    'ScenarioGroup'  : 'RTP',
                                    'SegSummaryCSV'  : 'v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP32'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode' : 'RTP50',
                                    'ScenarioName' : '2050 Fiscally-Constrained',
                                    'ScenarioYear' :  2050,
                                    'ScenarioGroup': 'RTP',
                                    'SegSummaryCSV': 'v901_RTP_SE50_Net50_Summary_SEGID_Detailed.csv',
                                    'Process'      : 'Yes',
                                    'PrvScenarioCode': 'RTP42'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB3228',
                                    'ScenarioName'   : '2032 SE on a 2028 Network',
                                    'ScenarioYear'   :  2032,
                                    'ScenarioGroup'  : 'NoBuild28',
                                    'AdjScenarioCode': 'RTP32',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP28'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB4228',
                                    'ScenarioName'   : '2042 SE on a 2028 Network',
                                    'ScenarioYear'   :  2042,
                                    'ScenarioGroup'  : 'NoBuild28',
                                    'AdjScenarioCode': 'RTP42',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'NB3228'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB5028',
                                    'ScenarioName'   : '2050 SE on a 2028 Network',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'NoBuild28',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'NB4228'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB4232',
                                    'ScenarioName'   : '2042 SE on a 2032 Network',
                                    'ScenarioYear'   :  2042,
                                    'ScenarioGroup'  : 'NoBuild32',
                                    'AdjScenarioCode': 'RTP42',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE42_Net32_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP32'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB5032',
                                    'ScenarioName'   : '2050 SE on a 2032 Network',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'NoBuild32',
                                    'AdjScenarioCode': 'RTP32',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE50_Net32_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'NB4232'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB5042',
                                    'ScenarioName'   : '2050 SE on a 2042 Network',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'NoBuild42',
                                    'AdjScenarioCode': 'RTP42',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP42'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF32',
                                    'ScenarioName'   : '2032 Needs-Based',
                                    'ScenarioYear'   :  2032,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP32',
                                    'SegSummaryCSV'  : 'v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP28'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF42',
                                    'ScenarioName'   : '2042 Needs-Based',
                                    'ScenarioYear'   :  2042,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP42',
                                    'SegSummaryCSV'  : 'v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'UF32'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF50',
                                    'ScenarioName'   : '2050 Needs-Based',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'UF42'
                                   },ignore_index=True)

#df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF32_MAG',
#                                    'ScenarioName'   : '2032 Needs-Based - MAG',
#                                    'ScenarioYear'   :  2032,
#                                    'ScenarioGroup'  : 'NeedsMAG',
#                                    'AdjScenarioCode': 'RTP32',
#                                    'SegSummaryCSV'  : 'v901_Needs_MAG_SE32_Net32_Summary_SEGID_Detailed.csv',
#                                    'Process'        : 'Yes',
#                                    'PrvScenarioCode': 'TIP28'
#                                   },ignore_index=True)
#
#df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF42_MAG',
#                                    'ScenarioName'   : '2042 Needs-Based - MAG',
#                                    'ScenarioYear'   :  2042,
#                                    'ScenarioGroup'  : 'NeedsMAG',
#                                    'AdjScenarioCode': 'RTP42',
#                                    'SegSummaryCSV'  : 'v901_Needs_MAG_SE42_Net42_Summary_SEGID_Detailed.csv',
#                                    'Process'        : 'Yes',
#                                    'PrvScenarioCode': 'UF32_MAG'
#                                   },ignore_index=True)
#
#df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF50_MAG',
#                                    'ScenarioName'   : '2050 Needs-Based - MAG',
#                                    'ScenarioYear'   :  2050,
#                                    'ScenarioGroup'  : 'NeedsMAG',
#                                    'AdjScenarioCode': 'RTP50',
#                                    'SegSummaryCSV'  : 'v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detailed.csv',
#                                    'Process'        : 'Yes',
#                                    'PrvScenarioCode': 'UF42_MAG'
#                                   },ignore_index=True)


#all scenarios will be read in to allow for pivoting if needed
df_Scenarios.index.name = 'ScenarioID'
df_Scenarios['ScenarioYear'] = pd.to_numeric(df_Scenarios['ScenarioYear'], downcast='integer')
df_Scenarios

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
5,RTP50,2050 Fiscally-Constrained Scenario,2050,RTP,v901_RTP_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,RTP42,NaN
6,NB3228,2032 SE on a 2028 Network Scenario,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,NB4228,2042 SE on a 2028 Network Scenario,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,NB3228,RTP42
8,NB5028,2050 SE on a 2028 Network Scenario,2050,NoBuild28,v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed...,Yes,NB4228,RTP50


In [16]:
df_Scenarios_toProcess = df_Scenarios[df_Scenarios['Process']=='Yes']
df_Scenarios_toProcess

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
5,RTP50,2050 Fiscally-Constrained Scenario,2050,RTP,v901_RTP_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,RTP42,NaN
6,NB3228,2032 SE on a 2028 Network Scenario,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,NB4228,2042 SE on a 2028 Network Scenario,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,NB3228,RTP42
8,NB5028,2050 SE on a 2028 Network Scenario,2050,NoBuild28,v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed...,Yes,NB4228,RTP50


In [17]:
df_Scenarios_toAdjust = df_Scenarios_toProcess[df_Scenarios_toProcess['AdjScenarioCode'].notnull()]
df_Scenarios_toAdjust

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
6,NB3228,2032 SE on a 2028 Network Scenario,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,NB4228,2042 SE on a 2028 Network Scenario,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,NB3228,RTP42
8,NB5028,2050 SE on a 2028 Network Scenario,2050,NoBuild28,v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed...,Yes,NB4228,RTP50
9,NB4232,2042 SE on a 2032 Network,2042,NoBuild32,v901_NoBuild_SE42_Net32_Summary_SEGID_Detailed...,Yes,RTP32,RTP42
10,NB5032,2050 SE on a 2042 Network,2050,NoBuild32,v901_NoBuild_SE50_Net32_Summary_SEGID_Detailed...,Yes,NB4232,RTP32
11,NB5042,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP42
12,UF32,2032 Needs-Based Scenario,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based Scenario,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42
14,UF50,2050 Needs-Based Scenario,2050,Needs,v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,UF42,RTP50


In [18]:
#define functions to get scenario data
def idFromCode (sCode):
    return df_Scenarios.index[df_Scenarios['ScenarioCode'] == sCode].tolist()[0]
def yrFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioYear']
def codeFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioCode']
def yearFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioYear']

#quick tests of functions
#display(idFromCode('50RTP'))
#display(yrFromCode('50RTP'))
#display(codeFromID(4))
#display(yearFromID(4))
#display(df_Scenarios.loc[7]['ScenarioCode'])

# Define Desired V/C Groupings

In [19]:
#season groups

df_VCGroup_Seasons = pd.DataFrame()

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Ann',
                                                'SeasonName'    : 'Annual',
                                                'SEASONGROUP'   : 'S00-Ann',
                                                'SeasonType'    : 'Season'
                                               },ignore_index=True)

#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Win',
#                                                'SeasonName'    : 'Winter',
#                                                'SEASONGROUP'   : 'S01-Win',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Spr',
#                                                'SeasonName'    : 'Spring',
#                                                'SEASONGROUP'   : 'S02-Spr',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sum',
#                                                'SeasonName'    : 'Summer',
#                                                'SEASONGROUP'   : 'S03-Sum',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Fal',
#                                                'SeasonName'    : 'Fall',
#                                                'SEASONGROUP'   : 'S04-Fal',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jan',
#                                                'SeasonName'    : 'January',
#                                                'SEASONGROUP'   : 'M01-Jan',
#                                                'SeasonType'    : 'Month',
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Feb',
#                                                'SeasonName'    : 'February',
#                                                'SEASONGROUP'   : 'M02-Feb',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Mar',
#                                                'SeasonName'    : 'March',
#                                                'SEASONGROUP'   : 'M03-Mar',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Apr',
#                                                'SeasonName'    : 'April',
#                                                'SeasonGroup'   : 'M04-Apr',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'May',
#                                                'SeasonName'    : 'May',
#                                                'SEASONGROUP'   : 'M05-May',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jun',
#                                                'SeasonName'    : 'June',
#                                                'SEASONGROUP'   : 'M06-Jun',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jul',
#                                                'SeasonName'    : 'July',
#                                                'SEASONGROUP'   : 'M07-Jul',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Aug',
#                                                'SeasonName'    : 'August',
#                                                'SEASONGROUP'   : 'M08-Aug',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sep',
#                                                'SeasonName'    : 'September',
#                                                'SEASONGROUP'   : 'M09-Sep',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Oct',
#                                                'SeasonName'    : 'October',
#                                                'SEASONGROUP'   : 'M10-Oct',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Nov',
#                                                'SeasonName'    : 'November',
#                                                'SEASONGROUP'   : 'M11-Nov',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Dec',
#                                                'SeasonName'    : 'December',
#                                                'SEASONGROUP'   : 'M12-Dec',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)

display(df_VCGroup_Seasons)

#day-of-week groups

df_VCGroup_DOWPkHr = pd.DataFrame()

df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'WkAM',
                                                'DOWPkName' : 'AM - Weekday',
                                                'DOWGROUP'  : '1-Weekday (Tu-Th)',
                                                'PERIOD'    : 'AM',
                                                'HRPCTOF'   : 'Prd' #calculate peak Hr using Prd volume
                                               },ignore_index=True)

df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'WkMD',
                                                'DOWPkName' : 'Midday - Weekday',
                                                'DOWGROUP'  : '1-Weekday (Tu-Th)',
                                                'PERIOD'    : 'MD',
                                                'HRPCTOF'   : 'Prd' #calculate peak Hr using Prd volume
                                               },ignore_index=True)

df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'WkPM',
                                                'DOWPkName' : 'PM - Weekday',
                                                'DOWGROUP'  : '1-Weekday (Tu-Th)',
                                                'PERIOD'    : 'PM',
                                                'HRPCTOF'   : 'Prd' #calculate peak Hr using Prd volume
                                               },ignore_index=True)

#df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'FrPM',
#                                                'DOWPkName' : 'PM - Friday',
#                                                'DOWGROUP'  : '5-Friday',
#                                                'PERIOD'    : 'PM',
#                                                'HRPCTOF'   : 'DY' #calculate peak Hr using DY volume
#                                               },ignore_index=True)
#
#df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'SaDY',
#                                                'DOWPkName' : 'Saturday',
#                                                'DOWGROUP'  : '6-Saturday',
#                                                'PERIOD'    : 'DY',
#                                                'HRPCTOF'   : 'DY' #calculate peak Hr using DY volume
#                                               },ignore_index=True)
#
#df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'SuDY',
#                                                'DOWPkName' : 'Sunday',
#                                                'DOWGROUP'  : '7-Sunday',
#                                                'PERIOD'    : 'DY',
#                                                'HRPCTOF'   : 'DY' #calculate peak Hr using DY volume
#                                               },ignore_index=True)

display(df_VCGroup_DOWPkHr)

#directional factor when using DY volumes for peak Hr calcs
#taken from HCM 2000 Exhibit 9-2
dDYDirectionalFactor = 0.6

#directional factor not used in Prd volume calcs, since those include directionality

,SeasonCode,SeasonName,SEASONGROUP,SeasonType
0,Ann,Annual,S00-Ann,Season


,DOWPkCode,DOWPkName,DOWGROUP,PERIOD,HRPCTOF
0,WkAM,AM - Weekday,1-Weekday (Tu-Th),AM,Prd
1,WkMD,Midday - Weekday,1-Weekday (Tu-Th),MD,Prd
2,WkPM,PM - Weekday,1-Weekday (Tu-Th),PM,Prd


In [20]:
import itertools

#create df with all possible season and day-of-weak/peak groups
df_VCGroups = pd.DataFrame(list(itertools.product(df_VCGroup_Seasons['SeasonCode'],df_VCGroup_DOWPkHr['DOWPkCode'])),columns=['SeasonCode','DOWPkCode'])

df_VCGroups = pd.DataFrame(df_VCGroups)
pd.set_option('display.max_rows', df_VCGroups.shape[0]+1)

#display(df_VCGroups)

#add season data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_Seasons, on='SeasonCode')

#add dow-peak data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_DOWPkHr, on='DOWPkCode')

#set id name
df_VCGroups.index.name = 'VCGroupID'

df_VCGroups['VCGroupCode'] = df_VCGroups['SeasonCode'] + df_VCGroups['DOWPkCode']

df_VCGroups = df_VCGroups.drop(columns=['SeasonCode','DOWPkCode','SeasonName','DOWPkName'])

display(df_VCGroups)

,SEASONGROUP,SeasonType,DOWGROUP,PERIOD,HRPCTOF,VCGroupCode
VCGroupID,,,,,,
0,S00-Ann,Season,1-Weekday (Tu-Th),AM,Prd,AnnWkAM
1,S00-Ann,Season,1-Weekday (Tu-Th),MD,Prd,AnnWkMD
2,S00-Ann,Season,1-Weekday (Tu-Th),PM,Prd,AnnWkPM


# TDM SEGID Detail

In [21]:
#column names from TDM
nameSegID      = 'SEGID'
nameFG         = 'FUNCGROUP'
nameLinks      = 'LINKS'
nameLanes      = 'LANES'
nameVol        = 'VOL'
nameCap1HL     = 'CAP1HL'
nameAT         = 'AREATYPE'
nameFT         = 'FT'
nameFTClass    = 'FTCLASS'
nameFGFac      = 'FGFAC'
namePrdFac     = 'PRDFAC'
nameDYVOL      = 'DY_VOL'
nameDirections = 'DIRECTIONS'
nameDFac       = 'DFAC'
nameMDPct      = 'MDPCT'
nameHVPct      = 'HVPCT'
nameSegTotal   = 'SegDYVol_TDM'

#column for scenario year
nameSID = 'ScenarioID'
nameS =   'ScenarioName'
nameSYr = 'ScenarioYear'

#additional column name variables
nameLT = 'LT'
nameMD = 'MD'
nameHV = 'HV'

nameVolType = 'VolType'
namePrd     = 'Prd'
nameCapPer  = 'CapPer'
nameCap1Hr  = 'Cap1Hr'
nameDir     = 'Dir'

#value for functional group (FG) total row, so that it can be removed
nameFGTotalValue = 'Total'

#arrays for values by direction

aNameDir1Columns   = ['D1_'+nameFT,'D1_'+nameFTClass,'D1_'+nameLanes,'D1_'+nameCap1HL]
aNameDir2Columns   = ['D2_'+nameFT,'D2_'+nameFTClass,'D2_'+nameLanes,'D2_'+nameCap1HL]
aNameDirColumns    = [      nameFT,      nameFTClass,      nameLanes,      nameCap1HL]    
aNameDirIndexNoDir = [nameSegID,nameFG]
aNameDirIndexWiDir = [nameSegID,nameFG,nameDir]

#arrays for values by Prd
aNamePrdFac = ['DY_PRDFAC','AM_PRDFAC','MD_PRDFAC','PM_PRDFAC','EV_PRDFAC'] #add DY_PRDFAC = 1 later in script for calc ease

#arrays for values by direction, Prd
aNameDirPrdDFac      = ['D1_DY_'+nameDFac     ,'D1_AM_'+nameDFac     ,'D1_MD_'+nameDFac     ,'D1_PM_'+nameDFac     ,'D1_EV_'+nameDFac     ,'D2_DY_'+nameDFac     ,'D2_AM_'+nameDFac     ,'D2_MD_'+nameDFac     ,'D2_PM_'+nameDFac     ,'D2_EV_'+nameDFac     ]
aNameDirPrdMDPct = ['D1_DY_'+nameMDPct,'D1_AM_'+nameMDPct,'D1_MD_'+nameMDPct,'D1_PM_'+nameMDPct,'D1_EV_'+nameMDPct,'D2_DY_'+nameMDPct,'D2_AM_'+nameMDPct,'D2_MD_'+nameMDPct,'D2_PM_'+nameMDPct,'D2_EV_'+nameMDPct]
aNameDirPrdHVPct = ['D1_DY_'+nameHVPct,'D1_AM_'+nameHVPct,'D1_MD_'+nameHVPct,'D1_PM_'+nameHVPct,'D1_EV_'+nameHVPct,'D2_DY_'+nameHVPct,'D2_AM_'+nameHVPct,'D2_MD_'+nameHVPct,'D2_PM_'+nameHVPct,'D2_EV_'+nameHVPct]

indexTotal             = [nameSegID]
indexMelt              = [nameSegID,nameFG]
#indexDY             = [nameSegID,nameFG,nameCap1HL,nameLinks,nameLanes,nameAT,nameFT]

#Hrs within Prd, for capacity
dPrdDuration = {
     'PERIOD': ['AM','MD','PM','EV','DY'],
     'PRDHRS': [   3,   6,   3,  12,     24]
     
}
df_PrdDuration = pd.DataFrame(dPrdDuration)

In [40]:
dfs_TDM =[]
dfs_TDM_Total = []

#all scenarios will be read in to allow for pivoting

for (idx, row) in df_Scenarios.iterrows():

    print(row.loc[nameS] + ' CSV Processing')

    csvFileName = os.path.join(dirTDM, row.SegSummaryCSV)

    df_TDM_import = pd.read_csv(csvFileName)
    df_TDM_import[nameFG] = df_TDM_import[nameFG].str.strip() #strip leading and trailing spaces
    
    #totals for DY segment volume
    df_TDM_SegDYVol = df_TDM_import[df_TDM_import[nameFG] == nameFGTotalValue].copy()
    df_TDM_SegDYVol = df_TDM_SegDYVol[[nameSegID,nameDYVOL]]
    df_TDM_SegDYVol = df_TDM_SegDYVol.rename(columns={nameDYVOL: nameSegTotal})
    
    dfs_TDM_Total.append(df_TDM_SegDYVol[[nameSegID,nameSegTotal]])
    
    #display(df_TDM_SegDYVol)
    
    #Remove rows with no links and total
    df_TDM_import         = df_TDM_import[df_TDM_import[nameLinks] > 0                ]
    df_TDM_import         = df_TDM_import[df_TDM_import[nameFG]    != nameFGTotalValue]

    #data with segment level values
    df_TDM_Seg = pd.DataFrame.merge(df_TDM_import[[nameSegID,nameFG,nameAT,nameFGFac]],df_TDM_SegDYVol,on=nameSegID)    
    
    #normalize data with direction fields
    df_Dir1Columns          = df_TDM_import[aNameDirIndexNoDir + aNameDir1Columns].copy()
    df_Dir1Columns.columns  = (aNameDirIndexNoDir + aNameDirColumns)
    df_Dir1Columns[nameDir] = 'D1'
    
    df_Dir2Columns          = df_TDM_import[aNameDirIndexNoDir + aNameDir2Columns].copy()
    df_Dir2Columns.columns  = (aNameDirIndexNoDir + aNameDirColumns)
    df_Dir2Columns[nameDir] = 'D2'
    
    #display(df_Dir1Columns)
    #display(df_Dir2Columns)
    
    df_TDM_Dir = pd.concat([df_Dir1Columns,df_Dir2Columns])
    #display(df_Dir)
    
    #normalize data with Prd fields
    df_TDM_import['DY_PRDFAC'] = 1
    df_TDM_PrdFac = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNamePrdFac)

    #directional factors
    df_TDM_PrdFac[namePrd] = df_TDM_PrdFac['variable'].str.get(0) + df_TDM_PrdFac['variable'].str.get(1)
    df_TDM_PrdFac          = df_TDM_PrdFac.rename(columns={'value': namePrdFac})
    df_TDM_PrdFac          = df_TDM_PrdFac.drop(columns=['variable'])
    
    df_TDM_Prd = df_TDM_PrdFac
    
    #display(df_TDM_Prd)
    
    #normalize data with dir/Prd fields
    df_TDM_DFac      = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPrdDFac     )
    df_TDM_MDPct = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPrdMDPct)
    df_TDM_HVPct = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPrdHVPct)

    #directional factors
    df_TDM_DFac[nameDir] = df_TDM_DFac['variable'].str.get(0) + df_TDM_DFac['variable'].str.get(1)
    df_TDM_DFac[namePrd] = df_TDM_DFac['variable'].str.get(3) + df_TDM_DFac['variable'].str.get(4)
    df_TDM_DFac          = df_TDM_DFac.rename(columns={'value': nameDFac})
    df_TDM_DFac          = df_TDM_DFac.drop(columns=['variable'])
    
    #display(df_TDM_DFac)
    
    #medium truck Pcts
    df_TDM_MDPct[nameDir] = df_TDM_MDPct['variable'].str.get(0) + df_TDM_MDPct['variable'].str.get(1)
    df_TDM_MDPct[namePrd] = df_TDM_MDPct['variable'].str.get(3) + df_TDM_MDPct['variable'].str.get(4)
    df_TDM_MDPct          = df_TDM_MDPct.rename(columns={'value': nameMDPct})
    df_TDM_MDPct          = df_TDM_MDPct.drop(columns=['variable'])
    
    #display(df_TDM_MDPct)    
    
    #medium truck Pcts
    df_TDM_HVPct[nameDir] = df_TDM_HVPct['variable'].str.get(0) + df_TDM_HVPct['variable'].str.get(1)
    df_TDM_HVPct[namePrd] = df_TDM_HVPct['variable'].str.get(3) + df_TDM_HVPct['variable'].str.get(4)
    df_TDM_HVPct          = df_TDM_HVPct.rename(columns={'value': nameHVPct})
    df_TDM_HVPct          = df_TDM_HVPct.drop(columns=['variable'])
    
    #display(df_TDM_HVPct)
    
    #merge dir/Prd dataframes
    df_TDM_DirPrd = pd.DataFrame.merge(df_TDM_DFac  ,df_TDM_MDPct,on=(nameSegID,nameFG,nameDir,namePrd))
    df_TDM_DirPrd = pd.DataFrame.merge(df_TDM_DirPrd,df_TDM_HVPct,on=(nameSegID,nameFG,nameDir,namePrd))
    
    #display(df_DirPrd)
    
    df_TDM_byFGDirPrd = pd.DataFrame.merge(df_TDM_Seg          ,df_TDM_Dir   ,on=(nameSegID,nameFG)) #merge direction
    df_TDM_byFGDirPrd = pd.DataFrame.merge(df_TDM_byFGDirPrd,df_TDM_Prd   ,on=(nameSegID,nameFG)) #merge Prd
    df_TDM_byFGDirPrd = pd.DataFrame.merge(df_TDM_byFGDirPrd,df_TDM_DirPrd,on=(nameSegID,nameFG,namePrd,nameDir)) #merge Prd/direction

    #calculate DY directional for peak-Hr-Pct-of-DY calcs
    #assumes when DY volumes is used, TDM directional factor not valid, eg. weekends    
    #both directions use same directional factor
    df_TDM_byFGDirPrd.loc[(df_TDM_byFGDirPrd[namePrd] == 'DY'), nameDFac] = dDYDirectionalFactor
    df_TDM_byFGDirPrd.loc[(df_TDM_byFGDirPrd[namePrd] == 'DY'), namePrd] = 'DY'
        
    df_TDM_byFGDirPrd = df_TDM_byFGDirPrd[[nameSegID,nameAT,nameSegTotal,nameFG,nameFGFac,nameDir,nameLanes,nameFT,nameFTClass,nameCap1HL,nameDFac,namePrd,namePrdFac,nameMDPct,nameHVPct]]
    display(df_TDM_byFGDirPrd)

    dfs_TDM.append(df_TDM_byFGDirPrd)

    df_Check = df_TDM_byFGDirPrd.copy()
    df_Check['FactorsCombined'] = df_Check[nameFGFac] * df_Check[nameDFac] * df_Check[namePrdFac]
    
    #The sum of all factors should 2.2 (1.0 for non-total and 2xDFac(0.6) for totals, add little extra on each side for rounding
    df_Check = df_Check.groupby(indexTotal).agg({'FactorsCombined':[np.sum]})
    df_Check.columns = df_Check.columns.droplevel(1)
    df_Check = df_Check[((df_Check['FactorsCombined'] > 2.2003) | (df_Check['FactorsCombined'] < 2.1997) & (df_Check['FactorsCombined'] != 0))]
    display(print(row.loc[nameS] + ' Checks:'))
    display(df_Check)

print('Done')

2019 Base CSV Processing


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT
0,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,11.9,8.4
1,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6045,AM,0.2037,9.1,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43188,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.5842,PM,0.2619,7.7,1.9
43189,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.5353,EV,0.2535,8.5,2.3


2019 Base Checks:


None

,FactorsCombined
SEGID,


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,12.2,8.6
1,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6145,AM,0.1973,9.0,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52668,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,0.5298,PM,0.2887,9.5,5.6
52669,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,0.5584,EV,0.2322,13.6,3.8


2050 Needs-Based Scenario - MAG Checks:


None

,FactorsCombined
SEGID,


Done


In [41]:
df_TDM_byFGDirPrd[df_TDM_byFGDirPrd[nameSegID]=='2878_005.4']

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT
32160,2878_005.4,4.0,39215.3,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,11.9,5.8
32161,2878_005.4,4.0,39215.3,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5942,AM,0.1735,7.9,3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32168,2878_005.4,4.0,39215.3,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6026,PM,0.2297,8.8,4.5
32169,2878_005.4,4.0,39215.3,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6536,EV,0.2674,13.6,7.3


In [42]:
#df_TDM_Seg[df_TDM_Seg[nameSegID]=='0015_282.2']

In [43]:
df_TDM_byFGDirPrd

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,12.2,8.6
1,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6145,AM,0.1973,9.0,7.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52668,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,0.5298,PM,0.2887,9.5,5.6
52669,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,0.5584,EV,0.2322,13.6,3.8


In [44]:
df_Scenarios_toProcess

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
...,...,...,...,...,...,...,...,...
14,UF50,2050 Needs-Based Scenario,2050,Needs,v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,UF42,RTP50
15,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50


In [45]:
df_Scenarios_toAdjust

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
6,NB3228,2032 SE on a 2028 Network Scenario,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,NB4228,2042 SE on a 2028 Network Scenario,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,NB3228,RTP42
...,...,...,...,...,...,...,...,...
14,UF50,2050 Needs-Based Scenario,2050,Needs,v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,UF42,RTP50
15,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50


In [46]:
#calculate adjustment for forecast for those non-RTP scenarios

dfs_ForecastAdj = []

for (idx, row) in df_Scenarios_toAdjust.iterrows():

    #print(row.loc[nameS] + ' CSV Importing')
    
    df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]
    df_base.columns = (['SEGID','DYVolBase'])
    df_scenario  = dfs_TDM_Total[idFromCode(row.loc['ScenarioCode'])]
    df_scenario.columns = (['SEGID','DYVolScenario'])
    
    display(df_base)
    display(df_scenario)
    
    df_ForecastAdj = pd.DataFrame.merge(df_scenario,df_base,on=nameSegID,how='left')
    df_ForecastAdj['DYVolAdj'] = df_ForecastAdj['DYVolScenario'] - df_ForecastAdj['DYVolBase']
    
    df_ForecastAdj.to_csv(os.path.join(dirResults,'ForecastAdj_' + row.loc['ScenarioCode'] + '.csv'))
    
    display(df_ForecastAdj)
    
    dfs_ForecastAdj.append(df_ForecastAdj)

,SEGID,DYVolBase
0,0006_141.0,1871.0
5,0006_146.9,1882.8
...,...,...
22880,WFRC_8473,7350.7
22885,WFRC_8476,7305.9


,SEGID,DYVolScenario
0,0006_141.0,1832.0
5,0006_146.9,1840.8
...,...,...
21925,WFRC_8473,7396.1
21930,WFRC_8476,8677.3


,SEGID,DYVolScenario,DYVolBase,DYVolAdj
0,0006_141.0,1832.0,1871.0,-39.0
1,0006_146.9,1840.8,1882.8,-42.0
...,...,...,...,...
4385,WFRC_8473,7396.1,7350.7,45.4
4386,WFRC_8476,8677.3,7305.9,1371.4


,SEGID,DYVolBase
0,0006_141.0,2115.0
5,0006_146.9,2122.0
...,...,...
23355,WFRC_8473,8319.2
23360,WFRC_8476,8725.1


,SEGID,DYVolScenario
0,0006_141.0,2053.0
5,0006_146.9,2059.0
...,...,...
21925,WFRC_8473,8380.0
21930,WFRC_8476,10495.1


...

,SEGID,DYVolScenario
0,0006_141.0,2346.0
5,0006_146.9,2351.0
...,...,...
25170,WFRC_8473,9087.7
25175,WFRC_8476,14662.0


,SEGID,DYVolScenario,DYVolBase,DYVolAdj
0,0006_141.0,2346.0,2379.0,-33.0
1,0006_146.9,2351.0,2384.0,-33.0
...,...,...,...,...
5034,WFRC_8473,9087.7,9190.2,-102.5
5035,WFRC_8476,14662.0,15021.1,-359.1


,SEGID,DYVolBase
0,0006_141.0,2379.0
5,0006_146.9,2384.0
...,...,...
24510,WFRC_8473,9190.2
24515,WFRC_8476,15021.1


,SEGID,DYVolScenario
0,0006_141.0,2362.0
5,0006_146.9,2365.0
...,...,...
25175,WFRC_8473,9103.8
25180,WFRC_8476,14314.3


,SEGID,DYVolScenario,DYVolBase,DYVolAdj
0,0006_141.0,2362.0,2379.0,-17.0
1,0006_146.9,2365.0,2384.0,-19.0
...,...,...,...,...
5035,WFRC_8473,9103.8,9190.2,-86.4
5036,WFRC_8476,14314.3,15021.1,-706.8


# AADT and AvgAnnualPrdVol Forecasts (Segments, Forecasts, TDM)

In [47]:
#import arcgis libraries
from arcgis.gis import *
gis = GIS()
                        
#import into spatially-enabled DataFrame
sdf_Forecasts = pd.DataFrame.spatial.from_featureclass(shp_Forecasts)
sdf_Forecasts

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0,0023_000.0,0023,0.000,1.564,1.585102,5.0,2.0,Cache,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422233.4001000002, 4607564.0001],..."
1,1,0,0023_001.6,0023,1.564,2.146,0.580084,5.0,2.0,Cache,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422182.7999999998, 4610094.1], [4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5836,5836,0,WFRC_8475,WFRC,0.000,0.000,0.587918,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[422006.9967, 4481267.654200001], ..."
5837,5837,0,WFRC_8476,WFRC,0.000,0.000,0.589436,35.0,1.0,WFRC,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[422189.63999999966, 4480431.60999..."


In [48]:
for col in sdf_Forecasts.columns: 
    print(col) 

FID
Id
SEGID
ROUTE
BMP
EMP
DISTANCE
CO_FIPS
SUBAREAID
PLANAREA
F2019
F2023
F2028
F2032
F2042
F2050
CH19TO50
CH19TO23
CH23TO28
CH28TO32
CH32TO42
CH42TO50
AADTSTN
AADT2022
AADT2021
AADT2020
AADT2019
AADT2018
AADT2017
AADT2016
AADT2015
AADT2014
AADT2013
AADT2012
AADT2011
AADT2010
AADT2009
AADT2008
AADT2007
AADT2006
AADT2005
AADT2004
AADT2003
AADT2002
AADT2001
AADT2000
AADT1999
AADT1998
AADT1997
AADT1996
AADT1995
AADT1994
AADT1993
AADT1992
AADT1991
AADT1990
AADT1989
AADT1988
AADT1987
AADT1986
AADT1985
AADT1984
AADT1983
AADT1982
AADT1981
SUTRK2022
CUTRK2022
SUTRK2021
CUTRK2021
SUTRK2020
CUTRK2020
SUTRK2019
CUTRK2019
SUTRK2018
CUTRK2018
SUTRK2017
CUTRK2017
SUTRK2016
CUTRK2016
SUTRK2015
CUTRK2015
SUTRK2014
CUTRK2014
SUTRK2013
CUTRK2013
SUTRK2012
CUTRK2012
SUTRK2011
CUTRK2011
SUTRK2010
CUTRK2010
CCSGROUP19
FAC_MON
FAC_TUE
FAC_WED
FAC_THU
FAC_FRI
FAC_SAT
FAC_SUN
FAC_WDAVG
FAC_WEAVG
FAC_WEMAX
FAC_SPR
FAC_SUM
FAC_FAL
FAC_WIN
FAC_JAN
FAC_FEB
FAC_MAR
FAC_APR
FAC_MAY
FAC_JUN
FAC_JUL
FAC_AUG
FAC_SEP


In [49]:
df = sdf_Forecasts

df = df[(df[nameSegID]=='2878_005.4')]

df

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
3606,3606,0,2878_005.4,2878,5.38,5.875,0.565442,49.0,1.0,MAG,...,0.996,1.0223,1.0238,1.0103,0.9944,0.9907,5.0,1.0245,0.0,"{""paths"": [[[424758.85199999996, 4473905.148],..."


In [50]:
#normalize data with forecasts

meltIndex    = ['SEGID']
valueColumns = ['AADT2019','F2023','F2028','F2032','F2042','F2050']
df_Forecasts = pd.melt(sdf_Forecasts, id_vars=meltIndex, value_vars=valueColumns)
df_Forecasts.columns = ('SEGID','FCOL','ForecastAADT')
df_Forecasts['FYEAR'] = (df_Forecasts['FCOL'].str[-4:]).astype('int64')
df_Forecasts['ForecastAADTAdj'] = 0 #placeholder column to preserve location
df_Forecasts['AvgAnnualPrdVol'] = 0 #placeholder column to preserve location
df_Forecasts = pd.merge(df_Forecasts,df_SegmentsMatchedToFactorGroups, on='SEGID')
df_Forecasts = df_Forecasts[['SEGID','FYEAR','ForecastAADT','ForecastAADTAdj','AvgAnnualPrdVol','FACGEO']]
df_Forecasts

,SEGID,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
0,0006_141.0,2019,1517.0,0,0,Statewide
1,0006_141.0,2023,1600.0,0,0,Statewide
...,...,...,...,...,...,...
32092,WFRC_8476,2042,0.0,0,0,Statewide
32093,WFRC_8476,2050,0.0,0,0,Statewide


In [51]:
df_Forecasts[(df_Forecasts[nameSegID]=='2878_005.4')]

,SEGID,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
18702,2878_005.4,2019,1106.0,0,0,Statewide
18703,2878_005.4,2023,39500.0,0,0,Statewide
...,...,...,...,...,...,...
18706,2878_005.4,2042,31500.0,0,0,Statewide
18707,2878_005.4,2050,26000.0,0,0,Statewide


In [52]:
df_Scenarios_toProcess

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
...,...,...,...,...,...,...,...,...
14,UF50,2050 Needs-Based Scenario,2050,Needs,v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,UF42,RTP50
15,UF50_MAG,2050 Needs-Based Scenario - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF42_MAG,RTP50


In [53]:
#Hrs within Prd, for capacity
dRoundForecast = {
     'RoundIndex'  : [ 1,  2,   3,    4,     5],
     'ForecastBin' : [ 0,100,1000,10000,100000],
     'RoundToValue': [10, 50, 100,  500,  1000]
}
df_RoundForecast = pd.DataFrame(dRoundForecast)
display(df_RoundForecast)

lRoundBins = df_RoundForecast['ForecastBin'].to_list()
lRoundBase = df_RoundForecast['RoundToValue'].to_list()

def forecast_round(x):
    base = lRoundBase[np.digitize([x],lRoundBins)[0]-1]
    return int(base * round(float(x)/base))

#df = pd.Series([11,16,21]).apply(lambda x: custom_round(x, base=5))

,RoundIndex,ForecastBin,RoundToValue
0,1,0,10
1,2,100,50
...,...,...,...
3,4,10000,500
4,5,100000,1000


In [54]:
display(lRoundBase[np.digitize([1],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([54],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([123],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([1254],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([15456],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([2897651],lRoundBins)[0]-1])
display(lRoundBase[np.digitize([1564654654654],lRoundBins)[0]-1])

display(forecast_round(1))
display(forecast_round(54))
display(forecast_round(123))
display(forecast_round(1254))
display(forecast_round(15456))
display(forecast_round(2897651))
display(forecast_round(1564654654654))

10

10

50

100

500

...

100

1300

15500

2898000

1564654655000

In [55]:
#calculate forecast DY volume for each scenario to process

dfs_PrdForecasts = []
df_Forecasts_SYEAR_tocsv = pd.DataFrame()

for (idx, row) in df_Scenarios_toProcess.iterrows():

    _scenarioID = idx
    
    print(row.loc[nameS] + ' Processing')
    print(idx)
    #df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]

    #display(dfs_TDM[idx])
    
    df_TDM = dfs_TDM[idx]
    
    #get TDM year
    _scenarioyear = yearFromID(_scenarioID)
    
    #filter only for year of forecast
    
    df_Forecasts['ForecastAADTAdj'] = df_Forecasts['ForecastAADT'] # set adjusted volume so value is set for unadjusted scenarios
    
    df_Forecasts_SYEAR = df_Forecasts[df_Forecasts.FYEAR == _scenarioyear]
    
    #adjust forecast
    
    for (idx, row) in df_Scenarios_toAdjust.iterrows():
        if (_scenarioID == idx):
            #display("adjust this")
            
            #get adjust id
            _adjustId = df_Scenarios_toAdjust.index.get_loc(idx)

            #display(_adjustId)
            #display(dfs_ForecastAdj[_adjustId])
            
            df_Forecasts_Adjust = pd.DataFrame.merge(df_Forecasts_SYEAR,dfs_ForecastAdj[_adjustId],on='SEGID')
            
            #adjust volume. if scenario vol = 0 then zero out forecast volume, otherwise forecast volume is adjusted
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DYVolScenario'] == 0), 'ForecastAADTAdj'] = 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DYVolScenario']  > 0), 'ForecastAADTAdj'] = df_Forecasts_Adjust['ForecastAADT'] + df_Forecasts_Adjust['DYVolAdj'] 
            
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DYVolScenario'] == 0), 'ForecastAADTAdj'] = 0
            
            #if adjustment < 0 then bring to 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['ForecastAADTAdj']   < 0), 'ForecastAADTAdj'] = 0  
            
            #display(df_Forecasts_Adjust) 
            
            df_Forecasts_Adjust = df_Forecasts_Adjust[['SEGID','FYEAR','ForecastAADT','ForecastAADTAdj','AvgAnnualPrdVol','FACGEO']]
            #df_Forecasts_Adjust = df_Forecasts_Adjust.rename(columns={"DYVolBase": "ForecastAADT"})
            
            #display(df_Forecasts_Adjust['ForecastAADTAdj'])
            df_Forecasts_Adjust['RoundBin'] = np.digitize(df_Forecasts_Adjust['ForecastAADTAdj'],lRoundBins)
            df_Forecasts_Adjust_forRounding = pd.DataFrame.merge(df_Forecasts_Adjust,df_RoundForecast,left_on='RoundBin',right_on='RoundIndex')
            df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded'] = round((df_Forecasts_Adjust_forRounding['ForecastAADTAdj']/df_Forecasts_Adjust_forRounding['RoundToValue']),0)*df_Forecasts_Adjust_forRounding['RoundToValue']
            df_Forecasts_Adjust_forRounding = df_Forecasts_Adjust_forRounding.fillna(0)
            df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded'] = df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded'].astype(int)
            #display(df_Forecasts_Adjust_forRounding[['ForecastAADTAdj','RoundBin','RoundToValue','ForecastAADTAdjRounded']])
            #display(df_Forecasts_Adjust_forRounding[df_Forecasts_Adjust_forRounding['SEGID']=='0091_002.0'])
                    
            df_Forecasts_Adjust_forRounding['ForecastAADTAdj'] = df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded']
            
            df_Forecasts_Adjust_forRounding = df_Forecasts_Adjust_forRounding.drop(columns=['RoundIndex','ForecastBin','RoundBin','RoundToValue','ForecastAADTAdjRounded'])
            
            df_Forecasts_SYEAR = df_Forecasts_Adjust_forRounding.copy()      
            #display(df_Forecasts_SYEAR[df_Forecasts_SYEAR['SEGID']=='0091_002.0'])
            
    df_Forecasts_SYEAR = df_Forecasts_SYEAR.fillna(0)
    #df_Forecasts_SYEAR['ForecastAADT'] = df_Forecasts_SYEAR['ForecastAADT'].astype(int)
    #df_Forecasts_SYEAR['ForecastAADTAdj'] = df_Forecasts_SYEAR['ForecastAADTAdj'].astype(int)
    
    #display(df_Forecasts_SYEAR)

    df_Forecasts_SYEAR_export = df_Forecasts_SYEAR.copy()
    df_Forecasts_SYEAR_export['ScenarioCode'] = codeFromID(_scenarioID)
    df_Forecasts_SYEAR_export['ForecastAADT'] = df_Forecasts_SYEAR_export['ForecastAADTAdj'] #override for export
    df_Forecasts_SYEAR_export = df_Forecasts_SYEAR_export[['ScenarioCode','SEGID','ForecastAADT']]

    #display(df_Forecasts_SYEAR_export[df_Forecasts_SYEAR_export['SEGID']=='0091_002.0'])
    
    df_Forecasts_SYEAR_tocsv = pd.concat([df_Forecasts_SYEAR_tocsv,df_Forecasts_SYEAR_export])
    
    #display(os.path.join(dirResults + "0_Forecast_" + codeFromID(_scenarioID) + ".csv"))
    #df_Forecasts_SYEAR_export.to_csv(os.path.join(dirResults + r"\\0_Forecast_" + codeFromID(_scenarioID) + ".csv"),index=False)
    
    #join with TDM data
    df_Forecasts_withPrd = pd.DataFrame.merge(df_TDM, df_Forecasts_SYEAR, on=('SEGID'))
    
    df_Forecasts_withPrd['AvgAnnualPrdVol'] = df_Forecasts_withPrd['ForecastAADTAdj'] * df_Forecasts_withPrd[nameFGFac] * df_Forecasts_withPrd[nameDFac] * df_Forecasts_withPrd[namePrdFac]
    
    #display(df_Forecasts_withPrd.head(20))
    
    dfs_PrdForecasts.append(df_Forecasts_withPrd)

display(os.path.join(dirResults + r"\\0_Forecasts.csv"))
df_Forecasts_SYEAR_tocsv.to_csv(os.path.join(dirResults + r"\\0_Forecasts.csv"),index=False)
display('CSV Exported')

2019 Base Processing
0
2023 Base Processing
1
2028 TIP Projects Scenario Processing
2
2032 Fiscally-Constrained Scenario Processing
3
2042 Fiscally-Constrained Scenario Processing
4
2050 Fiscally-Constrained Scenario Processing
5
2032 SE on a 2028 Network Scenario Processing
6
2042 SE on a 2028 Network Scenario Processing
7
2050 SE on a 2028 Network Scenario Processing
8
2042 SE on a 2032 Network Processing
9
2050 SE on a 2042 Network Processing
10
2050 SE on a 2042 Network Processing
11
2032 Needs-Based Scenario Processing
12
2042 Needs-Based Scenario Processing
13
2050 Needs-Based Scenario Processing
14
2050 Needs-Based Scenario - MAG Processing
15


'e:\\GitHub\\V-over-C-Calculations\\results\\\\0_Forecasts.csv'

'CSV Exported'

In [56]:
df_Forecasts_SYEAR[df_Forecasts_SYEAR['SEGID']=='2878_005.4']

,SEGID,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
3907,2878_005.4,2050,26000.0,23000,0,Statewide


In [57]:
#format forecasts as table for joining with other analysis
table = pd.pivot_table(df_Forecasts_SYEAR_tocsv, values='ForecastAADT', index=['SEGID'],
                    columns=['ScenarioCode'], aggfunc=np.sum, fill_value=0)
display(table)

table.to_csv(os.path.join(dirResults + r"\\0_Forecasts_Table.csv"))

ScenarioCode,Base19,Base23,NB3228,NB4228,NB4232,NB5028,NB5032,NB5042,RTP32,RTP42,RTP50,TIP28,UF32,UF42,UF50,UF50_MAG
SEGID,,,,,,,,,,,,,,,,
0006_141.0,1517,1600,1800,2000,2100,2100,2700,2500,1800,2100,2300,1700,1800,2100,2300,2300
0006_146.9,1517,1600,1800,2000,2100,2100,2700,2500,1800,2100,2300,1800,1800,2100,2300,2300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WFRC_8475,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WFRC_8476,0,0,1400,1800,300,0,2100,300,0,0,0,0,0,1100,0,0


In [58]:
segid ='2878_005.4'
pd.set_option("max_rows", 13)
display(df_Forecasts_SYEAR_tocsv[df_Forecasts_SYEAR_tocsv[('SEGID')]==segid])

,ScenarioCode,SEGID,ForecastAADT
18702,Base19,2878_005.4,1106.0
18703,Base23,2878_005.4,39500.0
18704,TIP28,2878_005.4,35000.0
18705,RTP32,2878_005.4,28500.0
18706,RTP42,2878_005.4,31500.0
...,...,...,...
3778,NB5042,2878_005.4,27500.0
3735,UF32,2878_005.4,27000.0
3763,UF42,2878_005.4,24500.0
3910,UF50,2878_005.4,25500.0


In [59]:
dfs_PrdForecasts[0]

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
0,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,11.9,8.4,2019,1517.0,1517.0,910.200000,Statewide
1,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6045,AM,0.2037,9.1,8.2,2019,1517.0,1517.0,186.798298,Statewide
2,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5502,MD,0.3027,13.2,6.2,2019,1517.0,1517.0,252.649584,Statewide
3,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4749,PM,0.2656,10.7,8.3,2019,1517.0,1517.0,191.344428,Statewide
4,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5219,EV,0.2280,14.4,11.9,2019,1517.0,1517.0,180.512684,Statewide
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43185,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.6000,DY,1.0000,9.6,2.7,2019,0.0,0.0,0.000000,Statewide
43186,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.3390,AM,0.1635,12.0,3.8,2019,0.0,0.0,0.000000,Statewide
43187,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.4945,MD,0.3211,11.6,3.3,2019,0.0,0.0,0.000000,Statewide
43188,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.5842,PM,0.2619,7.7,1.9,2019,0.0,0.0,0.000000,Statewide


In [60]:
df = dfs_PrdForecasts[0]

pd.set_option('display.max_rows', 10)

df[(df[nameSegID]=='2878_005.4')].head(10)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
30790,2878_005.4,4.0,33688.9,Arterial,1.0,D1,2.0,3.0,Minor Arterial,815,0.6000,DY,1.0000,6.6,3.1,2019,1106.0,1106.0,663.600000,Statewide
30791,2878_005.4,4.0,33688.9,Arterial,1.0,D1,2.0,3.0,Minor Arterial,815,0.7681,AM,0.1894,3.1,1.2,2019,1106.0,1106.0,160.898823,Statewide
30792,2878_005.4,4.0,33688.9,Arterial,1.0,D1,2.0,3.0,Minor Arterial,815,0.5077,MD,0.2938,9.3,4.2,2019,1106.0,1106.0,164.973460,Statewide
30793,2878_005.4,4.0,33688.9,Arterial,1.0,D1,2.0,3.0,Minor Arterial,815,0.3755,PM,0.2755,7.0,3.4,2019,1106.0,1106.0,114.415977,Statewide
30794,2878_005.4,4.0,33688.9,Arterial,1.0,D1,2.0,3.0,Minor Arterial,815,0.5354,EV,0.2413,7.2,3.6,2019,1106.0,1106.0,142.886374,Statewide
30795,2878_005.4,4.0,33688.9,Arterial,1.0,D2,2.0,3.0,Minor Arterial,815,0.6000,DY,1.0000,8.2,3.6,2019,1106.0,1106.0,663.600000,Statewide
30796,2878_005.4,4.0,33688.9,Arterial,1.0,D2,2.0,3.0,Minor Arterial,815,0.2319,AM,0.1894,14.6,6.8,2019,1106.0,1106.0,48.577577,Statewide
30797,2878_005.4,4.0,33688.9,Arterial,1.0,D2,2.0,3.0,Minor Arterial,815,0.4923,MD,0.2938,10.4,5.1,2019,1106.0,1106.0,159.969340,Statewide
30798,2878_005.4,4.0,33688.9,Arterial,1.0,D2,2.0,3.0,Minor Arterial,815,0.6245,PM,0.2755,3.8,0.6,2019,1106.0,1106.0,190.287024,Statewide
30799,2878_005.4,4.0,33688.9,Arterial,1.0,D2,2.0,3.0,Minor Arterial,815,0.4646,EV,0.2413,9.6,4.8,2019,1106.0,1106.0,123.991426,Statewide


# Adjust for Season, DOW

In [61]:
display(df_VCGroup_Seasons)
display(df_VCGroup_DOWPkHr)
display(df_VCGroup_DOWPkHr[['DOWGROUP','PERIOD']].drop_duplicates())

,SeasonCode,SeasonName,SEASONGROUP,SeasonType
0,Ann,Annual,S00-Ann,Season


,DOWPkCode,DOWPkName,DOWGROUP,PERIOD,HRPCTOF
0,WkAM,AM - Weekday,1-Weekday (Tu-Th),AM,Prd
1,WkMD,Midday - Weekday,1-Weekday (Tu-Th),MD,Prd
2,WkPM,PM - Weekday,1-Weekday (Tu-Th),PM,Prd


,DOWGROUP,PERIOD
0,1-Weekday (Tu-Th),AM
1,1-Weekday (Tu-Th),MD
2,1-Weekday (Tu-Th),PM


In [62]:
#read in data from other notebook pre-processing
df_SeasonFactors = pd.read_csv(csv_SeasonFactors)
df_DOWFactors = pd.read_csv(csv_DOWFactors)
df_TODFactors = pd.read_csv(csv_TODFactors)

df_TODFactors.loc[(df_TODFactors['SEASONGROUP']=='Year'),'SEASONGROUP'] = 'S00-Ann'

df_SeasonFactors = pd.DataFrame.merge(df_SeasonFactors, df_VCGroup_Seasons['SEASONGROUP'].drop_duplicates(), on='SEASONGROUP')
df_DOWFactors = pd.DataFrame.merge(df_DOWFactors, df_VCGroup_DOWPkHr['DOWGROUP'].drop_duplicates(), on='DOWGROUP')
df_TODFactors = pd.DataFrame.merge(df_TODFactors, df_VCGroup_Seasons['SEASONGROUP'].drop_duplicates(), on='SEASONGROUP')
df_TODFactors = pd.DataFrame.merge(df_TODFactors, df_VCGroup_DOWPkHr[['DOWGROUP','PERIOD']].drop_duplicates(), on=('DOWGROUP','PERIOD'))

#hard coded for now
df_TODFactors = df_TODFactors[df_TODFactors['YEARGROUP']=='2015-2019']

df_SeasonFactors = df_SeasonFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_DOWFactors = df_DOWFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_TODFactors = df_TODFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})

display(df_SeasonFactors)
display(df_DOWFactors)
display(df_TODFactors)
display(df_TODFactors[df_TODFactors['SEASONGROUP']=='S00-Ann'])

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,STATIONGROUP,FACTOR,SEASONGROUP
0,CO0,1.0,S00-Ann
1,CO1,1.0,S00-Ann
2,CO2,1.0,S00-Ann
3,CO3,1.0,S00-Ann
4,CO4,1.0,S00-Ann
...,...,...,...
32,COW,1.0,S00-Ann
33,COX,1.0,S00-Ann
34,COY,1.0,S00-Ann
35,COZ,1.0,S00-Ann


,STATIONGROUP,FACTOR,DOWGROUP
0,CO0,0.715249,1-Weekday (Tu-Th)
1,CO1,0.824043,1-Weekday (Tu-Th)
2,CO2,0.790750,1-Weekday (Tu-Th)
3,CO3,0.853478,1-Weekday (Tu-Th)
4,CO4,0.904970,1-Weekday (Tu-Th)
...,...,...,...
32,COW,1.067677,1-Weekday (Tu-Th)
33,COX,1.087913,1-Weekday (Tu-Th)
34,COY,0.967245,1-Weekday (Tu-Th)
35,COZ,1.006499,1-Weekday (Tu-Th)


,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
4,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4054,Prd
12,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3652,Prd
19,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3819,Prd
28,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.5003,Prd
37,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4619,Prd
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
4,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4054,Prd
12,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3652,Prd
19,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3819,Prd
28,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.5003,Prd
37,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4619,Prd
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


In [63]:
df_FacGeoFunGroupAreaTypeVolume_ToStationGroup = pd.read_csv(csv_FacGeoFunGroupAreaTypeVolume_ToStationGroup)
display(df_FacGeoFunGroupAreaTypeVolume_ToStationGroup)
display(df_FacGeoFunGroupAreaTypeVolume_ToStationGroup[(df_FacGeoFunGroupAreaTypeVolume_ToStationGroup['FACGEO']=='Statewide') & (df_FacGeoFunGroupAreaTypeVolume_ToStationGroup['AREATYPE']==1) & (df_FacGeoFunGroupAreaTypeVolume_ToStationGroup['FUNCGROUP']=='Arterial')])

,FACGEO,AREATYPE,VOLUMEFROM,VOLUMETO,FUNCGROUP,STATIONGROUP
0,I80NevSR36,1,0,1000000,Arterial,COQ
1,I80NevSR36,2,0,1000000,Arterial,COS
2,I80NevSR36,3,0,1000000,Arterial,COT
3,I80NevSR36,4,0,1000000,Arterial,COU
4,I80NevSR36,5,0,1000000,Arterial,COU
...,...,...,...,...,...,...
583,Southeast,4,1500,1000000,CD Road,XX3
584,Southeast,5,1500,1000000,CD Road,XX3
585,Statewide,1,0,2500,CD Road,XX3
586,Statewide,1,2500,12500,CD Road,XX3


,FACGEO,AREATYPE,VOLUMEFROM,VOLUMETO,FUNCGROUP,STATIONGROUP
144,Statewide,1,0,2500,Arterial,COP
145,Statewide,1,2500,12500,Arterial,COQ
146,Statewide,1,12500,1000000,Arterial,COR


In [64]:
#add season factor

dfs_PrdForecasts_Season = []

#for (idx, row) in dfs_PrdForecasts.iterrows():

for df_PrdForecasts in dfs_PrdForecasts: 
    
    #joing forecasts data with station group matching
    df_PrdForecasts_Season = pd.DataFrame.merge(df_PrdForecasts,df_FacGeoFunGroupAreaTypeVolume_ToStationGroup,on=('FACGEO',nameFG,nameAT),how='left')
    #display(df_PrdForecasts_Season)
    
    #remove unapplicable volume classes
    df_PrdForecasts_Season = df_PrdForecasts_Season[(df_PrdForecasts_Season.ForecastAADT >= df_PrdForecasts_Season.VOLUMEFROM) & (df_PrdForecasts_Season.ForecastAADT < df_PrdForecasts_Season.VOLUMETO)]

    #join to season factors
    df_PrdForecasts_Season = pd.DataFrame.merge(df_PrdForecasts_Season,df_SeasonFactors,on=('STATIONGROUP'),how='left')
    
    #calculate Average Season
    df_PrdForecasts_Season['AvgSsnPrdVol'] = df_PrdForecasts_Season['AvgAnnualPrdVol'] * df_PrdForecasts_Season['FACTOR'] 
    df_PrdForecasts_Season['AvgSsnPrdVol'] = df_PrdForecasts_Season['AvgSsnPrdVol'].astype(int)
    
    
    df_PrdForecasts_Season = df_PrdForecasts_Season.rename(columns=({'FACTOR':'SsnFactor'}))
    
    display(df_PrdForecasts_Season)
    
    dfs_PrdForecasts_Season.append(df_PrdForecasts_Season)
    

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1517.0,1517.0,910.200000,Statewide,0,2500,COP,1.0,S00-Ann,910
1,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1517.0,1517.0,186.798298,Statewide,0,2500,COP,1.0,S00-Ann,186
2,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1517.0,1517.0,252.649584,Statewide,0,2500,COP,1.0,S00-Ann,252
3,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1517.0,1517.0,191.344428,Statewide,0,2500,COP,1.0,S00-Ann,191
4,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1517.0,1517.0,180.512684,Statewide,0,2500,COP,1.0,S00-Ann,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43185,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
43186,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
43187,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
43188,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1600.0,1600.0,960.000000,Statewide,0,2500,COP,1.0,S00-Ann,960
1,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1600.0,1600.0,199.120416,Statewide,0,2500,COP,1.0,S00-Ann,199
2,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1600.0,1600.0,267.205536,Statewide,0,2500,COP,1.0,S00-Ann,267
3,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1600.0,1600.0,194.951920,Statewide,0,2500,COP,1.0,S00-Ann,194
4,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1600.0,1600.0,190.852704,Statewide,0,2500,COP,1.0,S00-Ann,190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44275,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
44276,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
44277,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
44278,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1700.0,1700.0,1020.000000,Statewide,0,2500,COP,1.0,S00-Ann,1020
1,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1700.0,1700.0,208.618254,Statewide,0,2500,COP,1.0,S00-Ann,208
2,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1700.0,1700.0,287.761635,Statewide,0,2500,COP,1.0,S00-Ann,287
3,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1700.0,1700.0,206.460954,Statewide,0,2500,COP,1.0,S00-Ann,206
4,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1700.0,1700.0,206.090830,Statewide,0,2500,COP,1.0,S00-Ann,206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45065,WFRC_8476,3.0,8561.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
45066,WFRC_8476,3.0,8561.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
45067,WFRC_8476,3.0,8561.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
45068,WFRC_8476,3.0,8561.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800.0,1080.000000,Statewide,0,2500,COP,1.0,S00-Ann,1080
1,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800.0,220.152276,Statewide,0,2500,COP,1.0,S00-Ann,220
2,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800.0,310.042944,Statewide,0,2500,COP,1.0,S00-Ann,310
3,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800.0,221.433282,Statewide,0,2500,COP,1.0,S00-Ann,221
4,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800.0,211.113000,Statewide,0,2500,COP,1.0,S00-Ann,211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47185,WFRC_8476,3.0,7305.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
47186,WFRC_8476,3.0,7305.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
47187,WFRC_8476,3.0,7305.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
47188,WFRC_8476,3.0,7305.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0.0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100.0,1260.000000,Statewide,0,2500,COP,1.0,S00-Ann,1260
1,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100.0,262.994466,Statewide,0,2500,COP,1.0,S00-Ann,262
2,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100.0,351.765288,Statewide,0,2500,COP,1.0,S00-Ann,351
3,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100.0,261.083361,Statewide,0,2500,COP,1.0,S00-Ann,261
4,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100.0,250.479936,Statewide,0,2500,COP,1.0,S00-Ann,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48425,WFRC_8476,4.0,8725.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
48426,WFRC_8476,4.0,8725.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
48427,WFRC_8476,4.0,8725.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
48428,WFRC_8476,4.0,8725.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0.0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2500,1500.000000,Statewide,0,2500,COP,1.0,S00-Ann,1500
1,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2500,304.260050,Statewide,0,2500,COP,1.0,S00-Ann,304
2,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2500,428.325975,Statewide,0,2500,COP,1.0,S00-Ann,428
3,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2500,307.650750,Statewide,0,2500,COP,1.0,S00-Ann,307
4,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2500,301.990000,Statewide,0,2500,COP,1.0,S00-Ann,301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48485,WFRC_8476,4.0,9025.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,300,180.000000,Statewide,0,1000000,COU,1.0,S00-Ann,180
48486,WFRC_8476,4.0,9025.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,300,12.816705,Statewide,0,1000000,COU,1.0,S00-Ann,12
48487,WFRC_8476,4.0,9025.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,300,40.237764,Statewide,0,1000000,COU,1.0,S00-Ann,40
48488,WFRC_8476,4.0,9025.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,300,63.496860,Statewide,0,1000000,COU,1.0,S00-Ann,63


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800,1080.000000,Statewide,0,2500,COP,1.0,S00-Ann,1080
1,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800,220.013604,Statewide,0,2500,COP,1.0,S00-Ann,220
2,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800,301.223448,Statewide,0,2500,COP,1.0,S00-Ann,301
3,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800,221.029290,Statewide,0,2500,COP,1.0,S00-Ann,221
4,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,1800.0,1800,206.970624,Statewide,0,2500,COP,1.0,S00-Ann,206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47615,WFRC_8476,3.0,7174.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
47616,WFRC_8476,3.0,7174.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
47617,WFRC_8476,3.0,7174.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0
47618,WFRC_8476,3.0,7174.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,0.0,0,0.000000,Statewide,0,1000000,COT,1.0,S00-Ann,0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100,1260.000000,Statewide,0,2500,COP,1.0,S00-Ann,1260
1,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100,264.145560,Statewide,0,2500,COP,1.0,S00-Ann,264
2,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100,350.225400,Statewide,0,2500,COP,1.0,S00-Ann,350
3,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100,256.207350,Statewide,0,2500,COP,1.0,S00-Ann,256
4,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2100.0,2100,245.894670,Statewide,0,2500,COP,1.0,S00-Ann,245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49755,WFRC_8476,4.0,9850.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,1100,660.000000,Statewide,0,1000000,COU,1.0,S00-Ann,660
49756,WFRC_8476,4.0,9850.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,1100,41.038525,Statewide,0,1000000,COU,1.0,S00-Ann,41
49757,WFRC_8476,4.0,9850.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,1100,122.459722,Statewide,0,1000000,COU,1.0,S00-Ann,122
49758,WFRC_8476,4.0,9850.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,1100,294.859708,Statewide,0,1000000,COU,1.0,S00-Ann,294


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,1380.000000,Statewide,0,2500,COP,1.0,S00-Ann,1380
1,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,280.720566,Statewide,0,2500,COP,1.0,S00-Ann,280
2,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,383.442936,Statewide,0,2500,COP,1.0,S00-Ann,383
3,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,288.532125,Statewide,0,2500,COP,1.0,S00-Ann,288
4,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,276.277495,Statewide,0,2500,COP,1.0,S00-Ann,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52655,WFRC_8476,4.0,14662.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
52656,WFRC_8476,4.0,14662.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
52657,WFRC_8476,4.0,14662.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
52658,WFRC_8476,4.0,14662.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,1380.000000,Statewide,0,2500,COP,1.0,S00-Ann,1380
1,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,278.853955,Statewide,0,2500,COP,1.0,S00-Ann,278
2,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,383.907559,Statewide,0,2500,COP,1.0,S00-Ann,383
3,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,287.483946,Statewide,0,2500,COP,1.0,S00-Ann,287
4,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,2300.0,2300,273.727692,Statewide,0,2500,COP,1.0,S00-Ann,273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52665,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
52666,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
52667,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0
52668,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,0.0,0,0.000000,Statewide,0,1000000,COU,1.0,S00-Ann,0


In [65]:
#dfs_PrdForecasts[6]
#segid='2878_005.4'
#i=4
#display(dfs_PrdForecasts[i][dfs_PrdForecasts[i]['SEGID']==segid])
#display(df_SeasonFactors[df_SeasonFactors['STATIONGROUP']=='COT'])
#display(dfs_PrdForecasts_Season[i][dfs_PrdForecasts_Season[i]['SEGID']==segid])

In [66]:
df_DOWFactors

,STATIONGROUP,FACTOR,DOWGROUP
0,CO0,0.715249,1-Weekday (Tu-Th)
1,CO1,0.824043,1-Weekday (Tu-Th)
2,CO2,0.790750,1-Weekday (Tu-Th)
3,CO3,0.853478,1-Weekday (Tu-Th)
4,CO4,0.904970,1-Weekday (Tu-Th)
...,...,...,...
32,COW,1.067677,1-Weekday (Tu-Th)
33,COX,1.087913,1-Weekday (Tu-Th)
34,COY,0.967245,1-Weekday (Tu-Th)
35,COZ,1.006499,1-Weekday (Tu-Th)


In [67]:
#add dow factor

dfs_PrdForecasts_Season_DOW = []

#for (idx, row) in dfs_PrdForecasts.iterrows():

for df_PrdForecasts_Season in dfs_PrdForecasts_Season: 
    
    #joing forecasts data with season factors
    df_PrdForecasts_Season_DOW = pd.DataFrame.merge(df_PrdForecasts_Season,df_DOWFactors,on=('STATIONGROUP'),how='left')
    
    #calculate Average Season DOW
    df_PrdForecasts_Season_DOW['AvgSsnDOWPrdVol'] = df_PrdForecasts_Season_DOW['AvgSsnPrdVol'] * df_PrdForecasts_Season_DOW['FACTOR'] 
    df_PrdForecasts_Season_DOW['AvgSsnDOWPrdVol'] = df_PrdForecasts_Season_DOW['AvgSsnDOWPrdVol'].astype(int)

    df_PrdForecasts_Season_DOW = df_PrdForecasts_Season_DOW.rename(columns=({'FACTOR':'DOWFactor'}))
    
    display(df_PrdForecasts_Season_DOW)
    
    dfs_PrdForecasts_Season_DOW.append(df_PrdForecasts_Season_DOW)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,910,0.984028,1-Weekday (Tu-Th),895
1,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,186,0.984028,1-Weekday (Tu-Th),183
2,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,252,0.984028,1-Weekday (Tu-Th),247
3,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,191,0.984028,1-Weekday (Tu-Th),187
4,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,180,0.984028,1-Weekday (Tu-Th),177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43185,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
43186,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
43187,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
43188,WFRC_8476,3.0,6617.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,960,0.984028,1-Weekday (Tu-Th),944
1,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,199,0.984028,1-Weekday (Tu-Th),195
2,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,267,0.984028,1-Weekday (Tu-Th),262
3,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,194,0.984028,1-Weekday (Tu-Th),190
4,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,190,0.984028,1-Weekday (Tu-Th),186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44275,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
44276,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
44277,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
44278,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,1020,0.984028,1-Weekday (Tu-Th),1003
1,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,208,0.984028,1-Weekday (Tu-Th),204
2,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,287,0.984028,1-Weekday (Tu-Th),282
3,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,206,0.984028,1-Weekday (Tu-Th),202
4,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,206,0.984028,1-Weekday (Tu-Th),202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45065,WFRC_8476,3.0,8561.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
45066,WFRC_8476,3.0,8561.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
45067,WFRC_8476,3.0,8561.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
45068,WFRC_8476,3.0,8561.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,1080,0.984028,1-Weekday (Tu-Th),1062
1,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,220,0.984028,1-Weekday (Tu-Th),216
2,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,310,0.984028,1-Weekday (Tu-Th),305
3,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,221,0.984028,1-Weekday (Tu-Th),217
4,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,211,0.984028,1-Weekday (Tu-Th),207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47185,WFRC_8476,3.0,7305.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
47186,WFRC_8476,3.0,7305.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
47187,WFRC_8476,3.0,7305.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
47188,WFRC_8476,3.0,7305.9,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,1260,0.984028,1-Weekday (Tu-Th),1239
1,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,262,0.984028,1-Weekday (Tu-Th),257
2,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,351,0.984028,1-Weekday (Tu-Th),345
3,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,261,0.984028,1-Weekday (Tu-Th),256
4,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,250,0.984028,1-Weekday (Tu-Th),246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48425,WFRC_8476,4.0,8725.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
48426,WFRC_8476,4.0,8725.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
48427,WFRC_8476,4.0,8725.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
48428,WFRC_8476,4.0,8725.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,1500,0.984028,1-Weekday (Tu-Th),1476
1,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,304,0.984028,1-Weekday (Tu-Th),299
2,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,428,0.984028,1-Weekday (Tu-Th),421
3,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,307,0.984028,1-Weekday (Tu-Th),302
4,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,301,0.984028,1-Weekday (Tu-Th),296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48485,WFRC_8476,4.0,9025.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,180,1.094640,1-Weekday (Tu-Th),197
48486,WFRC_8476,4.0,9025.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,12,1.094640,1-Weekday (Tu-Th),13
48487,WFRC_8476,4.0,9025.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,40,1.094640,1-Weekday (Tu-Th),43
48488,WFRC_8476,4.0,9025.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,63,1.094640,1-Weekday (Tu-Th),68


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,1080,0.984028,1-Weekday (Tu-Th),1062
1,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,220,0.984028,1-Weekday (Tu-Th),216
2,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,301,0.984028,1-Weekday (Tu-Th),296
3,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,221,0.984028,1-Weekday (Tu-Th),217
4,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,206,0.984028,1-Weekday (Tu-Th),202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47615,WFRC_8476,3.0,7174.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
47616,WFRC_8476,3.0,7174.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
47617,WFRC_8476,3.0,7174.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0
47618,WFRC_8476,3.0,7174.1,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,...,Statewide,0,1000000,COT,1.0,S00-Ann,0,1.092362,1-Weekday (Tu-Th),0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,1260,0.984028,1-Weekday (Tu-Th),1239
1,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,264,0.984028,1-Weekday (Tu-Th),259
2,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,350,0.984028,1-Weekday (Tu-Th),344
3,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,256,0.984028,1-Weekday (Tu-Th),251
4,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,245,0.984028,1-Weekday (Tu-Th),241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49755,WFRC_8476,4.0,9850.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,660,1.094640,1-Weekday (Tu-Th),722
49756,WFRC_8476,4.0,9850.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,41,1.094640,1-Weekday (Tu-Th),44
49757,WFRC_8476,4.0,9850.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,122,1.094640,1-Weekday (Tu-Th),133
49758,WFRC_8476,4.0,9850.7,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,294,1.094640,1-Weekday (Tu-Th),321


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,1380,0.984028,1-Weekday (Tu-Th),1357
1,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,280,0.984028,1-Weekday (Tu-Th),275
2,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,383,0.984028,1-Weekday (Tu-Th),376
3,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,288,0.984028,1-Weekday (Tu-Th),283
4,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,276,0.984028,1-Weekday (Tu-Th),271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52655,WFRC_8476,4.0,14662.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
52656,WFRC_8476,4.0,14662.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
52657,WFRC_8476,4.0,14662.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
52658,WFRC_8476,4.0,14662.0,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,1380,0.984028,1-Weekday (Tu-Th),1357
1,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,278,0.984028,1-Weekday (Tu-Th),273
2,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,383,0.984028,1-Weekday (Tu-Th),376
3,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,287,0.984028,1-Weekday (Tu-Th),282
4,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,273,0.984028,1-Weekday (Tu-Th),268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52665,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
52666,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
52667,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
52668,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0


In [68]:
df_TODFactors

,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
4,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4054,Prd
12,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3652,Prd
19,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3819,Prd
28,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.5003,Prd
37,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4619,Prd
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


In [69]:
df_VCGroups

,SEASONGROUP,SeasonType,DOWGROUP,PERIOD,HRPCTOF,VCGroupCode
VCGroupID,,,,,,
0,S00-Ann,Season,1-Weekday (Tu-Th),AM,Prd,AnnWkAM
1,S00-Ann,Season,1-Weekday (Tu-Th),MD,Prd,AnnWkMD
2,S00-Ann,Season,1-Weekday (Tu-Th),PM,Prd,AnnWkPM


In [70]:
df_PrdForecasts_Season_DOW

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,...,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,1380,0.984028,1-Weekday (Tu-Th),1357
1,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,278,0.984028,1-Weekday (Tu-Th),273
2,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,383,0.984028,1-Weekday (Tu-Th),376
3,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,287,0.984028,1-Weekday (Tu-Th),282
4,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,...,Statewide,0,2500,COP,1.0,S00-Ann,273,0.984028,1-Weekday (Tu-Th),268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52665,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
52666,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
52667,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0
52668,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,...,Statewide,0,1000000,COU,1.0,S00-Ann,0,1.094640,1-Weekday (Tu-Th),0


In [71]:
df_PrdForecasts_Season_DOW.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd', 'PRDFAC', 'MDPCT',
       'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj', 'AvgAnnualPrdVol',
       'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP', 'SsnFactor',
       'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol'],
      dtype='object')

In [72]:
#add tod factor

segid = '2878_005.4'

pd.set_option('display.max_columns', None)

dfs_PrdForecasts_Season_DOW_TOD_wGroups = []

i=0

for df_PrdForecasts_Season_DOW in dfs_PrdForecasts_Season_DOW: 

    display(i)

    df_PrdForecasts_Season_DOW = df_PrdForecasts_Season_DOW.rename(columns={'Prd': namePrd + '_TDM'})

    #display(df_PrdForecasts_Season_DOW[namePrd + '_TDM'])
    
    #joing forecasts data with season factors
    
    df_TODFactors_Prd = df_TODFactors[df_TODFactors['HRPCTOF'] == 'Prd']
    df_TODFactors_DY  = df_TODFactors[df_TODFactors['HRPCTOF'] == 'DY']
    
    #get peak Hr as a Pct of Prd
    df_PrdForecasts_Season_DOW_TOD_Prd = pd.DataFrame.merge(df_PrdForecasts_Season_DOW,df_TODFactors_Prd,left_on=('STATIONGROUP','SEASONGROUP','DOWGROUP',namePrd + '_TDM'),right_on=('STATIONGROUP','SEASONGROUP','DOWGROUP','PERIOD'))
    display(df_PrdForecasts_Season_DOW_TOD_Prd[df_PrdForecasts_Season_DOW_TOD_Prd['SEGID']==segid])
    
    #get peak Hr as a Pct of DY
    df_PrdForecasts_Season_DOW_TOD_DY = pd.DataFrame.merge(df_PrdForecasts_Season_DOW,df_TODFactors_DY,left_on=('STATIONGROUP','SEASONGROUP','DOWGROUP',namePrd + '_TDM'),right_on=('STATIONGROUP','SEASONGROUP','DOWGROUP','HRPCTOF'))
 
    df_PrdForecasts_Season_DOW_TOD = pd.concat([df_PrdForecasts_Season_DOW_TOD_Prd, df_PrdForecasts_Season_DOW_TOD_DY])

    #display(df_PrdForecasts_Season_DOW_TOD)
    
    df_PrdForecasts_Season_DOW_TOD_wGroups = pd.DataFrame.merge(df_PrdForecasts_Season_DOW_TOD, df_VCGroups,on=('SEASONGROUP','DOWGROUP','HRPCTOF','PERIOD'))
      
    dfs_PrdForecasts_Season_DOW_TOD_wGroups.append(df_PrdForecasts_Season_DOW_TOD_wGroups)
    
    display(df_PrdForecasts_Season_DOW_TOD)
    
    i=i+1


0

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
7626,2878_005.4,4.0,33688.9,Arterial,1.0,D1,2.0,3.0,Minor Arterial,815,0.7681,AM,0.1894,3.1,1.2,2019,1106.0,1106.0,160.898823,Statewide,0,1000000,COU,1.0,S00-Ann,160,1.09464,1-Weekday (Tu-Th),175,2015-2019,AM,0.4029,Prd
7627,2878_005.4,4.0,33688.9,Arterial,1.0,D2,2.0,3.0,Minor Arterial,815,0.2319,AM,0.1894,14.6,6.8,2019,1106.0,1106.0,48.577577,Statewide,0,1000000,COU,1.0,S00-Ann,48,1.09464,1-Weekday (Tu-Th),52,2015-2019,AM,0.4029,Prd
10716,2878_005.4,4.0,33688.9,Arterial,1.0,D1,2.0,3.0,Minor Arterial,815,0.5077,MD,0.2938,9.3,4.2,2019,1106.0,1106.0,164.973460,Statewide,0,1000000,COU,1.0,S00-Ann,164,1.09464,1-Weekday (Tu-Th),179,2015-2019,MD,0.1956,Prd
10717,2878_005.4,4.0,33688.9,Arterial,1.0,D2,2.0,3.0,Minor Arterial,815,0.4923,MD,0.2938,10.4,5.1,2019,1106.0,1106.0,159.969340,Statewide,0,1000000,COU,1.0,S00-Ann,159,1.09464,1-Weekday (Tu-Th),174,2015-2019,MD,0.1956,Prd
13806,2878_005.4,4.0,33688.9,Arterial,1.0,D1,2.0,3.0,Minor Arterial,815,0.3755,PM,0.2755,7.0,3.4,2019,1106.0,1106.0,114.415977,Statewide,0,1000000,COU,1.0,S00-Ann,114,1.09464,1-Weekday (Tu-Th),124,2015-2019,PM,0.3615,Prd
13807,2878_005.4,4.0,33688.9,Arterial,1.0,D2,2.0,3.0,Minor Arterial,815,0.6245,PM,0.2755,3.8,0.6,2019,1106.0,1106.0,190.287024,Statewide,0,1000000,COU,1.0,S00-Ann,190,1.09464,1-Weekday (Tu-Th),207,2015-2019,PM,0.3615,Prd


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
0,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6045,AM,0.2037,9.1,8.2,2019,1517.0,1517.0,186.798298,Statewide,0,2500,COP,1.0,S00-Ann,186,0.984028,1-Weekday (Tu-Th),183,2015-2019,AM,0.4093,Prd
1,0006_141.0,1.0,1566.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3955,AM,0.2037,8.3,9.7,2019,1517.0,1517.0,122.214602,Statewide,0,2500,COP,1.0,S00-Ann,122,0.984028,1-Weekday (Tu-Th),120,2015-2019,AM,0.4093,Prd
2,0006_146.9,1.0,1571.8,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6779,AM,0.2034,9.1,7.8,2019,1517.0,1517.0,209.171333,Statewide,0,2500,COP,1.0,S00-Ann,209,0.984028,1-Weekday (Tu-Th),205,2015-2019,AM,0.4093,Prd
3,0006_146.9,1.0,1571.8,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3221,AM,0.2034,7.8,10.7,2019,1517.0,1517.0,99.386467,Statewide,0,2500,COP,1.0,S00-Ann,99,0.984028,1-Weekday (Tu-Th),97,2015-2019,AM,0.4093,Prd
4,0006_149.9,1.0,1635.2,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6609,AM,0.1672,10.5,6.1,2019,2441.0,2441.0,269.736554,Statewide,0,2500,COP,1.0,S00-Ann,269,0.984028,1-Weekday (Tu-Th),264,2015-2019,AM,0.4093,Prd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25909,0215_027.4,3.0,55027.9,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,14.7,8.8,2019,39875.0,39875.0,23925.000000,I215,0,1000000,CO8,1.0,S00-Ann,23925,1.134873,1-Weekday (Tu-Th),27151,2015-2019,MD,0.0561,DY
25910,0215_027.4,3.0,55027.9,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,14.7,8.8,2019,39875.0,39875.0,23925.000000,I215,0,1000000,CO8,1.0,S00-Ann,23925,1.134873,1-Weekday (Tu-Th),27151,2015-2019,PM,0.0916,DY
25911,0215_027.4,3.0,55027.9,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,17.0,10.6,2019,39875.0,39875.0,23925.000000,I215,0,1000000,CO8,1.0,S00-Ann,23925,1.134873,1-Weekday (Tu-Th),27151,2015-2019,AM,0.0812,DY
25912,0215_027.4,3.0,55027.9,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,17.0,10.6,2019,39875.0,39875.0,23925.000000,I215,0,1000000,CO8,1.0,S00-Ann,23925,1.134873,1-Weekday (Tu-Th),27151,2015-2019,MD,0.0561,DY


1

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
7482,2878_005.4,4.0,56513.9,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6607,AM,0.1595,2.6,0.9,2023,39500.0,39500.0,4162.575175,Statewide,0,1000000,COU,1.0,S00-Ann,4162,1.09464,1-Weekday (Tu-Th),4555,2015-2019,AM,0.4029,Prd
7483,2878_005.4,4.0,56513.9,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.3393,AM,0.1595,16.3,5.1,2023,39500.0,39500.0,2137.674825,Statewide,0,1000000,COU,1.0,S00-Ann,2137,1.09464,1-Weekday (Tu-Th),2339,2015-2019,AM,0.4029,Prd
10766,2878_005.4,4.0,56513.9,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5332,MD,0.3250,7.3,2.3,2023,39500.0,39500.0,6844.955000,Statewide,0,1000000,COU,1.0,S00-Ann,6844,1.09464,1-Weekday (Tu-Th),7491,2015-2019,MD,0.1956,Prd
10767,2878_005.4,4.0,56513.9,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4668,MD,0.3250,7.6,3.5,2023,39500.0,39500.0,5992.545000,Statewide,0,1000000,COU,1.0,S00-Ann,5992,1.09464,1-Weekday (Tu-Th),6559,2015-2019,MD,0.1956,Prd
14050,2878_005.4,4.0,56513.9,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4078,PM,0.1986,13.1,5.4,2023,39500.0,39500.0,3199.068660,Statewide,0,1000000,COU,1.0,S00-Ann,3199,1.09464,1-Weekday (Tu-Th),3501,2015-2019,PM,0.3615,Prd
14051,2878_005.4,4.0,56513.9,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5922,PM,0.1986,1.0,0.3,2023,39500.0,39500.0,4645.631340,Statewide,0,1000000,COU,1.0,S00-Ann,4645,1.09464,1-Weekday (Tu-Th),5084,2015-2019,PM,0.3615,Prd


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
18934,2878_005.4,4.0,41683.2,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5642,AM,0.1674,8.0,3.5,2050,26000.0,25500,2408.400540,Statewide,0,1000000,COU,1.0,S00-Ann,2408,1.09464,1-Weekday (Tu-Th),2635,2015-2019,AM,0.4029,Prd
18935,2878_005.4,4.0,41683.2,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4358,AM,0.1674,17.7,10.0,2050,26000.0,25500,1860.299460,Statewide,0,1000000,COU,1.0,S00-Ann,1860,1.09464,1-Weekday (Tu-Th),2036,2015-2019,AM,0.4029,Prd
23128,2878_005.4,4.0,41683.2,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4415,MD,0.3391,11.9,5.7,2050,26000.0,25500,3817.672575,Statewide,0,1000000,COU,1.0,S00-Ann,3817,1.09464,1-Weekday (Tu-Th),4178,2015-2019,MD,0.1956,Prd
23129,2878_005.4,4.0,41683.2,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5585,MD,0.3391,15.7,7.9,2050,26000.0,25500,4829.377425,Statewide,0,1000000,COU,1.0,S00-Ann,4829,1.09464,1-Weekday (Tu-Th),5286,2015-2019,MD,0.1956,Prd
27322,2878_005.4,4.0,41683.2,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3759,PM,0.2210,13.2,6.7,2050,26000.0,25500,2118.384450,Statewide,0,1000000,COU,1.0,S00-Ann,2118,1.09464,1-Weekday (Tu-Th),2318,2015-2019,PM,0.3615,Prd
27323,2878_005.4,4.0,41683.2,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6241,PM,0.2210,7.7,4.0,2050,26000.0,25500,3517.115550,Statewide,0,1000000,COU,1.0,S00-Ann,3517,1.09464,1-Weekday (Tu-Th),3849,2015-2019,PM,0.3615,Prd


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
0,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6183,AM,0.1974,9.0,7.5,2050,2300.0,2300,280.720566,Statewide,0,2500,COP,1.0,S00-Ann,280,0.984028,1-Weekday (Tu-Th),275,2015-2019,AM,0.4093,Prd
1,0006_141.0,1.0,2346.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3817,AM,0.1974,8.0,8.2,2050,2300.0,2300,173.299434,Statewide,0,2500,COP,1.0,S00-Ann,173,0.984028,1-Weekday (Tu-Th),170,2015-2019,AM,0.4093,Prd
2,0006_146.9,1.0,2351.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.7047,AM,0.1974,9.2,7.3,2050,2300.0,2300,319.947894,Statewide,0,2500,COP,1.0,S00-Ann,319,0.984028,1-Weekday (Tu-Th),313,2015-2019,AM,0.4093,Prd
3,0006_146.9,1.0,2351.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.2953,AM,0.1974,7.3,8.8,2050,2300.0,2300,134.072106,Statewide,0,2500,COP,1.0,S00-Ann,134,0.984028,1-Weekday (Tu-Th),131,2015-2019,AM,0.4093,Prd
4,0065_000.1,1.0,608.3,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6490,AM,0.1920,6.6,4.0,2050,1800.0,1800,224.294400,Statewide,0,2500,COP,1.0,S00-Ann,224,0.984028,1-Weekday (Tu-Th),220,2015-2019,AM,0.4093,Prd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31591,0215_027.4,3.0,58652.9,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,14.6,8.5,2050,34500.0,39000,23400.000000,I215,0,1000000,CO8,1.0,S00-Ann,23400,1.134873,1-Weekday (Tu-Th),26556,2015-2019,MD,0.0561,DY
31592,0215_027.4,3.0,58652.9,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,14.6,8.5,2050,34500.0,39000,23400.000000,I215,0,1000000,CO8,1.0,S00-Ann,23400,1.134873,1-Weekday (Tu-Th),26556,2015-2019,PM,0.0916,DY
31593,0215_027.4,3.0,58652.9,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,12.4,5.8,2050,34500.0,39000,23400.000000,I215,0,1000000,CO8,1.0,S00-Ann,23400,1.134873,1-Weekday (Tu-Th),26556,2015-2019,AM,0.0812,DY
31594,0215_027.4,3.0,58652.9,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,12.4,5.8,2050,34500.0,39000,23400.000000,I215,0,1000000,CO8,1.0,S00-Ann,23400,1.134873,1-Weekday (Tu-Th),26556,2015-2019,MD,0.0561,DY


15

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
18934,2878_005.4,4.0,39215.3,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5942,AM,0.1735,7.9,3.4,2050,26000.0,23000,2371.15510,Statewide,0,1000000,COU,1.0,S00-Ann,2371,1.09464,1-Weekday (Tu-Th),2595,2015-2019,AM,0.4029,Prd
18935,2878_005.4,4.0,39215.3,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4058,AM,0.1735,18.0,9.7,2050,26000.0,23000,1619.34490,Statewide,0,1000000,COU,1.0,S00-Ann,1619,1.09464,1-Weekday (Tu-Th),1772,2015-2019,AM,0.4029,Prd
23128,2878_005.4,4.0,39215.3,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4319,MD,0.3294,12.2,6.1,2050,26000.0,23000,3272.16078,Statewide,0,1000000,COU,1.0,S00-Ann,3272,1.09464,1-Weekday (Tu-Th),3581,2015-2019,MD,0.1956,Prd
23129,2878_005.4,4.0,39215.3,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5681,MD,0.3294,15.8,8.4,2050,26000.0,23000,4304.03922,Statewide,0,1000000,COU,1.0,S00-Ann,4304,1.09464,1-Weekday (Tu-Th),4711,2015-2019,MD,0.1956,Prd
27322,2878_005.4,4.0,39215.3,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.3974,PM,0.2297,14.0,7.3,2050,26000.0,23000,2099.50394,Statewide,0,1000000,COU,1.0,S00-Ann,2099,1.09464,1-Weekday (Tu-Th),2297,2015-2019,PM,0.3615,Prd
27323,2878_005.4,4.0,39215.3,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6026,PM,0.2297,8.8,4.5,2050,26000.0,23000,3183.59606,Statewide,0,1000000,COU,1.0,S00-Ann,3183,1.09464,1-Weekday (Tu-Th),3484,2015-2019,PM,0.3615,Prd


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6145,AM,0.1973,9.0,7.6,2050,2300.0,2300,278.853955,Statewide,0,2500,COP,1.0,S00-Ann,278,0.984028,1-Weekday (Tu-Th),273,2015-2019,AM,0.4093,Prd
1,0006_141.0,1.0,2362.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3855,AM,0.1973,7.9,8.5,2050,2300.0,2300,174.936045,Statewide,0,2500,COP,1.0,S00-Ann,174,0.984028,1-Weekday (Tu-Th),171,2015-2019,AM,0.4093,Prd
2,0006_146.9,1.0,2365.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6981,AM,0.1975,9.2,7.4,2050,2300.0,2300,317.111925,Statewide,0,2500,COP,1.0,S00-Ann,317,0.984028,1-Weekday (Tu-Th),311,2015-2019,AM,0.4093,Prd
3,0006_146.9,1.0,2365.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3019,AM,0.1975,7.1,9.2,2050,2300.0,2300,137.138075,Statewide,0,2500,COP,1.0,S00-Ann,137,0.984028,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd
4,0065_000.1,1.0,606.3,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6303,AM,0.1829,4.3,1.4,2050,1800.0,1800,207.507366,Statewide,0,2500,COP,1.0,S00-Ann,207,0.984028,1-Weekday (Tu-Th),203,2015-2019,AM,0.4093,Prd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31597,0215_027.4,3.0,58587.8,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,14.6,8.5,2050,34500.0,39000,23400.000000,I215,0,1000000,CO8,1.0,S00-Ann,23400,1.134873,1-Weekday (Tu-Th),26556,2015-2019,MD,0.0561,DY
31598,0215_027.4,3.0,58587.8,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,14.6,8.5,2050,34500.0,39000,23400.000000,I215,0,1000000,CO8,1.0,S00-Ann,23400,1.134873,1-Weekday (Tu-Th),26556,2015-2019,PM,0.0916,DY
31599,0215_027.4,3.0,58587.8,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,12.4,5.9,2050,34500.0,39000,23400.000000,I215,0,1000000,CO8,1.0,S00-Ann,23400,1.134873,1-Weekday (Tu-Th),26556,2015-2019,AM,0.0812,DY
31600,0215_027.4,3.0,58587.8,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.6000,DY,1.0000,12.4,5.9,2050,34500.0,39000,23400.000000,I215,0,1000000,CO8,1.0,S00-Ann,23400,1.134873,1-Weekday (Tu-Th),26556,2015-2019,MD,0.0561,DY


In [73]:
#TROUBLESHOOTING

#df_PrdForecasts_Season_DOW_TOD_DY[(df_PrdForecasts_Season_DOW_TOD_DY['SEGID']=='0006_146.9') & (df_PrdForecasts_Season_DOW_TOD_DY['DOWGroup']=='6-Saturday')].head(20)

#segid = '2878_005.4'
#i=4
#
#ccsgroup = 'COT'
#display(dfs_PrdForecasts_Season[i][dfs_PrdForecasts_Season[i]['SEGID']==segid])
#display(dfs_PrdForecasts_Season_DOW[i][dfs_PrdForecasts_Season_DOW[i]['SEGID']==segid])
#display(dfs_PrdForecasts_Season_DOW_TOD_wGroups[i][dfs_PrdForecasts_Season_DOW_TOD_wGroups[i]['SEGID']==segid])
#
#display(df_TODFactors_Prd[df_TODFactors_Prd['STATIONGROUP']==ccsgroup])
#
#display(df_PrdForecasts_Season_DOW_TOD_Prd[df_PrdForecasts_Season_DOW_TOD_Prd['SEGID']==segid])
#
#display(df_PrdForecasts_Season_DOW_TOD_DY[(df_PrdForecasts_Season_DOW_TOD_DY['SEGID']==segid)].head(20))


In [74]:
#df_PrdForecasts_Season_DOW_TOD[df_PrdForecasts_Season_DOW_TOD['SEGID']=='0006_146.9'].head(20)

In [75]:
#dftemp = dfs_PrdForecasts_Season_DOW_TOD_wGroups[0].groupby(['VCGroupCode']).agg({'VolMaxHrPct':[np.size]})
df_TODFactors_DY[df_TODFactors_DY['SEASONGROUP']=='S00-Ann']

,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
1871,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0749,DY
1879,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0670,DY
1886,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0614,DY
1895,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0435,DY
1904,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0575,DY
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


In [76]:
df_PrdDuration

,PERIOD,PRDHRS
0,AM,3
1,MD,6
2,PM,3
3,EV,12
4,DY,24


In [77]:
dfs_PkHrForecasts = []

for df_PrdForecasts_Season_DOW_TOD_wGroups in dfs_PrdForecasts_Season_DOW_TOD_wGroups: 
    
    df_PkHrForecasts = pd.DataFrame.merge(df_PrdForecasts_Season_DOW_TOD_wGroups,df_PrdDuration,on='PERIOD',how='left')   
    
    df_PkHrForecasts['AvgSsnDOWPkHrVol'] = df_PkHrForecasts['AvgSsnDOWPrdVol'] * df_PkHrForecasts['VOLPKHRPCT']

    #add truck Pctage adjustments
    #PCE for medium and heavy trucks from HCM 2000 Exhibit 23-8
    Emd = 1.2 #same as RVs
    Ehv = 1.5
    df_PkHrForecasts['TrkFac'] = 1 + (df_PkHrForecasts[nameMDPct]/100 * (Emd-1)) + (df_PkHrForecasts[nameHVPct]/100 * (Ehv-1))
    
    #calculate PCE
    df_PkHrForecasts['PrdPCEFlow' ] = (df_PkHrForecasts['AvgSsnDOWPrdVol' ] *  df_PkHrForecasts['TrkFac']) /  df_PkHrForecasts['PRDHRS']
    df_PkHrForecasts['PkHrPCEFlow'] =  df_PkHrForecasts['AvgSsnDOWPkHrVol'] *  df_PkHrForecasts['TrkFac']
    df_PkHrForecasts['PrdVC'      ] =  df_PkHrForecasts['PrdPCEFlow'      ] / (df_PkHrForecasts[nameCap1HL] * df_PkHrForecasts[nameLanes])
    df_PkHrForecasts['PkHrVC'     ] =  df_PkHrForecasts['PkHrPCEFlow'     ] / (df_PkHrForecasts[nameCap1HL] * df_PkHrForecasts[nameLanes])
    
    display(df_PkHrForecasts)
    
    dfs_PkHrForecasts.append(df_PkHrForecasts)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1566.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6045,AM,0.2037,9.1,8.2,2019,1517.0,1517.0,186.798298,Statewide,0,2500,COP,1.0,S00-Ann,186,0.984028,1-Weekday (Tu-Th),183,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,74.9019,1.0592,64.611200,79.336092,0.060668,0.074494
1,0006_141.0,1.0,1566.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3955,AM,0.2037,8.3,9.7,2019,1517.0,1517.0,122.214602,Statewide,0,2500,COP,1.0,S00-Ann,122,0.984028,1-Weekday (Tu-Th),120,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,49.1160,1.0651,42.604000,52.313452,0.040004,0.049121
2,0006_146.9,1.0,1571.8,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6779,AM,0.2034,9.1,7.8,2019,1517.0,1517.0,209.171333,Statewide,0,2500,COP,1.0,S00-Ann,209,0.984028,1-Weekday (Tu-Th),205,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.9065,1.0572,72.242000,88.705952,0.067833,0.083292
3,0006_146.9,1.0,1571.8,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3221,AM,0.2034,7.8,10.7,2019,1517.0,1517.0,99.386467,Statewide,0,2500,COP,1.0,S00-Ann,99,0.984028,1-Weekday (Tu-Th),97,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,39.7021,1.0691,34.567567,42.445515,0.032458,0.039855
4,0006_149.9,1.0,1635.2,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6609,AM,0.1672,10.5,6.1,2019,2441.0,2441.0,269.736554,Statewide,0,2500,COP,1.0,S00-Ann,269,0.984028,1-Weekday (Tu-Th),264,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,108.0552,1.0515,92.532000,113.620043,0.086885,0.106685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25909,0215_025.7,2.0,109267.4,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4381,PM,0.2779,10.8,6.5,2019,75533.0,75533.0,9195.990929,I215,0,1000000,CO8,1.0,S00-Ann,9195,1.134873,1-Weekday (Tu-Th),10435,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,3819.2100,1.0541,3666.511167,4025.829261,0.489913,0.537925
25910,0215_026.7,3.0,72892.6,Freeway,1.0,D1,3.0,34.0,Freeway,1964,0.5014,PM,0.1989,19.7,11.6,2019,42492.0,42492.0,4237.661722,I215,0,1000000,CO8,1.0,S00-Ann,4237,1.134873,1-Weekday (Tu-Th),4808,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1759.7280,1.0974,1758.766400,1931.125507,0.298501,0.327754
25911,0215_026.7,3.0,72892.6,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.4986,PM,0.1989,19.9,12.0,2019,42492.0,42492.0,4213.997078,I215,0,1000000,CO8,1.0,S00-Ann,4213,1.134873,1-Weekday (Tu-Th),4781,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1749.8460,1.0998,1752.714600,1924.480631,0.421326,0.462616
25912,0215_027.4,3.0,55027.9,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5304,PM,0.1788,20.9,12.1,2019,39875.0,39875.0,3781.566360,I215,0,1000000,CO8,1.0,S00-Ann,3781,1.134873,1-Weekday (Tu-Th),4290,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1570.1400,1.1023,1576.289000,1730.765322,0.378916,0.416049


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5862,AM,0.2123,8.6,8.2,2023,1600.0,1600.0,199.120416,Statewide,0,2500,COP,1.0,S00-Ann,199,0.984028,1-Weekday (Tu-Th),195,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,79.8135,1.0582,68.783000,84.458646,0.064585,0.079304
1,0006_141.0,1.0,1663.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4138,AM,0.2123,6.3,9.6,2023,1600.0,1600.0,140.559584,Statewide,0,2500,COP,1.0,S00-Ann,140,0.984028,1-Weekday (Tu-Th),137,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.0741,1.0606,48.434067,59.472190,0.045478,0.055842
2,0006_146.9,1.0,1671.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6479,AM,0.2124,9.1,7.8,2023,1600.0,1600.0,220.182336,Statewide,0,2500,COP,1.0,S00-Ann,220,0.984028,1-Weekday (Tu-Th),216,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,88.4088,1.0572,76.118400,93.465783,0.071473,0.087761
3,0006_146.9,1.0,1671.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3521,AM,0.2124,4.8,10.4,2023,1600.0,1600.0,119.657664,Statewide,0,2500,COP,1.0,S00-Ann,119,0.984028,1-Weekday (Tu-Th),117,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,47.8881,1.0616,41.402400,50.838007,0.038875,0.047735
4,0065_000.1,1.0,399.0,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6750,AM,0.2005,7.4,1.9,2023,1600.0,1600.0,216.540000,Statewide,0,2500,COP,1.0,S00-Ann,216,0.984028,1-Weekday (Tu-Th),212,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,86.7716,1.0243,72.383867,88.880150,0.089806,0.110273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26563,0215_025.7,3.0,121975.2,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4446,PM,0.2705,10.2,6.2,2023,85000.0,85000.0,10222.465500,I215,0,1000000,CO8,1.0,S00-Ann,10222,1.134873,1-Weekday (Tu-Th),11600,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4245.6000,1.0514,4065.413333,4463.823840,0.543214,0.596449
26564,0215_026.7,3.0,58801.3,Freeway,1.0,D1,3.0,34.0,Freeway,1964,0.5220,PM,0.2452,19.8,11.9,2023,27000.0,27000.0,3455.848800,I215,0,1000000,CO8,1.0,S00-Ann,3455,1.134873,1-Weekday (Tu-Th),3920,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1434.7200,1.0991,1436.157333,1576.900752,0.243747,0.267634
26565,0215_026.7,3.0,58801.3,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.4780,PM,0.2452,21.6,13.2,2023,27000.0,27000.0,3164.551200,I215,0,1000000,CO8,1.0,S00-Ann,3164,1.134873,1-Weekday (Tu-Th),3590,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1313.9400,1.1092,1327.342667,1457.422248,0.319073,0.350342
26566,0215_027.4,3.0,39438.0,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5606,PM,0.2461,20.6,11.7,2023,23500.0,23500.0,3242.146010,I215,0,1000000,CO8,1.0,S00-Ann,3242,1.134873,1-Weekday (Tu-Th),3679,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1346.5140,1.0997,1348.598767,1480.761446,0.324182,0.355952


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5827,AM,0.2106,9.1,7.5,2028,1700.0,1700.0,208.618254,Statewide,0,2500,COP,1.0,S00-Ann,208,0.984028,1-Weekday (Tu-Th),204,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.4972,1.0557,71.787600,88.147994,0.067406,0.082768
1,0006_141.0,1.0,1781.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4173,AM,0.2106,7.1,7.5,2028,1700.0,1700.0,149.401746,Statewide,0,2500,COP,1.0,S00-Ann,149,0.984028,1-Weekday (Tu-Th),146,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,59.7578,1.0517,51.182733,62.847278,0.048059,0.059012
2,0006_146.9,1.0,1786.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6419,AM,0.2111,9.5,7.4,2028,1800.0,1800.0,243.909162,Statewide,0,2500,COP,1.0,S00-Ann,243,0.984028,1-Weekday (Tu-Th),239,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,97.8227,1.0560,84.128000,103.300771,0.078993,0.096996
3,0006_146.9,1.0,1786.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3581,AM,0.2111,5.9,7.4,2028,1800.0,1800.0,136.070838,Statewide,0,2500,COP,1.0,S00-Ann,136,0.984028,1-Weekday (Tu-Th),133,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.4369,1.0488,46.496800,57.093421,0.043659,0.053609
4,0065_000.1,1.0,431.0,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6747,AM,0.1926,5.4,0.0,2028,1600.0,1600.0,207.915552,Statewide,0,2500,COP,1.0,S00-Ann,207,0.984028,1-Weekday (Tu-Th),203,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.0879,1.0108,68.397467,83.985249,0.084860,0.104200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27037,0215_025.7,3.0,129195.0,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4369,PM,0.2590,11.6,7.3,2028,91500.0,91500.0,10353.874650,I215,0,1000000,CO8,1.0,S00-Ann,10353,1.134873,1-Weekday (Tu-Th),11749,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4300.1340,1.0597,4150.138433,4556.852000,0.554535,0.608879
27038,0215_026.7,3.0,56537.5,Freeway,1.0,D1,3.0,34.0,Freeway,1964,0.5606,PM,0.2628,11.7,7.0,2028,25000.0,25000.0,3683.142000,I215,0,1000000,CO8,1.0,S00-Ann,3683,1.134873,1-Weekday (Tu-Th),4179,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1529.5140,1.0584,1474.351200,1618.837618,0.250229,0.274752
27039,0215_026.7,3.0,56537.5,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.4394,PM,0.2628,14.3,8.9,2028,25000.0,25000.0,2886.858000,I215,0,1000000,CO8,1.0,S00-Ann,2886,1.134873,1-Weekday (Tu-Th),3275,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1198.6500,1.0731,1171.467500,1286.271315,0.281603,0.309200
27040,0215_027.4,3.0,37297.2,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.6149,PM,0.2756,8.9,5.2,2028,21500.0,21500.0,3643.528460,I215,0,1000000,CO8,1.0,S00-Ann,3643,1.134873,1-Weekday (Tu-Th),4134,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1513.0440,1.0438,1438.356400,1579.315327,0.345759,0.379643


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6022,AM,0.2031,8.7,6.6,2032,1800.0,1800.0,220.152276,Statewide,0,2500,COP,1.0,S00-Ann,220,0.984028,1-Weekday (Tu-Th),216,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,88.4088,1.0504,75.628800,92.864604,0.071013,0.087197
1,0006_141.0,1.0,1871.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3978,AM,0.2031,5.9,6.5,2032,1800.0,1800.0,145.427724,Statewide,0,2500,COP,1.0,S00-Ann,145,0.984028,1-Weekday (Tu-Th),142,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,58.1206,1.0443,49.430200,60.695343,0.046413,0.056991
2,0006_146.9,1.0,1882.8,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6754,AM,0.2029,9.3,6.6,2032,1800.0,1800.0,246.669588,Statewide,0,2500,COP,1.0,S00-Ann,246,0.984028,1-Weekday (Tu-Th),242,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,99.0506,1.0516,84.829067,104.161611,0.079652,0.097804
3,0006_146.9,1.0,1882.8,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3246,AM,0.2029,4.0,6.5,2032,1800.0,1800.0,118.550412,Statewide,0,2500,COP,1.0,S00-Ann,118,0.984028,1-Weekday (Tu-Th),116,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,47.4788,1.0405,40.232667,49.401691,0.037777,0.046387
4,0065_000.1,1.0,453.8,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6019,AM,0.1882,7.8,1.9,2032,1600.0,1600.0,181.244128,Statewide,0,2500,COP,1.0,S00-Ann,181,0.984028,1-Weekday (Tu-Th),178,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,72.8554,1.0251,60.822600,74.684071,0.075462,0.092660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28309,0215_025.7,3.0,144038.4,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4412,PM,0.2527,12.4,7.1,2032,101000.0,101000.0,11260.615240,I215,0,1000000,CO8,1.0,S00-Ann,11260,1.134873,1-Weekday (Tu-Th),12778,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4676.7480,1.0603,4516.171133,4958.755904,0.506297,0.555914
28310,0215_026.7,3.0,61961.0,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5411,PM,0.2497,19.9,10.2,2032,27500.0,27500.0,3715.598425,I215,0,1000000,CO8,1.0,S00-Ann,3715,1.134873,1-Weekday (Tu-Th),4216,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1543.0560,1.0908,1532.937600,1683.165485,0.204829,0.224902
28311,0215_026.7,3.0,61961.0,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4589,PM,0.2497,13.6,7.0,2032,27500.0,27500.0,3151.151575,I215,0,1000000,CO8,1.0,S00-Ann,3151,1.134873,1-Weekday (Tu-Th),3575,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1308.4500,1.0622,1265.788333,1389.835590,0.169133,0.185708
28312,0215_027.4,3.0,50607.5,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5447,PM,0.2003,23.2,12.2,2032,33000.0,33000.0,3600.412530,I215,0,1000000,CO8,1.0,S00-Ann,3600,1.134873,1-Weekday (Tu-Th),4085,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1495.1100,1.1074,1507.909667,1655.684814,0.362478,0.398001


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6118,AM,0.2047,8.4,7.5,2042,2100.0,2100.0,262.994466,Statewide,0,2500,COP,1.0,S00-Ann,262,0.984028,1-Weekday (Tu-Th),257,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,105.1901,1.0543,90.318367,110.901922,0.084806,0.104133
1,0006_141.0,1.0,2115.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3882,AM,0.2047,5.3,8.4,2042,2100.0,2100.0,166.875534,Statewide,0,2500,COP,1.0,S00-Ann,166,0.984028,1-Weekday (Tu-Th),163,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,66.7159,1.0526,57.191267,70.225156,0.053701,0.065939
2,0006_146.9,1.0,2122.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6920,AM,0.2050,9.0,7.3,2042,2100.0,2100.0,297.906000,Statewide,0,2500,COP,1.0,S00-Ann,297,0.984028,1-Weekday (Tu-Th),292,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,119.5156,1.0545,102.638000,126.029200,0.096374,0.118337
3,0006_146.9,1.0,2122.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3080,AM,0.2050,3.0,9.0,2042,2100.0,2100.0,132.594000,Statewide,0,2500,COP,1.0,S00-Ann,132,0.984028,1-Weekday (Tu-Th),129,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,52.7997,1.0510,45.193000,55.492485,0.042435,0.052106
4,0065_000.1,1.0,537.8,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6514,AM,0.1867,3.1,1.5,2042,1700.0,1700.0,206.747846,Statewide,0,2500,COP,1.0,S00-Ann,206,0.984028,1-Weekday (Tu-Th),202,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,82.6786,1.0137,68.255800,83.811297,0.084685,0.103984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29053,0215_025.7,3.0,158610.4,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4349,PM,0.2482,11.9,6.5,2042,105000.0,105000.0,11333.928900,I215,0,1000000,CO8,1.0,S00-Ann,11333,1.134873,1-Weekday (Tu-Th),12861,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4707.1260,1.0563,4528.358100,4972.137194,0.507663,0.557414
29054,0215_026.7,3.0,66703.6,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5625,PM,0.2689,16.3,8.6,2042,28500.0,28500.0,4310.803125,I215,0,1000000,CO8,1.0,S00-Ann,4310,1.134873,1-Weekday (Tu-Th),4891,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1790.1060,1.0756,1753.586533,1925.438014,0.234311,0.257274
29055,0215_026.7,3.0,66703.6,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4375,PM,0.2689,12.6,6.6,2042,28500.0,28500.0,3352.846875,I215,0,1000000,CO8,1.0,S00-Ann,3352,1.134873,1-Weekday (Tu-Th),3804,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1392.2640,1.0582,1341.797600,1473.293765,0.179289,0.196859
29056,0215_027.4,3.0,54625.5,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5597,PM,0.2271,18.5,12.4,2042,35000.0,35000.0,4448.775450,I215,0,1000000,CO8,1.0,S00-Ann,4448,1.134873,1-Weekday (Tu-Th),5047,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1847.2020,1.0990,1848.884333,2030.074998,0.444443,0.487999


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6067,AM,0.2006,9.1,7.7,2050,2300.0,2500,304.260050,Statewide,0,2500,COP,1.0,S00-Ann,304,0.984028,1-Weekday (Tu-Th),299,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,122.3807,1.0567,105.317767,129.319686,0.098890,0.121427
1,0006_141.0,1.0,2303.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3933,AM,0.2006,7.4,8.4,2050,2300.0,2500,197.239950,Statewide,0,2500,COP,1.0,S00-Ann,197,0.984028,1-Weekday (Tu-Th),193,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,78.9949,1.0568,67.987467,83.481810,0.063838,0.078387
2,0006_146.9,1.0,2307.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6832,AM,0.2011,9.5,7.6,2050,2300.0,2500,343.478800,Statewide,0,2500,COP,1.0,S00-Ann,343,0.984028,1-Weekday (Tu-Th),337,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,137.9341,1.0570,118.736333,145.796344,0.111490,0.136898
3,0006_146.9,1.0,2307.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3168,AM,0.2011,6.1,8.8,2050,2300.0,2500,159.271200,Statewide,0,2500,COP,1.0,S00-Ann,159,0.984028,1-Weekday (Tu-Th),156,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.8508,1.0562,54.922400,67.439215,0.051570,0.063323
4,0065_000.1,1.0,614.1,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6182,AM,0.1791,2.9,1.5,2050,1800.0,1900,210.367278,Statewide,0,2500,COP,1.0,S00-Ann,210,0.984028,1-Weekday (Tu-Th),206,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,84.3158,1.0133,69.579933,85.437200,0.086327,0.106001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29089,0215_025.7,3.0,141936.9,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4293,PM,0.2350,10.0,5.9,2050,111000.0,94500,9533.679750,I215,0,1000000,CO8,1.0,S00-Ann,9533,1.134873,1-Weekday (Tu-Th),10818,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,3959.3880,1.0495,3784.497000,4155.377706,0.424271,0.465850
29090,0215_026.7,3.0,59122.6,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5447,PM,0.2521,9.9,6.9,2050,25500.0,18000,2471.739660,I215,0,1000000,CO8,1.0,S00-Ann,2471,1.134873,1-Weekday (Tu-Th),2804,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1026.2640,1.0543,985.419067,1081.990135,0.131670,0.144574
29091,0215_026.7,3.0,59122.6,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4553,PM,0.2521,13.6,7.3,2050,25500.0,18000,2066.060340,I215,0,1000000,CO8,1.0,S00-Ann,2066,1.134873,1-Weekday (Tu-Th),2344,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,857.9040,1.0637,831.104267,912.552485,0.111051,0.121934
29092,0215_027.4,3.0,52448.7,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5207,PM,0.2238,5.1,5.0,2050,34500.0,32500,3787.311450,I215,0,1000000,CO8,1.0,S00-Ann,3787,1.134873,1-Weekday (Tu-Th),4297,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1572.7020,1.0352,1482.751467,1628.061110,0.356431,0.391361


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5771,AM,0.2118,8.9,6.4,2032,1800.0,1800,220.013604,Statewide,0,2500,COP,1.0,S00-Ann,220,0.984028,1-Weekday (Tu-Th),216,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,88.4088,1.0498,75.585600,92.811558,0.070972,0.087147
1,0006_141.0,1.0,1917.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4229,AM,0.2118,7.1,5.8,2032,1800.0,1800,161.226396,Statewide,0,2500,COP,1.0,S00-Ann,161,0.984028,1-Weekday (Tu-Th),158,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,64.6694,1.0432,54.941867,67.463118,0.051589,0.063346
2,0006_146.9,1.0,1926.8,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6339,AM,0.2112,9.3,6.6,2032,1800.0,1800,240.983424,Statewide,0,2500,COP,1.0,S00-Ann,240,0.984028,1-Weekday (Tu-Th),236,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,96.5948,1.0516,82.725867,101.579092,0.077677,0.095379
3,0006_146.9,1.0,1926.8,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3661,AM,0.2112,6.0,5.4,2032,1800.0,1800,139.176576,Statewide,0,2500,COP,1.0,S00-Ann,139,0.984028,1-Weekday (Tu-Th),136,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,55.6648,1.0390,47.101333,57.835727,0.044227,0.054306
4,0065_000.1,1.0,453.5,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6193,AM,0.1969,5.8,0.0,2032,1600.0,1600,195.104272,Statewide,0,2500,COP,1.0,S00-Ann,195,0.984028,1-Weekday (Tu-Th),191,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,78.1763,1.0116,64.405200,79.083145,0.079907,0.098118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28567,0215_025.7,3.0,144306.8,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4420,PM,0.2532,12.6,7.2,2032,101000.0,101000,11303.354400,I215,0,1000000,CO8,1.0,S00-Ann,11303,1.134873,1-Weekday (Tu-Th),12827,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4694.6820,1.0612,4537.337467,4981.996538,0.508670,0.558520
28568,0215_026.7,3.0,61753.3,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5372,PM,0.2523,19.2,9.3,2032,27500.0,27500,3727.227900,I215,0,1000000,CO8,1.0,S00-Ann,3727,1.134873,1-Weekday (Tu-Th),4229,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1547.8140,1.0849,1529.347367,1679.223409,0.204349,0.224375
28569,0215_026.7,3.0,61753.3,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4628,PM,0.2523,13.6,6.9,2032,27500.0,27500,3211.022100,I215,0,1000000,CO8,1.0,S00-Ann,3211,1.134873,1-Weekday (Tu-Th),3644,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1333.7040,1.0617,1289.611600,1415.993537,0.172316,0.189203
28570,0215_027.4,3.0,49902.1,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5389,PM,0.2011,23.1,13.2,2032,33000.0,32500,3522.115675,I215,0,1000000,CO8,1.0,S00-Ann,3522,1.134873,1-Weekday (Tu-Th),3997,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1462.9020,1.1122,1481.821133,1627.039604,0.356207,0.391115


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6106,AM,0.2060,8.1,7.3,2042,2100.0,2100,264.145560,Statewide,0,2500,COP,1.0,S00-Ann,264,0.984028,1-Weekday (Tu-Th),259,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,106.0087,1.0527,90.883100,111.595358,0.085336,0.104784
1,0006_141.0,1.0,2151.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3894,AM,0.2060,4.7,7.7,2042,2100.0,2100,168.454440,Statewide,0,2500,COP,1.0,S00-Ann,168,0.984028,1-Weekday (Tu-Th),165,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,67.5345,1.0479,57.634500,70.769403,0.054117,0.066450
2,0006_146.9,1.0,2157.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6914,AM,0.2058,8.8,7.2,2042,2100.0,2100,298.809252,Statewide,0,2500,COP,1.0,S00-Ann,298,0.984028,1-Weekday (Tu-Th),293,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,119.9249,1.0536,102.901600,126.352875,0.096621,0.118641
3,0006_146.9,1.0,2157.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3086,AM,0.2058,2.2,8.0,2042,2100.0,2100,133.370748,Statewide,0,2500,COP,1.0,S00-Ann,133,0.984028,1-Weekday (Tu-Th),130,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.2090,1.0444,45.257333,55.571480,0.042495,0.052180
4,0065_000.1,1.0,529.0,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6515,AM,0.1953,4.5,1.5,2042,1700.0,1700,216.304515,Statewide,0,2500,COP,1.0,S00-Ann,216,0.984028,1-Weekday (Tu-Th),212,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,86.7716,1.0165,71.832667,88.203331,0.089122,0.109433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29851,0215_025.7,3.0,160828.3,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4333,PM,0.2562,11.8,6.2,2042,105000.0,107000,11878.226220,I215,0,1000000,CO8,1.0,S00-Ann,11878,1.134873,1-Weekday (Tu-Th),13480,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4933.6800,1.0546,4738.669333,5203.058928,0.531241,0.583303
29852,0215_026.7,3.0,64618.9,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5552,PM,0.2597,17.5,9.2,2042,28500.0,26500,3820.914160,I215,0,1000000,CO8,1.0,S00-Ann,3820,1.134873,1-Weekday (Tu-Th),4335,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1586.6100,1.0810,1562.045000,1715.125410,0.208718,0.229172
29853,0215_026.7,3.0,64618.9,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4448,PM,0.2597,13.3,6.5,2042,28500.0,26500,3061.135840,I215,0,1000000,CO8,1.0,S00-Ann,3061,1.134873,1-Weekday (Tu-Th),3473,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1271.1180,1.0591,1226.084767,1346.241074,0.163827,0.179883
29854,0215_027.4,3.0,56849.7,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5251,PM,0.2203,19.4,13.4,2042,35000.0,37000,4280.142610,I215,0,1000000,CO8,1.0,S00-Ann,4280,1.134873,1-Weekday (Tu-Th),4857,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1777.6620,1.1058,1790.290200,1965.738640,0.430358,0.472533


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6183,AM,0.1974,9.0,7.5,2050,2300.0,2300,280.720566,Statewide,0,2500,COP,1.0,S00-Ann,280,0.984028,1-Weekday (Tu-Th),275,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,112.5575,1.0555,96.754167,118.804441,0.090849,0.111553
1,0006_141.0,1.0,2346.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3817,AM,0.1974,8.0,8.2,2050,2300.0,2300,173.299434,Statewide,0,2500,COP,1.0,S00-Ann,173,0.984028,1-Weekday (Tu-Th),170,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,69.5810,1.0570,59.896667,73.547117,0.056241,0.069058
2,0006_146.9,1.0,2351.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.7047,AM,0.1974,9.2,7.3,2050,2300.0,2300,319.947894,Statewide,0,2500,COP,1.0,S00-Ann,319,0.984028,1-Weekday (Tu-Th),313,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,128.1109,1.0549,110.061233,135.144188,0.103344,0.126896
3,0006_146.9,1.0,2351.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.2953,AM,0.1974,7.3,8.8,2050,2300.0,2300,134.072106,Statewide,0,2500,COP,1.0,S00-Ann,134,0.984028,1-Weekday (Tu-Th),131,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.6183,1.0586,46.225533,56.760332,0.043404,0.053296
4,0065_000.1,1.0,608.3,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6490,AM,0.1920,6.6,4.0,2050,1800.0,1800,224.294400,Statewide,0,2500,COP,1.0,S00-Ann,224,0.984028,1-Weekday (Tu-Th),220,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,90.0460,1.0332,75.768000,93.035527,0.094005,0.115429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31591,0215_025.7,3.0,168749.3,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4207,PM,0.2483,10.4,6.3,2050,111000.0,113000,11803.958530,I215,0,1000000,CO8,1.0,S00-Ann,11803,1.134873,1-Weekday (Tu-Th),13394,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4902.2040,1.0523,4698.168733,5158.589269,0.526701,0.578317
31592,0215_026.7,3.0,66134.1,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5845,PM,0.2563,16.6,9.8,2050,25500.0,28000,4194.605800,I215,0,1000000,CO8,1.0,S00-Ann,4194,1.134873,1-Weekday (Tu-Th),4759,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1741.7940,1.0822,1716.729933,1884.969467,0.229387,0.251867
31593,0215_026.7,3.0,66134.1,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4155,PM,0.2563,13.2,7.4,2050,25500.0,28000,2981.794200,I215,0,1000000,CO8,1.0,S00-Ann,2981,1.134873,1-Weekday (Tu-Th),3383,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1238.1780,1.0634,1199.160733,1316.678485,0.160230,0.175932
31594,0215_027.4,3.0,58652.9,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5818,PM,0.2251,16.5,10.3,2050,34500.0,39000,5107.564020,I215,0,1000000,CO8,1.0,S00-Ann,5107,1.134873,1-Weekday (Tu-Th),5795,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,2120.9700,1.0845,2094.892500,2300.191965,0.503580,0.552931


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6145,AM,0.1973,9.0,7.6,2050,2300.0,2300,278.853955,Statewide,0,2500,COP,1.0,S00-Ann,278,0.984028,1-Weekday (Tu-Th),273,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,111.7389,1.0560,96.096000,117.996278,0.090231,0.110795
1,0006_141.0,1.0,2362.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3855,AM,0.1973,7.9,8.5,2050,2300.0,2300,174.936045,Statewide,0,2500,COP,1.0,S00-Ann,174,0.984028,1-Weekday (Tu-Th),171,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,69.9903,1.0583,60.323100,74.070734,0.056641,0.069550
2,0006_146.9,1.0,2365.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6981,AM,0.1975,9.2,7.4,2050,2300.0,2300,317.111925,Statewide,0,2500,COP,1.0,S00-Ann,317,0.984028,1-Weekday (Tu-Th),311,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,127.2923,1.0554,109.409800,134.344293,0.102732,0.126145
3,0006_146.9,1.0,2365.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3019,AM,0.1975,7.1,9.2,2050,2300.0,2300,137.138075,Statewide,0,2500,COP,1.0,S00-Ann,137,0.984028,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.8462,1.0602,47.355600,58.147941,0.044465,0.054599
4,0065_000.1,1.0,606.3,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6303,AM,0.1829,4.3,1.4,2050,1800.0,1800,207.507366,Statewide,0,2500,COP,1.0,S00-Ann,207,0.984028,1-Weekday (Tu-Th),203,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.0879,1.0156,68.722267,84.384071,0.085263,0.104695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31597,0215_025.7,3.0,168159.4,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4204,PM,0.2479,10.4,6.3,2050,111000.0,112000,11672.321920,I215,0,1000000,CO8,1.0,S00-Ann,11672,1.134873,1-Weekday (Tu-Th),13246,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4848.0360,1.0523,4646.255267,5101.588283,0.520881,0.571927
31598,0215_026.7,3.0,65775.5,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5857,PM,0.2546,16.9,9.9,2050,25500.0,27500,4100.778550,I215,0,1000000,CO8,1.0,S00-Ann,4100,1.134873,1-Weekday (Tu-Th),4652,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1702.6320,1.0833,1679.837200,1844.461246,0.224457,0.246454
31599,0215_026.7,3.0,65775.5,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4143,PM,0.2546,13.3,7.8,2050,25500.0,27500,2900.721450,I215,0,1000000,CO8,1.0,S00-Ann,2900,1.134873,1-Weekday (Tu-Th),3291,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1204.5060,1.0656,1168.963200,1283.521594,0.156195,0.171502
31600,0215_027.4,3.0,58587.8,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5822,PM,0.2219,16.5,10.6,2050,34500.0,39000,5038.417020,I215,0,1000000,CO8,1.0,S00-Ann,5038,1.134873,1-Weekday (Tu-Th),5717,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,2092.4220,1.0860,2069.554000,2272.370292,0.497489,0.546243


In [78]:
df_PkHrForecasts.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd_TDM', 'PRDFAC',
       'MDPCT', 'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj',
       'AvgAnnualPrdVol', 'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP',
       'SsnFactor', 'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol', 'YEARGROUP', 'PERIOD', 'VOLPKHRPCT', 'HRPCTOF',
       'SeasonType', 'VCGroupCode', 'PRDHRS', 'AvgSsnDOWPkHrVol', 'TrkFac',
       'PrdPCEFlow', 'PkHrPCEFlow', 'PrdVC', 'PkHrVC'],
      dtype='object')

In [79]:
df_PkHrForecasts[df_PkHrForecasts['SEASONGROUP']=='S00-Ann']

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6145,AM,0.1973,9.0,7.6,2050,2300.0,2300,278.853955,Statewide,0,2500,COP,1.0,S00-Ann,278,0.984028,1-Weekday (Tu-Th),273,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,111.7389,1.0560,96.096000,117.996278,0.090231,0.110795
1,0006_141.0,1.0,2362.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3855,AM,0.1973,7.9,8.5,2050,2300.0,2300,174.936045,Statewide,0,2500,COP,1.0,S00-Ann,174,0.984028,1-Weekday (Tu-Th),171,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,69.9903,1.0583,60.323100,74.070734,0.056641,0.069550
2,0006_146.9,1.0,2365.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6981,AM,0.1975,9.2,7.4,2050,2300.0,2300,317.111925,Statewide,0,2500,COP,1.0,S00-Ann,317,0.984028,1-Weekday (Tu-Th),311,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,127.2923,1.0554,109.409800,134.344293,0.102732,0.126145
3,0006_146.9,1.0,2365.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3019,AM,0.1975,7.1,9.2,2050,2300.0,2300,137.138075,Statewide,0,2500,COP,1.0,S00-Ann,137,0.984028,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.8462,1.0602,47.355600,58.147941,0.044465,0.054599
4,0065_000.1,1.0,606.3,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6303,AM,0.1829,4.3,1.4,2050,1800.0,1800,207.507366,Statewide,0,2500,COP,1.0,S00-Ann,207,0.984028,1-Weekday (Tu-Th),203,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.0879,1.0156,68.722267,84.384071,0.085263,0.104695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31597,0215_025.7,3.0,168159.4,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4204,PM,0.2479,10.4,6.3,2050,111000.0,112000,11672.321920,I215,0,1000000,CO8,1.0,S00-Ann,11672,1.134873,1-Weekday (Tu-Th),13246,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4848.0360,1.0523,4646.255267,5101.588283,0.520881,0.571927
31598,0215_026.7,3.0,65775.5,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5857,PM,0.2546,16.9,9.9,2050,25500.0,27500,4100.778550,I215,0,1000000,CO8,1.0,S00-Ann,4100,1.134873,1-Weekday (Tu-Th),4652,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1702.6320,1.0833,1679.837200,1844.461246,0.224457,0.246454
31599,0215_026.7,3.0,65775.5,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4143,PM,0.2546,13.3,7.8,2050,25500.0,27500,2900.721450,I215,0,1000000,CO8,1.0,S00-Ann,2900,1.134873,1-Weekday (Tu-Th),3291,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1204.5060,1.0656,1168.963200,1283.521594,0.156195,0.171502
31600,0215_027.4,3.0,58587.8,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5822,PM,0.2219,16.5,10.6,2050,34500.0,39000,5038.417020,I215,0,1000000,CO8,1.0,S00-Ann,5038,1.134873,1-Weekday (Tu-Th),5717,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,2092.4220,1.0860,2069.554000,2272.370292,0.497489,0.546243


In [80]:
#15-Min Analysis

dfs_VC = []

for df_PkHrForecasts in dfs_PkHrForecasts:

    df_VC = df_PkHrForecasts
    
    #PHF conditions (from Chad's spreadsheet for Urban (0.92,0.95,0.97,0.98). HCM2000 Exhibit 9.2 says default PHF for Urban Areas is 0.92, Rural is 0.88. Rural estimated at (0.88,0.91,0.93,0.95)
    conditions = [
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] <= 0.85),
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] >  0.85) & (df_VC['PkHrVC'] <= 0.95),
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] >  0.95) & (df_VC['PkHrVC'] <= 1.00),
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] >  1.00),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] <= 0.85),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] >  0.85) & (df_VC['PkHrVC'] <= 0.95),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] >  0.95) & (df_VC['PkHrVC'] <= 1.00),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] >  1.00)
    ]
    choices = [0.92,0.95,0.97,0.98,0.88,0.91,0.93,0.95]

    df_VC['PHF'] = np.select(conditions, choices, default=0)


    #15-min volume and v/c calcs
    df_VC['Pk15PCEFlow'] = df_VC['PkHrPCEFlow']  / df_VC['PHF']
    df_VC['Pk15VC'     ] = df_VC['Pk15PCEFlow'] / (df_VC[nameCap1HL] * df_VC[nameLanes])

    #effective K-factor
    #df_VC['kFactor_effective']  = df_VC['ForecastPkHr']  / df_VC['ForecastAADT']

    #convert to int and round columns after calcs
    df_VC = df_VC.fillna(0)
    df_VC['AREATYPE'       ] = df_VC['AREATYPE'       ].astype(int)
    df_VC['FT'             ] = df_VC['FT'             ].astype(int)
    df_VC['LANES'          ] = df_VC['LANES'          ].astype(int)
    df_VC['ForecastAADT'   ] = df_VC['ForecastAADT'   ].astype(int)
    df_VC['ForecastAADTAdj'] = df_VC['ForecastAADTAdj'].astype(int)
    df_VC['PrdPCEFlow'     ] = df_VC['PrdPCEFlow'     ].astype(int)
    df_VC['PkHrPCEFlow'    ] = df_VC['PkHrPCEFlow'    ].astype(int)
    df_VC['Pk15PCEFlow'    ] = df_VC['Pk15PCEFlow'    ].astype(int)
    df_VC = df_VC.round({'SeasonFactor':2,'DOWFactor':2,'PrdVC':2,'PkHrVC':2,'Pk15VC':2,nameMDPct:2,nameHVPct:2,'VolMaxHrPct':2,'FactorTrucks':2})
    
    pd.set_option('display.max_columns', None)
    
    display(df_VC)
    
    dfs_VC.append(df_VC)
    

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,1566.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6045,AM,0.2037,9.1,8.2,2019,1517,1517,186.798298,Statewide,0,2500,COP,1.0,S00-Ann,186,0.98,1-Weekday (Tu-Th),183,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,74.9019,1.0592,64,79,0.06,0.07,0.88,90,0.08
1,0006_141.0,1,1566.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3955,AM,0.2037,8.3,9.7,2019,1517,1517,122.214602,Statewide,0,2500,COP,1.0,S00-Ann,122,0.98,1-Weekday (Tu-Th),120,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,49.1160,1.0651,42,52,0.04,0.05,0.88,59,0.06
2,0006_146.9,1,1571.8,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6779,AM,0.2034,9.1,7.8,2019,1517,1517,209.171333,Statewide,0,2500,COP,1.0,S00-Ann,209,0.98,1-Weekday (Tu-Th),205,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.9065,1.0572,72,88,0.07,0.08,0.88,100,0.09
3,0006_146.9,1,1571.8,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3221,AM,0.2034,7.8,10.7,2019,1517,1517,99.386467,Statewide,0,2500,COP,1.0,S00-Ann,99,0.98,1-Weekday (Tu-Th),97,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,39.7021,1.0691,34,42,0.03,0.04,0.88,48,0.05
4,0006_149.9,1,1635.2,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6609,AM,0.1672,10.5,6.1,2019,2441,2441,269.736554,Statewide,0,2500,COP,1.0,S00-Ann,269,0.98,1-Weekday (Tu-Th),264,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,108.0552,1.0515,92,113,0.09,0.11,0.88,129,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25909,0215_025.7,2,109267.4,Freeway,1.0,D2,4,34,Freeway,1871,0.4381,PM,0.2779,10.8,6.5,2019,75533,75533,9195.990929,I215,0,1000000,CO8,1.0,S00-Ann,9195,1.13,1-Weekday (Tu-Th),10435,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,3819.2100,1.0541,3666,4025,0.49,0.54,0.88,4574,0.61
25910,0215_026.7,3,72892.6,Freeway,1.0,D1,3,34,Freeway,1964,0.5014,PM,0.1989,19.7,11.6,2019,42492,42492,4237.661722,I215,0,1000000,CO8,1.0,S00-Ann,4237,1.13,1-Weekday (Tu-Th),4808,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1759.7280,1.0974,1758,1931,0.30,0.33,0.92,2099,0.36
25911,0215_026.7,3,72892.6,Freeway,1.0,D2,2,34,Freeway,2080,0.4986,PM,0.1989,19.9,12.0,2019,42492,42492,4213.997078,I215,0,1000000,CO8,1.0,S00-Ann,4213,1.13,1-Weekday (Tu-Th),4781,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1749.8460,1.0998,1752,1924,0.42,0.46,0.92,2091,0.50
25912,0215_027.4,3,55027.9,Freeway,1.0,D1,2,34,Freeway,2080,0.5304,PM,0.1788,20.9,12.1,2019,39875,39875,3781.566360,I215,0,1000000,CO8,1.0,S00-Ann,3781,1.13,1-Weekday (Tu-Th),4290,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1570.1400,1.1023,1576,1730,0.38,0.42,0.92,1881,0.45


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,1663.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5862,AM,0.2123,8.6,8.2,2023,1600,1600,199.120416,Statewide,0,2500,COP,1.0,S00-Ann,199,0.98,1-Weekday (Tu-Th),195,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,79.8135,1.0582,68,84,0.06,0.08,0.88,95,0.09
1,0006_141.0,1,1663.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4138,AM,0.2123,6.3,9.6,2023,1600,1600,140.559584,Statewide,0,2500,COP,1.0,S00-Ann,140,0.98,1-Weekday (Tu-Th),137,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.0741,1.0606,48,59,0.05,0.06,0.88,67,0.06
2,0006_146.9,1,1671.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6479,AM,0.2124,9.1,7.8,2023,1600,1600,220.182336,Statewide,0,2500,COP,1.0,S00-Ann,220,0.98,1-Weekday (Tu-Th),216,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,88.4088,1.0572,76,93,0.07,0.09,0.88,106,0.10
3,0006_146.9,1,1671.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3521,AM,0.2124,4.8,10.4,2023,1600,1600,119.657664,Statewide,0,2500,COP,1.0,S00-Ann,119,0.98,1-Weekday (Tu-Th),117,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,47.8881,1.0616,41,50,0.04,0.05,0.88,57,0.05
4,0065_000.1,1,399.0,Arterial,1.0,D1,1,4,Collector,806,0.6750,AM,0.2005,7.4,1.9,2023,1600,1600,216.540000,Statewide,0,2500,COP,1.0,S00-Ann,216,0.98,1-Weekday (Tu-Th),212,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,86.7716,1.0243,72,88,0.09,0.11,0.88,101,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26563,0215_025.7,3,121975.2,Freeway,1.0,D2,4,34,Freeway,1871,0.4446,PM,0.2705,10.2,6.2,2023,85000,85000,10222.465500,I215,0,1000000,CO8,1.0,S00-Ann,10222,1.13,1-Weekday (Tu-Th),11600,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4245.6000,1.0514,4065,4463,0.54,0.60,0.92,4851,0.65
26564,0215_026.7,3,58801.3,Freeway,1.0,D1,3,34,Freeway,1964,0.5220,PM,0.2452,19.8,11.9,2023,27000,27000,3455.848800,I215,0,1000000,CO8,1.0,S00-Ann,3455,1.13,1-Weekday (Tu-Th),3920,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1434.7200,1.0991,1436,1576,0.24,0.27,0.92,1714,0.29
26565,0215_026.7,3,58801.3,Freeway,1.0,D2,2,34,Freeway,2080,0.4780,PM,0.2452,21.6,13.2,2023,27000,27000,3164.551200,I215,0,1000000,CO8,1.0,S00-Ann,3164,1.13,1-Weekday (Tu-Th),3590,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1313.9400,1.1092,1327,1457,0.32,0.35,0.92,1584,0.38
26566,0215_027.4,3,39438.0,Freeway,1.0,D1,2,34,Freeway,2080,0.5606,PM,0.2461,20.6,11.7,2023,23500,23500,3242.146010,I215,0,1000000,CO8,1.0,S00-Ann,3242,1.13,1-Weekday (Tu-Th),3679,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1346.5140,1.0997,1348,1480,0.32,0.36,0.92,1609,0.39


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,1781.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5827,AM,0.2106,9.1,7.5,2028,1700,1700,208.618254,Statewide,0,2500,COP,1.0,S00-Ann,208,0.98,1-Weekday (Tu-Th),204,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.4972,1.0557,71,88,0.07,0.08,0.88,100,0.09
1,0006_141.0,1,1781.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4173,AM,0.2106,7.1,7.5,2028,1700,1700,149.401746,Statewide,0,2500,COP,1.0,S00-Ann,149,0.98,1-Weekday (Tu-Th),146,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,59.7578,1.0517,51,62,0.05,0.06,0.88,71,0.07
2,0006_146.9,1,1786.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6419,AM,0.2111,9.5,7.4,2028,1800,1800,243.909162,Statewide,0,2500,COP,1.0,S00-Ann,243,0.98,1-Weekday (Tu-Th),239,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,97.8227,1.0560,84,103,0.08,0.10,0.88,117,0.11
3,0006_146.9,1,1786.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3581,AM,0.2111,5.9,7.4,2028,1800,1800,136.070838,Statewide,0,2500,COP,1.0,S00-Ann,136,0.98,1-Weekday (Tu-Th),133,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.4369,1.0488,46,57,0.04,0.05,0.88,64,0.06
4,0065_000.1,1,431.0,Arterial,1.0,D1,1,4,Collector,806,0.6747,AM,0.1926,5.4,0.0,2028,1600,1600,207.915552,Statewide,0,2500,COP,1.0,S00-Ann,207,0.98,1-Weekday (Tu-Th),203,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.0879,1.0108,68,83,0.08,0.10,0.88,95,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27037,0215_025.7,3,129195.0,Freeway,1.0,D2,4,34,Freeway,1871,0.4369,PM,0.2590,11.6,7.3,2028,91500,91500,10353.874650,I215,0,1000000,CO8,1.0,S00-Ann,10353,1.13,1-Weekday (Tu-Th),11749,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4300.1340,1.0597,4150,4556,0.55,0.61,0.92,4953,0.66
27038,0215_026.7,3,56537.5,Freeway,1.0,D1,3,34,Freeway,1964,0.5606,PM,0.2628,11.7,7.0,2028,25000,25000,3683.142000,I215,0,1000000,CO8,1.0,S00-Ann,3683,1.13,1-Weekday (Tu-Th),4179,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1529.5140,1.0584,1474,1618,0.25,0.27,0.92,1759,0.30
27039,0215_026.7,3,56537.5,Freeway,1.0,D2,2,34,Freeway,2080,0.4394,PM,0.2628,14.3,8.9,2028,25000,25000,2886.858000,I215,0,1000000,CO8,1.0,S00-Ann,2886,1.13,1-Weekday (Tu-Th),3275,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1198.6500,1.0731,1171,1286,0.28,0.31,0.92,1398,0.34
27040,0215_027.4,3,37297.2,Freeway,1.0,D1,2,34,Freeway,2080,0.6149,PM,0.2756,8.9,5.2,2028,21500,21500,3643.528460,I215,0,1000000,CO8,1.0,S00-Ann,3643,1.13,1-Weekday (Tu-Th),4134,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1513.0440,1.0438,1438,1579,0.35,0.38,0.92,1716,0.41


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,1871.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6022,AM,0.2031,8.7,6.6,2032,1800,1800,220.152276,Statewide,0,2500,COP,1.0,S00-Ann,220,0.98,1-Weekday (Tu-Th),216,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,88.4088,1.0504,75,92,0.07,0.09,0.88,105,0.10
1,0006_141.0,1,1871.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3978,AM,0.2031,5.9,6.5,2032,1800,1800,145.427724,Statewide,0,2500,COP,1.0,S00-Ann,145,0.98,1-Weekday (Tu-Th),142,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,58.1206,1.0443,49,60,0.05,0.06,0.88,68,0.06
2,0006_146.9,1,1882.8,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6754,AM,0.2029,9.3,6.6,2032,1800,1800,246.669588,Statewide,0,2500,COP,1.0,S00-Ann,246,0.98,1-Weekday (Tu-Th),242,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,99.0506,1.0516,84,104,0.08,0.10,0.88,118,0.11
3,0006_146.9,1,1882.8,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3246,AM,0.2029,4.0,6.5,2032,1800,1800,118.550412,Statewide,0,2500,COP,1.0,S00-Ann,118,0.98,1-Weekday (Tu-Th),116,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,47.4788,1.0405,40,49,0.04,0.05,0.88,56,0.05
4,0065_000.1,1,453.8,Arterial,1.0,D1,1,4,Collector,806,0.6019,AM,0.1882,7.8,1.9,2032,1600,1600,181.244128,Statewide,0,2500,COP,1.0,S00-Ann,181,0.98,1-Weekday (Tu-Th),178,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,72.8554,1.0251,60,74,0.08,0.09,0.88,84,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28309,0215_025.7,3,144038.4,Freeway,1.0,D2,5,34,Freeway,1784,0.4412,PM,0.2527,12.4,7.1,2032,101000,101000,11260.615240,I215,0,1000000,CO8,1.0,S00-Ann,11260,1.13,1-Weekday (Tu-Th),12778,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4676.7480,1.0603,4516,4958,0.51,0.56,0.92,5389,0.60
28310,0215_026.7,3,61961.0,Freeway,1.0,D1,4,34,Freeway,1871,0.5411,PM,0.2497,19.9,10.2,2032,27500,27500,3715.598425,I215,0,1000000,CO8,1.0,S00-Ann,3715,1.13,1-Weekday (Tu-Th),4216,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1543.0560,1.0908,1532,1683,0.20,0.22,0.92,1829,0.24
28311,0215_026.7,3,61961.0,Freeway,1.0,D2,4,34,Freeway,1871,0.4589,PM,0.2497,13.6,7.0,2032,27500,27500,3151.151575,I215,0,1000000,CO8,1.0,S00-Ann,3151,1.13,1-Weekday (Tu-Th),3575,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1308.4500,1.0622,1265,1389,0.17,0.19,0.92,1510,0.20
28312,0215_027.4,3,50607.5,Freeway,1.0,D1,2,34,Freeway,2080,0.5447,PM,0.2003,23.2,12.2,2032,33000,33000,3600.412530,I215,0,1000000,CO8,1.0,S00-Ann,3600,1.13,1-Weekday (Tu-Th),4085,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1495.1100,1.1074,1507,1655,0.36,0.40,0.92,1799,0.43


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,2115.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6118,AM,0.2047,8.4,7.5,2042,2100,2100,262.994466,Statewide,0,2500,COP,1.0,S00-Ann,262,0.98,1-Weekday (Tu-Th),257,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,105.1901,1.0543,90,110,0.08,0.10,0.88,126,0.12
1,0006_141.0,1,2115.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3882,AM,0.2047,5.3,8.4,2042,2100,2100,166.875534,Statewide,0,2500,COP,1.0,S00-Ann,166,0.98,1-Weekday (Tu-Th),163,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,66.7159,1.0526,57,70,0.05,0.07,0.88,79,0.07
2,0006_146.9,1,2122.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6920,AM,0.2050,9.0,7.3,2042,2100,2100,297.906000,Statewide,0,2500,COP,1.0,S00-Ann,297,0.98,1-Weekday (Tu-Th),292,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,119.5156,1.0545,102,126,0.10,0.12,0.88,143,0.13
3,0006_146.9,1,2122.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3080,AM,0.2050,3.0,9.0,2042,2100,2100,132.594000,Statewide,0,2500,COP,1.0,S00-Ann,132,0.98,1-Weekday (Tu-Th),129,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,52.7997,1.0510,45,55,0.04,0.05,0.88,63,0.06
4,0065_000.1,1,537.8,Arterial,1.0,D1,1,4,Collector,806,0.6514,AM,0.1867,3.1,1.5,2042,1700,1700,206.747846,Statewide,0,2500,COP,1.0,S00-Ann,206,0.98,1-Weekday (Tu-Th),202,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,82.6786,1.0137,68,83,0.08,0.10,0.88,95,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29053,0215_025.7,3,158610.4,Freeway,1.0,D2,5,34,Freeway,1784,0.4349,PM,0.2482,11.9,6.5,2042,105000,105000,11333.928900,I215,0,1000000,CO8,1.0,S00-Ann,11333,1.13,1-Weekday (Tu-Th),12861,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4707.1260,1.0563,4528,4972,0.51,0.56,0.92,5404,0.61
29054,0215_026.7,3,66703.6,Freeway,1.0,D1,4,34,Freeway,1871,0.5625,PM,0.2689,16.3,8.6,2042,28500,28500,4310.803125,I215,0,1000000,CO8,1.0,S00-Ann,4310,1.13,1-Weekday (Tu-Th),4891,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1790.1060,1.0756,1753,1925,0.23,0.26,0.92,2092,0.28
29055,0215_026.7,3,66703.6,Freeway,1.0,D2,4,34,Freeway,1871,0.4375,PM,0.2689,12.6,6.6,2042,28500,28500,3352.846875,I215,0,1000000,CO8,1.0,S00-Ann,3352,1.13,1-Weekday (Tu-Th),3804,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1392.2640,1.0582,1341,1473,0.18,0.20,0.92,1601,0.21
29056,0215_027.4,3,54625.5,Freeway,1.0,D1,2,34,Freeway,2080,0.5597,PM,0.2271,18.5,12.4,2042,35000,35000,4448.775450,I215,0,1000000,CO8,1.0,S00-Ann,4448,1.13,1-Weekday (Tu-Th),5047,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1847.2020,1.0990,1848,2030,0.44,0.49,0.92,2206,0.53


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,2303.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6067,AM,0.2006,9.1,7.7,2050,2300,2500,304.260050,Statewide,0,2500,COP,1.0,S00-Ann,304,0.98,1-Weekday (Tu-Th),299,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,122.3807,1.0567,105,129,0.10,0.12,0.88,146,0.14
1,0006_141.0,1,2303.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3933,AM,0.2006,7.4,8.4,2050,2300,2500,197.239950,Statewide,0,2500,COP,1.0,S00-Ann,197,0.98,1-Weekday (Tu-Th),193,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,78.9949,1.0568,67,83,0.06,0.08,0.88,94,0.09
2,0006_146.9,1,2307.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6832,AM,0.2011,9.5,7.6,2050,2300,2500,343.478800,Statewide,0,2500,COP,1.0,S00-Ann,343,0.98,1-Weekday (Tu-Th),337,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,137.9341,1.0570,118,145,0.11,0.14,0.88,165,0.16
3,0006_146.9,1,2307.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3168,AM,0.2011,6.1,8.8,2050,2300,2500,159.271200,Statewide,0,2500,COP,1.0,S00-Ann,159,0.98,1-Weekday (Tu-Th),156,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,63.8508,1.0562,54,67,0.05,0.06,0.88,76,0.07
4,0065_000.1,1,614.1,Arterial,1.0,D1,1,4,Collector,806,0.6182,AM,0.1791,2.9,1.5,2050,1800,1900,210.367278,Statewide,0,2500,COP,1.0,S00-Ann,210,0.98,1-Weekday (Tu-Th),206,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,84.3158,1.0133,69,85,0.09,0.11,0.88,97,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29089,0215_025.7,3,141936.9,Freeway,1.0,D2,5,34,Freeway,1784,0.4293,PM,0.2350,10.0,5.9,2050,111000,94500,9533.679750,I215,0,1000000,CO8,1.0,S00-Ann,9533,1.13,1-Weekday (Tu-Th),10818,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,3959.3880,1.0495,3784,4155,0.42,0.47,0.92,4516,0.51
29090,0215_026.7,3,59122.6,Freeway,1.0,D1,4,34,Freeway,1871,0.5447,PM,0.2521,9.9,6.9,2050,25500,18000,2471.739660,I215,0,1000000,CO8,1.0,S00-Ann,2471,1.13,1-Weekday (Tu-Th),2804,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1026.2640,1.0543,985,1081,0.13,0.14,0.92,1176,0.16
29091,0215_026.7,3,59122.6,Freeway,1.0,D2,4,34,Freeway,1871,0.4553,PM,0.2521,13.6,7.3,2050,25500,18000,2066.060340,I215,0,1000000,CO8,1.0,S00-Ann,2066,1.13,1-Weekday (Tu-Th),2344,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,857.9040,1.0637,831,912,0.11,0.12,0.92,991,0.13
29092,0215_027.4,3,52448.7,Freeway,1.0,D1,2,34,Freeway,2080,0.5207,PM,0.2238,5.1,5.0,2050,34500,32500,3787.311450,I215,0,1000000,CO8,1.0,S00-Ann,3787,1.13,1-Weekday (Tu-Th),4297,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1572.7020,1.0352,1482,1628,0.36,0.39,0.92,1769,0.43


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,1917.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.5771,AM,0.2118,8.9,6.4,2032,1800,1800,220.013604,Statewide,0,2500,COP,1.0,S00-Ann,220,0.98,1-Weekday (Tu-Th),216,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,88.4088,1.0498,75,92,0.07,0.09,0.88,105,0.10
1,0006_141.0,1,1917.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.4229,AM,0.2118,7.1,5.8,2032,1800,1800,161.226396,Statewide,0,2500,COP,1.0,S00-Ann,161,0.98,1-Weekday (Tu-Th),158,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,64.6694,1.0432,54,67,0.05,0.06,0.88,76,0.07
2,0006_146.9,1,1926.8,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6339,AM,0.2112,9.3,6.6,2032,1800,1800,240.983424,Statewide,0,2500,COP,1.0,S00-Ann,240,0.98,1-Weekday (Tu-Th),236,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,96.5948,1.0516,82,101,0.08,0.10,0.88,115,0.11
3,0006_146.9,1,1926.8,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3661,AM,0.2112,6.0,5.4,2032,1800,1800,139.176576,Statewide,0,2500,COP,1.0,S00-Ann,139,0.98,1-Weekday (Tu-Th),136,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,55.6648,1.0390,47,57,0.04,0.05,0.88,65,0.06
4,0065_000.1,1,453.5,Arterial,1.0,D1,1,4,Collector,806,0.6193,AM,0.1969,5.8,0.0,2032,1600,1600,195.104272,Statewide,0,2500,COP,1.0,S00-Ann,195,0.98,1-Weekday (Tu-Th),191,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,78.1763,1.0116,64,79,0.08,0.10,0.88,89,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28567,0215_025.7,3,144306.8,Freeway,1.0,D2,5,34,Freeway,1784,0.4420,PM,0.2532,12.6,7.2,2032,101000,101000,11303.354400,I215,0,1000000,CO8,1.0,S00-Ann,11303,1.13,1-Weekday (Tu-Th),12827,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4694.6820,1.0612,4537,4981,0.51,0.56,0.92,5415,0.61
28568,0215_026.7,3,61753.3,Freeway,1.0,D1,4,34,Freeway,1871,0.5372,PM,0.2523,19.2,9.3,2032,27500,27500,3727.227900,I215,0,1000000,CO8,1.0,S00-Ann,3727,1.13,1-Weekday (Tu-Th),4229,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1547.8140,1.0849,1529,1679,0.20,0.22,0.92,1825,0.24
28569,0215_026.7,3,61753.3,Freeway,1.0,D2,4,34,Freeway,1871,0.4628,PM,0.2523,13.6,6.9,2032,27500,27500,3211.022100,I215,0,1000000,CO8,1.0,S00-Ann,3211,1.13,1-Weekday (Tu-Th),3644,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1333.7040,1.0617,1289,1415,0.17,0.19,0.92,1539,0.21
28570,0215_027.4,3,49902.1,Freeway,1.0,D1,2,34,Freeway,2080,0.5389,PM,0.2011,23.1,13.2,2032,33000,32500,3522.115675,I215,0,1000000,CO8,1.0,S00-Ann,3522,1.13,1-Weekday (Tu-Th),3997,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1462.9020,1.1122,1481,1627,0.36,0.39,0.92,1768,0.43


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,2151.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6106,AM,0.2060,8.1,7.3,2042,2100,2100,264.145560,Statewide,0,2500,COP,1.0,S00-Ann,264,0.98,1-Weekday (Tu-Th),259,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,106.0087,1.0527,90,111,0.09,0.10,0.88,126,0.12
1,0006_141.0,1,2151.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3894,AM,0.2060,4.7,7.7,2042,2100,2100,168.454440,Statewide,0,2500,COP,1.0,S00-Ann,168,0.98,1-Weekday (Tu-Th),165,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,67.5345,1.0479,57,70,0.05,0.07,0.88,80,0.08
2,0006_146.9,1,2157.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6914,AM,0.2058,8.8,7.2,2042,2100,2100,298.809252,Statewide,0,2500,COP,1.0,S00-Ann,298,0.98,1-Weekday (Tu-Th),293,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,119.9249,1.0536,102,126,0.10,0.12,0.88,143,0.13
3,0006_146.9,1,2157.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3086,AM,0.2058,2.2,8.0,2042,2100,2100,133.370748,Statewide,0,2500,COP,1.0,S00-Ann,133,0.98,1-Weekday (Tu-Th),130,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.2090,1.0444,45,55,0.04,0.05,0.88,63,0.06
4,0065_000.1,1,529.0,Arterial,1.0,D1,1,4,Collector,806,0.6515,AM,0.1953,4.5,1.5,2042,1700,1700,216.304515,Statewide,0,2500,COP,1.0,S00-Ann,216,0.98,1-Weekday (Tu-Th),212,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,86.7716,1.0165,71,88,0.09,0.11,0.88,100,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29851,0215_025.7,3,160828.3,Freeway,1.0,D2,5,34,Freeway,1784,0.4333,PM,0.2562,11.8,6.2,2042,105000,107000,11878.226220,I215,0,1000000,CO8,1.0,S00-Ann,11878,1.13,1-Weekday (Tu-Th),13480,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4933.6800,1.0546,4738,5203,0.53,0.58,0.92,5655,0.63
29852,0215_026.7,3,64618.9,Freeway,1.0,D1,4,34,Freeway,1871,0.5552,PM,0.2597,17.5,9.2,2042,28500,26500,3820.914160,I215,0,1000000,CO8,1.0,S00-Ann,3820,1.13,1-Weekday (Tu-Th),4335,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1586.6100,1.0810,1562,1715,0.21,0.23,0.92,1864,0.25
29853,0215_026.7,3,64618.9,Freeway,1.0,D2,4,34,Freeway,1871,0.4448,PM,0.2597,13.3,6.5,2042,28500,26500,3061.135840,I215,0,1000000,CO8,1.0,S00-Ann,3061,1.13,1-Weekday (Tu-Th),3473,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1271.1180,1.0591,1226,1346,0.16,0.18,0.92,1463,0.20
29854,0215_027.4,3,56849.7,Freeway,1.0,D1,2,34,Freeway,2080,0.5251,PM,0.2203,19.4,13.4,2042,35000,37000,4280.142610,I215,0,1000000,CO8,1.0,S00-Ann,4280,1.13,1-Weekday (Tu-Th),4857,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1777.6620,1.1058,1790,1965,0.43,0.47,0.92,2136,0.51


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,2346.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6183,AM,0.1974,9.0,7.5,2050,2300,2300,280.720566,Statewide,0,2500,COP,1.0,S00-Ann,280,0.98,1-Weekday (Tu-Th),275,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,112.5575,1.0555,96,118,0.09,0.11,0.88,135,0.13
1,0006_141.0,1,2346.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3817,AM,0.1974,8.0,8.2,2050,2300,2300,173.299434,Statewide,0,2500,COP,1.0,S00-Ann,173,0.98,1-Weekday (Tu-Th),170,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,69.5810,1.0570,59,73,0.06,0.07,0.88,83,0.08
2,0006_146.9,1,2351.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.7047,AM,0.1974,9.2,7.3,2050,2300,2300,319.947894,Statewide,0,2500,COP,1.0,S00-Ann,319,0.98,1-Weekday (Tu-Th),313,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,128.1109,1.0549,110,135,0.10,0.13,0.88,153,0.14
3,0006_146.9,1,2351.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.2953,AM,0.1974,7.3,8.8,2050,2300,2300,134.072106,Statewide,0,2500,COP,1.0,S00-Ann,134,0.98,1-Weekday (Tu-Th),131,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.6183,1.0586,46,56,0.04,0.05,0.88,64,0.06
4,0065_000.1,1,608.3,Arterial,1.0,D1,1,4,Collector,806,0.6490,AM,0.1920,6.6,4.0,2050,1800,1800,224.294400,Statewide,0,2500,COP,1.0,S00-Ann,224,0.98,1-Weekday (Tu-Th),220,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,90.0460,1.0332,75,93,0.09,0.12,0.88,105,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31591,0215_025.7,3,168749.3,Freeway,1.0,D2,5,34,Freeway,1784,0.4207,PM,0.2483,10.4,6.3,2050,111000,113000,11803.958530,I215,0,1000000,CO8,1.0,S00-Ann,11803,1.13,1-Weekday (Tu-Th),13394,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4902.2040,1.0523,4698,5158,0.53,0.58,0.92,5607,0.63
31592,0215_026.7,3,66134.1,Freeway,1.0,D1,4,34,Freeway,1871,0.5845,PM,0.2563,16.6,9.8,2050,25500,28000,4194.605800,I215,0,1000000,CO8,1.0,S00-Ann,4194,1.13,1-Weekday (Tu-Th),4759,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1741.7940,1.0822,1716,1884,0.23,0.25,0.92,2048,0.27
31593,0215_026.7,3,66134.1,Freeway,1.0,D2,4,34,Freeway,1871,0.4155,PM,0.2563,13.2,7.4,2050,25500,28000,2981.794200,I215,0,1000000,CO8,1.0,S00-Ann,2981,1.13,1-Weekday (Tu-Th),3383,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1238.1780,1.0634,1199,1316,0.16,0.18,0.92,1431,0.19
31594,0215_027.4,3,58652.9,Freeway,1.0,D1,2,34,Freeway,2080,0.5818,PM,0.2251,16.5,10.3,2050,34500,39000,5107.564020,I215,0,1000000,CO8,1.0,S00-Ann,5107,1.13,1-Weekday (Tu-Th),5795,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,2120.9700,1.0845,2094,2300,0.50,0.55,0.92,2500,0.60


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
0,0006_141.0,1,2362.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6145,AM,0.1973,9.0,7.6,2050,2300,2300,278.853955,Statewide,0,2500,COP,1.0,S00-Ann,278,0.98,1-Weekday (Tu-Th),273,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,111.7389,1.0560,96,117,0.09,0.11,0.88,134,0.13
1,0006_141.0,1,2362.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3855,AM,0.1973,7.9,8.5,2050,2300,2300,174.936045,Statewide,0,2500,COP,1.0,S00-Ann,174,0.98,1-Weekday (Tu-Th),171,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,69.9903,1.0583,60,74,0.06,0.07,0.88,84,0.08
2,0006_146.9,1,2365.0,Arterial,1.0,D1,1,2,Principal Arterial,1065,0.6981,AM,0.1975,9.2,7.4,2050,2300,2300,317.111925,Statewide,0,2500,COP,1.0,S00-Ann,317,0.98,1-Weekday (Tu-Th),311,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,127.2923,1.0554,109,134,0.10,0.13,0.88,152,0.14
3,0006_146.9,1,2365.0,Arterial,1.0,D2,1,2,Principal Arterial,1065,0.3019,AM,0.1975,7.1,9.2,2050,2300,2300,137.138075,Statewide,0,2500,COP,1.0,S00-Ann,137,0.98,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.8462,1.0602,47,58,0.04,0.05,0.88,66,0.06
4,0065_000.1,1,606.3,Arterial,1.0,D1,1,4,Collector,806,0.6303,AM,0.1829,4.3,1.4,2050,1800,1800,207.507366,Statewide,0,2500,COP,1.0,S00-Ann,207,0.98,1-Weekday (Tu-Th),203,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.0879,1.0156,68,84,0.09,0.10,0.88,95,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31597,0215_025.7,3,168159.4,Freeway,1.0,D2,5,34,Freeway,1784,0.4204,PM,0.2479,10.4,6.3,2050,111000,112000,11672.321920,I215,0,1000000,CO8,1.0,S00-Ann,11672,1.13,1-Weekday (Tu-Th),13246,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4848.0360,1.0523,4646,5101,0.52,0.57,0.92,5545,0.62
31598,0215_026.7,3,65775.5,Freeway,1.0,D1,4,34,Freeway,1871,0.5857,PM,0.2546,16.9,9.9,2050,25500,27500,4100.778550,I215,0,1000000,CO8,1.0,S00-Ann,4100,1.13,1-Weekday (Tu-Th),4652,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1702.6320,1.0833,1679,1844,0.22,0.25,0.92,2004,0.27
31599,0215_026.7,3,65775.5,Freeway,1.0,D2,4,34,Freeway,1871,0.4143,PM,0.2546,13.3,7.8,2050,25500,27500,2900.721450,I215,0,1000000,CO8,1.0,S00-Ann,2900,1.13,1-Weekday (Tu-Th),3291,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1204.5060,1.0656,1168,1283,0.16,0.17,0.92,1395,0.19
31600,0215_027.4,3,58587.8,Freeway,1.0,D1,2,34,Freeway,2080,0.5822,PM,0.2219,16.5,10.6,2050,34500,39000,5038.417020,I215,0,1000000,CO8,1.0,S00-Ann,5038,1.13,1-Weekday (Tu-Th),5717,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,2092.4220,1.0860,2069,2272,0.50,0.55,0.92,2469,0.59


In [81]:
df = dfs_VC[0]

df = df[(df[nameSegID]=='0015_282.2') * (df['VCGroupCode']=='AnnWkPM')]

df

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
19064,0015_282.2,4,184675.8,Arterial,0.0360,D1,1,4,Collector,696,0.4545,PM,0.3867,9.7,6.6,2019,0,0,0.0,I15,0,1000000,COU,1.0,S00-Ann,0,1.09,1-Weekday (Tu-Th),0,2015-2019,PM,0.3615,Prd,Season,AnnWkPM,3,0.0,1.0524,0,0,0.0,0.0,0.92,0,0.0
19065,0015_282.2,4,184675.8,Arterial,0.0360,D2,1,4,Collector,696,0.5455,PM,0.3867,9.7,5.5,2019,0,0,0.0,I15,0,1000000,COU,1.0,S00-Ann,0,1.09,1-Weekday (Tu-Th),0,2015-2019,PM,0.3615,Prd,Season,AnnWkPM,3,0.0,1.0469,0,0,0.0,0.0,0.92,0,0.0
25464,0015_282.2,4,184675.8,Freeway,0.8558,D1,3,33,Freeway,1730,0.4682,PM,0.2121,8.7,6.6,2019,0,0,0.0,I15,0,1000000,CO6,1.0,S00-Ann,0,1.07,1-Weekday (Tu-Th),0,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,0.0,1.0504,0,0,0.0,0.0,0.92,0,0.0
25465,0015_282.2,4,184675.8,Freeway,0.8558,D2,3,33,Freeway,1730,0.5318,PM,0.2121,7.3,6.0,2019,0,0,0.0,I15,0,1000000,CO6,1.0,S00-Ann,0,1.07,1-Weekday (Tu-Th),0,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,0.0,1.0446,0,0,0.0,0.0,0.92,0,0.0
25466,0015_282.2,4,184675.8,Managed,0.1082,D1,1,38,Freeway,1767,0.4368,PM,0.3087,0.0,0.0,2019,0,0,0.0,I15,0,1000000,CO6,1.0,S00-Ann,0,1.07,1-Weekday (Tu-Th),0,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,0.0,1.0000,0,0,0.0,0.0,0.92,0,0.0
25467,0015_282.2,4,184675.8,Managed,0.1082,D2,1,38,Freeway,1767,0.5632,PM,0.3087,0.0,0.0,2019,0,0,0.0,I15,0,1000000,CO6,1.0,S00-Ann,0,1.07,1-Weekday (Tu-Th),0,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,0.0,1.0000,0,0,0.0,0.0,0.92,0,0.0


In [82]:
df_VC.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd_TDM', 'PRDFAC',
       'MDPCT', 'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj',
       'AvgAnnualPrdVol', 'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP',
       'SsnFactor', 'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol', 'YEARGROUP', 'PERIOD', 'VOLPKHRPCT', 'HRPCTOF',
       'SeasonType', 'VCGroupCode', 'PRDHRS', 'AvgSsnDOWPkHrVol', 'TrkFac',
       'PrdPCEFlow', 'PkHrPCEFlow', 'PrdVC', 'PkHrVC', 'PHF', 'Pk15PCEFlow',
       'Pk15VC'],
      dtype='object')

In [83]:
df1 = df_VC.groupby(['SEGID'                  ],as_index=False).agg(num=(nameDir,'size'))
df2 = df_VC.groupby(['SEGID','ForecastAADTAdj'],as_index=False).agg(num=(nameDir,'size'))
df3 = pd.DataFrame.merge(df1,df2,on='SEGID',how='outer')

In [84]:
df2g = df2.groupby(['SEGID'],as_index=False).agg(numFor=('ForecastAADTAdj','size'))
df2g[df2g['numFor']==2]


,SEGID,numFor


In [85]:
df2[df2['SEGID']=='0038_003.2']

,SEGID,ForecastAADTAdj,num
172,0038_003.2,5500,6


In [86]:
dfs_VC_MaxDirFG = []

for df_VC in dfs_VC:

    df_VC_MaxDirFG = df_VC.groupby([nameSegID,'PERIOD','SEASONGROUP','DOWGROUP','HRPCTOF','VCGroupCode'], as_index=False).agg({nameDir:[np.size],nameCap1HL:[np.sum],'PrdPCEFlow':[np.sum],'PkHrPCEFlow':[np.sum],'Pk15PCEFlow':[np.sum],'PrdVC':[np.max],'PkHrVC':[np.max],'Pk15VC':[np.max]})

    df_VC_MaxDirFG.columns = ([nameSegID,'PERIOD','SEASONGROUP','DOWGROUP','HRPCTOF','VCGroupCode','numRecords',nameCap1HL+'Sum','PrdPCEFlowSum','PkHrPCEFlowSum','Pk15PCEFlowSum','PrdVCMax','PkHrVCMax','Pk15VCMax'])
    
    dfs_VC_MaxDirFG.append(df_VC_MaxDirFG)
    
    display(df_VC_MaxDirFG)
                                            
    #df_Peak_VC_wGroups = pd.DataFrame.merge(df_VCGroups,df_Peak_VC,on=('SeasonGroup','DOWGroup','Prd','HrPctOf'))

,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,106,131,149,0.06,0.07,0.08
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,77,90,103,0.04,0.05,0.05
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,137,152,172,0.07,0.07,0.08
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,106,130,148,0.07,0.08,0.09
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,77,91,103,0.04,0.05,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12676,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1030,0,0,0,0.00,0.00,0.00
12677,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1030,0,0,0,0.00,0.00,0.00
12678,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1788,0,0,0,0.00,0.00,0.00
12679,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1788,0,0,0,0.00,0.00,0.00


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,116,143,162,0.06,0.08,0.09
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,85,99,113,0.04,0.05,0.06
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,134,148,168,0.06,0.07,0.08
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,117,143,163,0.07,0.09,0.10
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,84,98,112,0.05,0.05,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12952,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1030,428,494,536,0.43,0.50,0.54
12953,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,1030,649,722,785,0.64,0.71,0.78
12954,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1788,0,0,0,0.00,0.00,0.00
12955,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1788,0,0,0,0.00,0.00,0.00


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,122,150,171,0.07,0.08,0.09
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,91,108,123,0.05,0.05,0.06
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,139,154,176,0.07,0.07,0.08
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,130,160,181,0.08,0.10,0.11
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,97,113,129,0.05,0.06,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13153,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,978,404,474,516,0.43,0.50,0.55
13154,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,978,582,630,686,0.64,0.69,0.75
13155,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1788,0,0,0,0.00,0.00,0.00
13156,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1788,0,0,0,0.00,0.00,0.00


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,124,152,173,0.07,0.09,0.10
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,100,117,133,0.05,0.06,0.07
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,153,170,193,0.07,0.08,0.09
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,124,153,174,0.08,0.10,0.11
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,100,116,132,0.05,0.06,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13729,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,978,424,497,541,0.45,0.52,0.57
13730,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,978,631,685,744,0.69,0.75,0.82
13731,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1788,0,0,0,0.00,0.00,0.00
13732,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1788,0,0,0,0.00,0.00,0.00


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,147,180,205,0.08,0.10,0.12
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,108,126,143,0.06,0.07,0.08
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,188,208,237,0.09,0.10,0.12
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,147,181,206,0.10,0.12,0.13
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,107,127,143,0.06,0.07,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14014,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,978,472,554,603,0.50,0.59,0.64
14015,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,978,667,723,786,0.72,0.78,0.85
14016,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1698,0,0,0,0.00,0.00,0.00
14017,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1698,0,0,0,0.00,0.00,0.00


...

,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,172,212,240,0.10,0.12,0.14
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,135,158,180,0.07,0.08,0.09
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,214,237,269,0.10,0.11,0.13
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,172,212,241,0.11,0.14,0.16
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,134,158,179,0.07,0.09,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14035,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,978,565,663,722,0.59,0.70,0.76
14036,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,978,795,862,911,0.88,0.96,0.99
14037,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1698,24,29,31,0.02,0.03,0.03
14038,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1698,14,16,18,0.01,0.01,0.01


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,129,159,181,0.07,0.09,0.10
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,96,112,128,0.05,0.06,0.07
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,157,175,198,0.08,0.09,0.10
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,129,158,180,0.08,0.10,0.11
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,96,113,129,0.05,0.06,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13825,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,978,427,501,544,0.45,0.53,0.57
13826,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,978,626,679,738,0.68,0.74,0.81
13827,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1788,0,0,0,0.00,0.00,0.00
13828,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1788,0,0,0,0.00,0.00,0.00


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,147,181,206,0.09,0.10,0.12
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,109,128,145,0.06,0.07,0.08
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,185,205,233,0.09,0.10,0.11
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,147,181,206,0.10,0.12,0.13
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,109,127,145,0.06,0.07,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14242,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,978,451,529,576,0.48,0.57,0.61
14243,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,978,645,699,761,0.70,0.76,0.83
14244,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1698,77,93,102,0.07,0.09,0.10
14245,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1698,47,55,60,0.03,0.03,0.04


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,155,191,218,0.09,0.11,0.13
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,118,138,158,0.06,0.07,0.08
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,208,231,263,0.10,0.11,0.13
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,156,191,217,0.10,0.13,0.14
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,119,139,158,0.07,0.08,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15103,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,978,508,596,648,0.53,0.63,0.68
15104,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,978,752,815,871,0.83,0.90,0.95
15105,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1698,0,0,0,0.00,0.00,0.00
15106,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1698,0,0,0,0.00,0.00,0.00


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
0,0006_141.0,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,156,191,218,0.09,0.11,0.13
1,0006_141.0,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,121,142,161,0.06,0.07,0.08
2,0006_141.0,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,2130,208,231,262,0.10,0.11,0.13
3,0006_146.9,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,2130,156,192,218,0.10,0.13,0.14
4,0006_146.9,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,2130,121,142,161,0.07,0.08,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15106,WFRC_8473,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,978,504,591,642,0.53,0.62,0.68
15107,WFRC_8473,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,2,978,749,812,867,0.83,0.90,0.94
15108,WFRC_8476,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,2,1698,0,0,0,0.00,0.00,0.00
15109,WFRC_8476,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,2,1698,0,0,0,0.00,0.00,0.00


In [87]:
display(df_Scenarios)

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
...,...,...,...,...,...,...,...,...
11,NB5042,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP42
12,UF32,2032 Needs-Based Scenario,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based Scenario,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


In [88]:
df = dfs_VC[0]

df = df[(df[nameSegID]=='0015_295.6') * (df['VCGroupCode']=='AnnWkPM')]

display(df)

df = dfs_VC_MaxDirFG[0]

df = df[(df[nameSegID]=='0015_295.6')]

pd.set_option('display.max_columns', None)

display(df)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
25512,0015_295.6,4,291541.8,Freeway,0.7363,D1,4,34,Freeway,1871,0.4318,PM,0.1990,12.0,8.2,2019,263955,263955,16700.151384,I15,0,1000000,CO6,1.0,S00-Ann,16700,1.07,1-Weekday (Tu-Th),17791,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,6244.641,1.0650,6315,6650,0.84,0.89,0.95,7000,0.94
25513,0015_295.6,4,291541.8,Freeway,0.7363,D2,4,34,Freeway,1871,0.5682,PM,0.1990,8.1,5.6,2019,263955,263955,21975.511849,I15,0,1000000,CO6,1.0,S00-Ann,21975,1.07,1-Weekday (Tu-Th),23411,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,8217.261,1.0442,8148,8580,1.09,1.15,0.98,8755,1.17
25514,0015_295.6,4,291541.8,Managed,0.1137,D1,1,38,Freeway,1767,0.4425,PM,0.3086,0.0,0.0,2019,263955,263955,4098.260446,I15,0,1000000,CO6,1.0,S00-Ann,4098,1.07,1-Weekday (Tu-Th),4365,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,1532.115,1.0000,1455,1532,0.82,0.87,0.95,1612,0.91
25515,0015_295.6,4,291541.8,Managed,0.1137,D2,1,38,Freeway,1767,0.5575,PM,0.3086,0.0,0.0,2019,263955,263955,5163.345082,I15,0,1000000,CO6,1.0,S00-Ann,5163,1.07,1-Weekday (Tu-Th),5500,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,1930.500,1.0000,1833,1930,1.04,1.09,0.98,1969,1.11
25516,0015_295.6,4,291541.8,CD Road,0.1500,D1,2,31,Freeway,1473,0.0000,PM,0.2176,0.0,0.0,2019,263955,263955,0.000000,I15,0,1000000,CO6,1.0,S00-Ann,0,1.07,1-Weekday (Tu-Th),0,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,0.000,1.0000,0,0,0.00,0.00,0.92,0,0.00
25517,0015_295.6,4,291541.8,CD Road,0.1500,D2,3,31,Freeway,1473,1.0000,PM,0.2176,8.2,6.1,2019,263955,263955,8615.491200,I15,0,1000000,CO6,1.0,S00-Ann,8615,1.07,1-Weekday (Tu-Th),9178,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,3221.478,1.0469,3202,3372,0.72,0.76,0.92,3665,0.83


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
267,0015_295.6,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,6,10222,17761,19720,20629,0.92,1.03,1.05
268,0015_295.6,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,6,10222,16251,18245,19416,0.88,0.99,1.02
269,0015_295.6,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,6,10222,20953,22064,23001,1.09,1.15,1.17


# VC Groups as Columns

In [89]:
df_VCGroups

,SEASONGROUP,SeasonType,DOWGROUP,PERIOD,HRPCTOF,VCGroupCode
VCGroupID,,,,,,
0,S00-Ann,Season,1-Weekday (Tu-Th),AM,Prd,AnnWkAM
1,S00-Ann,Season,1-Weekday (Tu-Th),MD,Prd,AnnWkMD
2,S00-Ann,Season,1-Weekday (Tu-Th),PM,Prd,AnnWkPM


In [90]:
dfs_VC_MaxDirFG_wVCGroups_15 = []

for df_VC_MaxDirFG in dfs_VC_MaxDirFG:

    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG[['VCGroupCode','SEGID','Pk15VCMax']].copy()
    
    df_VC_MaxDirFG_wVCGroups_15['VCGroupCode'] = df_VC_MaxDirFG_wVCGroups_15['VCGroupCode'] + "15"
    
    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG_wVCGroups_15.round({'Pk15VC': 2})
    
    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG_wVCGroups_15.pivot(index='SEGID', columns="VCGroupCode",values='Pk15VCMax')
    
    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG_wVCGroups_15.reset_index()
    
    dfs_VC_MaxDirFG_wVCGroups_15.append(df_VC_MaxDirFG_wVCGroups_15)

    display(df_VC_MaxDirFG_wVCGroups_15)
    

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.08,0.05,0.08
1,0006_146.9,0.09,0.06,0.09
2,0006_149.9,0.12,0.09,0.16
3,0006_150.6,0.12,0.09,0.16
4,0006_152.6,0.14,0.10,0.19
...,...,...,...,...
4222,WFRC_8457,0.00,0.00,0.00
4223,WFRC_8461,0.00,0.00,0.00
4224,WFRC_8466,0.00,0.00,0.00
4225,WFRC_8473,0.00,0.00,0.00


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.09,0.06,0.08
1,0006_146.9,0.10,0.06,0.08
2,0006_149.9,0.13,0.09,0.18
3,0006_150.6,0.12,0.09,0.17
4,0006_152.6,0.14,0.09,0.18
...,...,...,...,...
4314,WFRC_8461,0.20,0.16,0.24
4315,WFRC_8466,0.37,0.21,0.40
4316,WFRC_8467,0.55,0.32,0.67
4317,WFRC_8473,0.61,0.54,0.78


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.09,0.06,0.08
1,0006_146.9,0.11,0.07,0.09
2,0006_149.9,0.14,0.10,0.21
3,0006_150.6,0.14,0.10,0.20
4,0006_152.6,0.16,0.10,0.21
...,...,...,...,...
4381,WFRC_8461,0.20,0.14,0.43
4382,WFRC_8466,0.42,0.23,0.45
4383,WFRC_8467,0.68,0.38,0.79
4384,WFRC_8473,0.62,0.55,0.75


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.10,0.07,0.09
1,0006_146.9,0.11,0.07,0.09
2,0006_149.9,0.15,0.10,0.20
3,0006_150.6,0.15,0.10,0.20
4,0006_152.6,0.17,0.11,0.19
...,...,...,...,...
4573,WFRC_8466,0.44,0.25,0.48
4574,WFRC_8467,0.68,0.40,0.86
4575,WFRC_8471,0.51,0.38,0.63
4576,WFRC_8473,0.67,0.57,0.82


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.12,0.08,0.12
1,0006_146.9,0.13,0.08,0.12
2,0006_149.9,0.21,0.11,0.22
3,0006_150.6,0.21,0.12,0.23
4,0006_152.6,0.23,0.13,0.24
...,...,...,...,...
4668,WFRC_8466,0.51,0.32,0.53
4669,WFRC_8467,0.70,0.48,0.76
4670,WFRC_8471,0.63,0.46,0.76
4671,WFRC_8473,0.74,0.64,0.85


...

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.14,0.09,0.13
1,0006_146.9,0.16,0.10,0.13
2,0006_149.9,0.37,0.22,0.44
3,0006_150.6,0.36,0.22,0.43
4,0006_152.6,0.40,0.24,0.47
...,...,...,...,...
4675,WFRC_8466,0.64,0.43,0.66
4676,WFRC_8467,0.74,0.53,0.80
4677,WFRC_8471,0.72,0.55,0.88
4678,WFRC_8473,0.91,0.76,0.99


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.10,0.07,0.10
1,0006_146.9,0.11,0.07,0.10
2,0006_149.9,0.15,0.10,0.19
3,0006_150.6,0.14,0.10,0.19
4,0006_152.6,0.16,0.11,0.18
...,...,...,...,...
4605,WFRC_8466,0.44,0.24,0.48
4606,WFRC_8467,0.68,0.39,0.77
4607,WFRC_8471,0.50,0.41,0.64
4608,WFRC_8473,0.67,0.57,0.81


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.12,0.08,0.11
1,0006_146.9,0.13,0.08,0.12
2,0006_149.9,0.14,0.11,0.40
3,0006_150.6,0.14,0.11,0.39
4,0006_152.6,0.16,0.12,0.35
...,...,...,...,...
4744,WFRC_8470,0.00,0.00,0.00
4745,WFRC_8471,0.65,0.52,0.81
4746,WFRC_8472,0.00,0.00,0.00
4747,WFRC_8473,0.74,0.61,0.83


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.13,0.08,0.13
1,0006_146.9,0.14,0.09,0.13
2,0006_149.9,0.17,0.16,0.27
3,0006_150.6,0.17,0.16,0.26
4,0006_152.6,0.21,0.17,0.26
...,...,...,...,...
5031,WFRC_8470,0.00,0.00,0.00
5032,WFRC_8471,0.64,0.55,0.80
5033,WFRC_8472,0.00,0.00,0.00
5034,WFRC_8473,0.87,0.68,0.95


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.13,0.08,0.13
1,0006_146.9,0.14,0.09,0.13
2,0006_149.9,0.17,0.15,0.27
3,0006_150.6,0.17,0.15,0.28
4,0006_152.6,0.20,0.16,0.27
...,...,...,...,...
5032,WFRC_8470,0.00,0.00,0.00
5033,WFRC_8471,0.64,0.55,0.80
5034,WFRC_8472,0.00,0.00,0.00
5035,WFRC_8473,0.88,0.68,0.94


In [91]:
df_VC_MaxDirFG.columns

Index(['SEGID', 'PERIOD', 'SEASONGROUP', 'DOWGROUP', 'HRPCTOF', 'VCGroupCode',
       'numRecords', 'CAP1HLSum', 'PrdPCEFlowSum', 'PkHrPCEFlowSum',
       'Pk15PCEFlowSum', 'PrdVCMax', 'PkHrVCMax', 'Pk15VCMax'],
      dtype='object')

In [92]:
dfs_VC_MaxDirFG_wVCGroups_Hr = []

for df_VC_MaxDirFG in dfs_VC_MaxDirFG:

    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG[['VCGroupCode','SEGID','PkHrVCMax']].copy()
    
    df_VC_MaxDirFG_wVCGroups_Hr['VCGroupCode'] = df_VC_MaxDirFG_wVCGroups_Hr['VCGroupCode'] + "Hr"
    
    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG_wVCGroups_Hr.round({'PkHrVC': 2})
    
    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG_wVCGroups_Hr.pivot(index=('SEGID'), columns="VCGroupCode",values='PkHrVCMax')
    
    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG_wVCGroups_Hr.reset_index()
    
    dfs_VC_MaxDirFG_wVCGroups_Hr.append(df_VC_MaxDirFG_wVCGroups_Hr)

    display(df_VC_MaxDirFG_wVCGroups_Hr)
    

VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.07,0.05,0.07
1,0006_146.9,0.08,0.05,0.08
2,0006_149.9,0.11,0.08,0.14
3,0006_150.6,0.10,0.08,0.14
4,0006_152.6,0.13,0.09,0.16
...,...,...,...,...
4222,WFRC_8457,0.00,0.00,0.00
4223,WFRC_8461,0.00,0.00,0.00
4224,WFRC_8466,0.00,0.00,0.00
4225,WFRC_8473,0.00,0.00,0.00


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.08,0.05,0.07
1,0006_146.9,0.09,0.05,0.07
2,0006_149.9,0.11,0.08,0.16
3,0006_150.6,0.11,0.08,0.15
4,0006_152.6,0.13,0.08,0.16
...,...,...,...,...
4314,WFRC_8461,0.18,0.14,0.21
4315,WFRC_8466,0.34,0.19,0.37
4316,WFRC_8467,0.51,0.29,0.62
4317,WFRC_8473,0.56,0.50,0.71


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.08,0.05,0.07
1,0006_146.9,0.10,0.06,0.08
2,0006_149.9,0.12,0.09,0.18
3,0006_150.6,0.12,0.09,0.18
4,0006_152.6,0.14,0.09,0.18
...,...,...,...,...
4381,WFRC_8461,0.18,0.13,0.40
4382,WFRC_8466,0.39,0.21,0.41
4383,WFRC_8467,0.63,0.35,0.73
4384,WFRC_8473,0.57,0.50,0.69


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.09,0.06,0.08
1,0006_146.9,0.10,0.06,0.08
2,0006_149.9,0.13,0.08,0.18
3,0006_150.6,0.13,0.09,0.18
4,0006_152.6,0.15,0.10,0.17
...,...,...,...,...
4573,WFRC_8466,0.40,0.23,0.44
4574,WFRC_8467,0.63,0.36,0.79
4575,WFRC_8471,0.47,0.35,0.58
4576,WFRC_8473,0.62,0.52,0.75


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.10,0.07,0.10
1,0006_146.9,0.12,0.07,0.10
2,0006_149.9,0.19,0.10,0.20
3,0006_150.6,0.19,0.10,0.20
4,0006_152.6,0.20,0.11,0.21
...,...,...,...,...
4668,WFRC_8466,0.47,0.29,0.48
4669,WFRC_8467,0.65,0.44,0.70
4670,WFRC_8471,0.58,0.43,0.70
4671,WFRC_8473,0.68,0.59,0.78


...

VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.12,0.08,0.11
1,0006_146.9,0.14,0.09,0.11
2,0006_149.9,0.32,0.19,0.39
3,0006_150.6,0.31,0.19,0.38
4,0006_152.6,0.35,0.21,0.41
...,...,...,...,...
4675,WFRC_8466,0.59,0.39,0.61
4676,WFRC_8467,0.68,0.49,0.73
4677,WFRC_8471,0.66,0.51,0.81
4678,WFRC_8473,0.84,0.70,0.96


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.09,0.06,0.09
1,0006_146.9,0.10,0.06,0.09
2,0006_149.9,0.13,0.08,0.17
3,0006_150.6,0.13,0.08,0.17
4,0006_152.6,0.14,0.09,0.16
...,...,...,...,...
4605,WFRC_8466,0.41,0.22,0.44
4606,WFRC_8467,0.62,0.36,0.71
4607,WFRC_8471,0.46,0.37,0.59
4608,WFRC_8473,0.61,0.53,0.74


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.10,0.07,0.10
1,0006_146.9,0.12,0.07,0.10
2,0006_149.9,0.12,0.10,0.35
3,0006_150.6,0.13,0.10,0.35
4,0006_152.6,0.14,0.11,0.31
...,...,...,...,...
4744,WFRC_8470,0.00,0.00,0.00
4745,WFRC_8471,0.60,0.48,0.75
4746,WFRC_8472,0.00,0.00,0.00
4747,WFRC_8473,0.68,0.57,0.76


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.11,0.07,0.11
1,0006_146.9,0.13,0.08,0.12
2,0006_149.9,0.15,0.14,0.24
3,0006_150.6,0.15,0.14,0.23
4,0006_152.6,0.18,0.15,0.23
...,...,...,...,...
5031,WFRC_8470,0.00,0.00,0.00
5032,WFRC_8471,0.59,0.51,0.73
5033,WFRC_8472,0.00,0.00,0.00
5034,WFRC_8473,0.80,0.63,0.90


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.11,0.07,0.11
1,0006_146.9,0.13,0.08,0.12
2,0006_149.9,0.15,0.13,0.24
3,0006_150.6,0.15,0.14,0.24
4,0006_152.6,0.18,0.14,0.23
...,...,...,...,...
5032,WFRC_8470,0.00,0.00,0.00
5033,WFRC_8471,0.59,0.51,0.73
5034,WFRC_8472,0.00,0.00,0.00
5035,WFRC_8473,0.81,0.62,0.90


In [93]:
dfs_VC_MaxDirFG_wVCGroups_Prd = []

for df_VC_MaxDirFG in dfs_VC_MaxDirFG:

    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG[df_VC_MaxDirFG['HRPCTOF'] == 'Prd'].copy()
    
    df_VC_MaxDirFG_wVCGroups_Prd['VCGroupCode'] = df_VC_MaxDirFG_wVCGroups_Prd['VCGroupCode'] + "Prd"
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd[['VCGroupCode','SEGID','PrdVCMax']]
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.round({'PrdVC': 2})
    
    #df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.pivot(index=('SEGID','ForecastAADT'), columns="VCGroupCode",values='PrdVCMax')
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.pivot(index=('SEGID'), columns="VCGroupCode",values='PrdVCMax')
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.reset_index()
    
    dfs_VC_MaxDirFG_wVCGroups_Prd.append(df_VC_MaxDirFG_wVCGroups_Prd)

    display(df_VC_MaxDirFG_wVCGroups_Prd)

VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.06,0.04,0.07
1,0006_146.9,0.07,0.04,0.07
2,0006_149.9,0.09,0.07,0.13
3,0006_150.6,0.08,0.07,0.13
4,0006_152.6,0.10,0.07,0.15
...,...,...,...,...
4222,WFRC_8457,0.00,0.00,0.00
4223,WFRC_8461,0.00,0.00,0.00
4224,WFRC_8466,0.00,0.00,0.00
4225,WFRC_8473,0.00,0.00,0.00


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.06,0.04,0.06
1,0006_146.9,0.07,0.05,0.06
2,0006_149.9,0.09,0.07,0.15
3,0006_150.6,0.09,0.07,0.14
4,0006_152.6,0.10,0.07,0.15
...,...,...,...,...
4314,WFRC_8461,0.15,0.12,0.18
4315,WFRC_8466,0.28,0.16,0.34
4316,WFRC_8467,0.42,0.25,0.55
4317,WFRC_8473,0.47,0.43,0.64


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.07,0.05,0.07
1,0006_146.9,0.08,0.05,0.07
2,0006_149.9,0.10,0.07,0.17
3,0006_150.6,0.10,0.08,0.16
4,0006_152.6,0.11,0.08,0.17
...,...,...,...,...
4381,WFRC_8461,0.15,0.12,0.36
4382,WFRC_8466,0.32,0.18,0.38
4383,WFRC_8467,0.52,0.30,0.66
4384,WFRC_8473,0.47,0.43,0.64


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.07,0.05,0.07
1,0006_146.9,0.08,0.05,0.07
2,0006_149.9,0.11,0.07,0.17
3,0006_150.6,0.11,0.08,0.16
4,0006_152.6,0.12,0.08,0.16
...,...,...,...,...
4573,WFRC_8466,0.33,0.19,0.40
4574,WFRC_8467,0.52,0.32,0.71
4575,WFRC_8471,0.39,0.30,0.52
4576,WFRC_8473,0.51,0.45,0.69


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.08,0.06,0.09
1,0006_146.9,0.10,0.06,0.09
2,0006_149.9,0.16,0.09,0.18
3,0006_150.6,0.15,0.09,0.19
4,0006_152.6,0.17,0.10,0.19
...,...,...,...,...
4668,WFRC_8466,0.39,0.25,0.45
4669,WFRC_8467,0.54,0.38,0.63
4670,WFRC_8471,0.48,0.37,0.63
4671,WFRC_8473,0.57,0.50,0.72


...

VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.10,0.07,0.10
1,0006_146.9,0.11,0.07,0.10
2,0006_149.9,0.27,0.17,0.36
3,0006_150.6,0.26,0.17,0.35
4,0006_152.6,0.29,0.19,0.38
...,...,...,...,...
4675,WFRC_8466,0.49,0.33,0.56
4676,WFRC_8467,0.57,0.42,0.66
4677,WFRC_8471,0.55,0.44,0.73
4678,WFRC_8473,0.69,0.59,0.88


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.07,0.05,0.08
1,0006_146.9,0.08,0.05,0.08
2,0006_149.9,0.11,0.07,0.16
3,0006_150.6,0.10,0.07,0.15
4,0006_152.6,0.12,0.08,0.15
...,...,...,...,...
4605,WFRC_8466,0.34,0.19,0.41
4606,WFRC_8467,0.52,0.31,0.64
4607,WFRC_8471,0.38,0.32,0.53
4608,WFRC_8473,0.51,0.45,0.68


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.09
1,0006_146.9,0.10,0.06,0.09
2,0006_149.9,0.10,0.09,0.33
3,0006_150.6,0.10,0.09,0.32
4,0006_152.6,0.12,0.09,0.29
...,...,...,...,...
4744,WFRC_8470,0.00,0.00,0.00
4745,WFRC_8471,0.50,0.42,0.67
4746,WFRC_8472,0.00,0.00,0.00
4747,WFRC_8473,0.56,0.48,0.70


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.10
1,0006_146.9,0.10,0.07,0.11
2,0006_149.9,0.13,0.12,0.22
3,0006_150.6,0.12,0.12,0.22
4,0006_152.6,0.15,0.13,0.21
...,...,...,...,...
5031,WFRC_8470,0.00,0.00,0.00
5032,WFRC_8471,0.49,0.44,0.66
5033,WFRC_8472,0.00,0.00,0.00
5034,WFRC_8473,0.66,0.53,0.83


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.10
1,0006_146.9,0.10,0.07,0.11
2,0006_149.9,0.12,0.12,0.22
3,0006_150.6,0.13,0.12,0.23
4,0006_152.6,0.15,0.13,0.22
...,...,...,...,...
5032,WFRC_8470,0.00,0.00,0.00
5033,WFRC_8471,0.49,0.44,0.66
5034,WFRC_8472,0.00,0.00,0.00
5035,WFRC_8473,0.67,0.53,0.83


In [94]:
df_Scenarios

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
...,...,...,...,...,...,...,...,...
11,NB5042,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP42
12,UF32,2032 Needs-Based Scenario,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based Scenario,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


In [95]:
dfs_VC_MaxDirFG_wVCGroups = []

i=-1
for (idx, row) in df_Scenarios.iterrows():

    if row.loc['Process'] == 'Yes':

        i=i+1
        
        df_VC_MaxDirFG_wVCGroups = pd.DataFrame.merge(dfs_VC_MaxDirFG_wVCGroups_15[i],dfs_VC_MaxDirFG_wVCGroups_Hr[i],on='SEGID')
        df_VC_MaxDirFG_wVCGroups = pd.DataFrame.merge(df_VC_MaxDirFG_wVCGroups,dfs_VC_MaxDirFG_wVCGroups_Prd[i],on='SEGID')
        dfs_VC_MaxDirFG_wVCGroups.append(df_VC_MaxDirFG_wVCGroups)
        display(df_VC_MaxDirFG_wVCGroups)

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.08,0.05,0.08,0.07,0.05,0.07,0.06,0.04,0.07
1,0006_146.9,0.09,0.06,0.09,0.08,0.05,0.08,0.07,0.04,0.07
2,0006_149.9,0.12,0.09,0.16,0.11,0.08,0.14,0.09,0.07,0.13
3,0006_150.6,0.12,0.09,0.16,0.10,0.08,0.14,0.08,0.07,0.13
4,0006_152.6,0.14,0.10,0.19,0.13,0.09,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...
4222,WFRC_8457,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4223,WFRC_8461,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4224,WFRC_8466,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4225,WFRC_8473,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.08,0.08,0.05,0.07,0.06,0.04,0.06
1,0006_146.9,0.10,0.06,0.08,0.09,0.05,0.07,0.07,0.05,0.06
2,0006_149.9,0.13,0.09,0.18,0.11,0.08,0.16,0.09,0.07,0.15
3,0006_150.6,0.12,0.09,0.17,0.11,0.08,0.15,0.09,0.07,0.14
4,0006_152.6,0.14,0.09,0.18,0.13,0.08,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...
4314,WFRC_8461,0.20,0.16,0.24,0.18,0.14,0.21,0.15,0.12,0.18
4315,WFRC_8466,0.37,0.21,0.40,0.34,0.19,0.37,0.28,0.16,0.34
4316,WFRC_8467,0.55,0.32,0.67,0.51,0.29,0.62,0.42,0.25,0.55
4317,WFRC_8473,0.61,0.54,0.78,0.56,0.50,0.71,0.47,0.43,0.64


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.08,0.08,0.05,0.07,0.07,0.05,0.07
1,0006_146.9,0.11,0.07,0.09,0.10,0.06,0.08,0.08,0.05,0.07
2,0006_149.9,0.14,0.10,0.21,0.12,0.09,0.18,0.10,0.07,0.17
3,0006_150.6,0.14,0.10,0.20,0.12,0.09,0.18,0.10,0.08,0.16
4,0006_152.6,0.16,0.10,0.21,0.14,0.09,0.18,0.11,0.08,0.17
...,...,...,...,...,...,...,...,...,...,...
4381,WFRC_8461,0.20,0.14,0.43,0.18,0.13,0.40,0.15,0.12,0.36
4382,WFRC_8466,0.42,0.23,0.45,0.39,0.21,0.41,0.32,0.18,0.38
4383,WFRC_8467,0.68,0.38,0.79,0.63,0.35,0.73,0.52,0.30,0.66
4384,WFRC_8473,0.62,0.55,0.75,0.57,0.50,0.69,0.47,0.43,0.64


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.10,0.07,0.09,0.09,0.06,0.08,0.07,0.05,0.07
1,0006_146.9,0.11,0.07,0.09,0.10,0.06,0.08,0.08,0.05,0.07
2,0006_149.9,0.15,0.10,0.20,0.13,0.08,0.18,0.11,0.07,0.17
3,0006_150.6,0.15,0.10,0.20,0.13,0.09,0.18,0.11,0.08,0.16
4,0006_152.6,0.17,0.11,0.19,0.15,0.10,0.17,0.12,0.08,0.16
...,...,...,...,...,...,...,...,...,...,...
4573,WFRC_8466,0.44,0.25,0.48,0.40,0.23,0.44,0.33,0.19,0.40
4574,WFRC_8467,0.68,0.40,0.86,0.63,0.36,0.79,0.52,0.32,0.71
4575,WFRC_8471,0.51,0.38,0.63,0.47,0.35,0.58,0.39,0.30,0.52
4576,WFRC_8473,0.67,0.57,0.82,0.62,0.52,0.75,0.51,0.45,0.69


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.12,0.08,0.12,0.10,0.07,0.10,0.08,0.06,0.09
1,0006_146.9,0.13,0.08,0.12,0.12,0.07,0.10,0.10,0.06,0.09
2,0006_149.9,0.21,0.11,0.22,0.19,0.10,0.20,0.16,0.09,0.18
3,0006_150.6,0.21,0.12,0.23,0.19,0.10,0.20,0.15,0.09,0.19
4,0006_152.6,0.23,0.13,0.24,0.20,0.11,0.21,0.17,0.10,0.19
...,...,...,...,...,...,...,...,...,...,...
4668,WFRC_8466,0.51,0.32,0.53,0.47,0.29,0.48,0.39,0.25,0.45
4669,WFRC_8467,0.70,0.48,0.76,0.65,0.44,0.70,0.54,0.38,0.63
4670,WFRC_8471,0.63,0.46,0.76,0.58,0.43,0.70,0.48,0.37,0.63
4671,WFRC_8473,0.74,0.64,0.85,0.68,0.59,0.78,0.57,0.50,0.72


...

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.14,0.09,0.13,0.12,0.08,0.11,0.10,0.07,0.10
1,0006_146.9,0.16,0.10,0.13,0.14,0.09,0.11,0.11,0.07,0.10
2,0006_149.9,0.37,0.22,0.44,0.32,0.19,0.39,0.27,0.17,0.36
3,0006_150.6,0.36,0.22,0.43,0.31,0.19,0.38,0.26,0.17,0.35
4,0006_152.6,0.40,0.24,0.47,0.35,0.21,0.41,0.29,0.19,0.38
...,...,...,...,...,...,...,...,...,...,...
4675,WFRC_8466,0.64,0.43,0.66,0.59,0.39,0.61,0.49,0.33,0.56
4676,WFRC_8467,0.74,0.53,0.80,0.68,0.49,0.73,0.57,0.42,0.66
4677,WFRC_8471,0.72,0.55,0.88,0.66,0.51,0.81,0.55,0.44,0.73
4678,WFRC_8473,0.91,0.76,0.99,0.84,0.70,0.96,0.69,0.59,0.88


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.10,0.07,0.10,0.09,0.06,0.09,0.07,0.05,0.08
1,0006_146.9,0.11,0.07,0.10,0.10,0.06,0.09,0.08,0.05,0.08
2,0006_149.9,0.15,0.10,0.19,0.13,0.08,0.17,0.11,0.07,0.16
3,0006_150.6,0.14,0.10,0.19,0.13,0.08,0.17,0.10,0.07,0.15
4,0006_152.6,0.16,0.11,0.18,0.14,0.09,0.16,0.12,0.08,0.15
...,...,...,...,...,...,...,...,...,...,...
4605,WFRC_8466,0.44,0.24,0.48,0.41,0.22,0.44,0.34,0.19,0.41
4606,WFRC_8467,0.68,0.39,0.77,0.62,0.36,0.71,0.52,0.31,0.64
4607,WFRC_8471,0.50,0.41,0.64,0.46,0.37,0.59,0.38,0.32,0.53
4608,WFRC_8473,0.67,0.57,0.81,0.61,0.53,0.74,0.51,0.45,0.68


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.12,0.08,0.11,0.10,0.07,0.10,0.09,0.06,0.09
1,0006_146.9,0.13,0.08,0.12,0.12,0.07,0.10,0.10,0.06,0.09
2,0006_149.9,0.14,0.11,0.40,0.12,0.10,0.35,0.10,0.09,0.33
3,0006_150.6,0.14,0.11,0.39,0.13,0.10,0.35,0.10,0.09,0.32
4,0006_152.6,0.16,0.12,0.35,0.14,0.11,0.31,0.12,0.09,0.29
...,...,...,...,...,...,...,...,...,...,...
4744,WFRC_8470,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4745,WFRC_8471,0.65,0.52,0.81,0.60,0.48,0.75,0.50,0.42,0.67
4746,WFRC_8472,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4747,WFRC_8473,0.74,0.61,0.83,0.68,0.57,0.76,0.56,0.48,0.70


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.13,0.08,0.13,0.11,0.07,0.11,0.09,0.06,0.10
1,0006_146.9,0.14,0.09,0.13,0.13,0.08,0.12,0.10,0.07,0.11
2,0006_149.9,0.17,0.16,0.27,0.15,0.14,0.24,0.13,0.12,0.22
3,0006_150.6,0.17,0.16,0.26,0.15,0.14,0.23,0.12,0.12,0.22
4,0006_152.6,0.21,0.17,0.26,0.18,0.15,0.23,0.15,0.13,0.21
...,...,...,...,...,...,...,...,...,...,...
5031,WFRC_8470,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5032,WFRC_8471,0.64,0.55,0.80,0.59,0.51,0.73,0.49,0.44,0.66
5033,WFRC_8472,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5034,WFRC_8473,0.87,0.68,0.95,0.80,0.63,0.90,0.66,0.53,0.83


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.13,0.08,0.13,0.11,0.07,0.11,0.09,0.06,0.10
1,0006_146.9,0.14,0.09,0.13,0.13,0.08,0.12,0.10,0.07,0.11
2,0006_149.9,0.17,0.15,0.27,0.15,0.13,0.24,0.12,0.12,0.22
3,0006_150.6,0.17,0.15,0.28,0.15,0.14,0.24,0.13,0.12,0.23
4,0006_152.6,0.20,0.16,0.27,0.18,0.14,0.23,0.15,0.13,0.22
...,...,...,...,...,...,...,...,...,...,...
5032,WFRC_8470,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5033,WFRC_8471,0.64,0.55,0.80,0.59,0.51,0.73,0.49,0.44,0.66
5034,WFRC_8472,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5035,WFRC_8473,0.88,0.68,0.94,0.81,0.62,0.90,0.67,0.53,0.83


# Export final tables to CSV

In [96]:
df_Scenarios.to_csv(os.path.join(dirResults, r'scenarios.csv'),index=False)

In [97]:
if export_results: 
    df_Scenarios.to_csv(os.path.join(dirResults, r'scenarios.csv'),index=False)
    df_VCGroup_DOWPkHr.to_csv(os.path.join(dirResults, r'vcgroups_dowpeak.csv'),index=False)
    df_VCGroup_Seasons.to_csv(os.path.join(dirResults, r'vcgroups_seasons.csv'),index=False)
    df_VCGroups.to_csv(os.path.join(dirResults, r'vcgroups.csv'),index=False)

In [98]:
if export_results: 

    df_Scenarios_toExport = df_Scenarios[df_Scenarios['Process']=='Yes']
    
    i=0
    for (idx, row) in df_Scenarios_toExport.iterrows():    
    
        filename = os.path.join(dirResults,"3_VC_MaxDirFG_wVCGroups_" + row.loc['ScenarioCode'] + '.csv')
        display('Exporting ' + filename)
        dfs_VC_MaxDirFG_wVCGroups[i].to_csv(filename, index=False)

        filename = os.path.join(dirResults,"2_VC_MaxDirFG_" + row.loc['ScenarioCode'] + '.csv')
        display('Exporting ' + filename)
        dfs_VC_MaxDirFG[i].to_csv(filename, index=False)

        filename = os.path.join(dirResults,"1_VC_" + row.loc['ScenarioCode'] + '.csv')
        display('Exporting ' + filename)
        dfs_VC[i].to_csv(filename, index=False)
        i=i+1

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\3_VC_MaxDirFG_wVCGroups_Base19.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\2_VC_MaxDirFG_Base19.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\1_VC_Base19.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\3_VC_MaxDirFG_wVCGroups_Base23.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\2_VC_MaxDirFG_Base23.csv'

...

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\2_VC_MaxDirFG_UF50.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\1_VC_UF50.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\3_VC_MaxDirFG_wVCGroups_UF50_MAG.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\2_VC_MaxDirFG_UF50_MAG.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\1_VC_UF50_MAG.csv'

In [99]:
for (idx, row) in df_Scenarios_toExport.iterrows():
    display(row.loc['ScenarioCode'])

'Base19'

'Base23'

'TIP28'

'RTP32'

'RTP42'

...

'NB5042'

'UF32'

'UF42'

'UF50'

'UF50_MAG'

In [100]:
df = dfs_VC_MaxDirFG_wVCGroups[0]
df[df['SEGID']=='0079_003.0']
df

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.08,0.05,0.08,0.07,0.05,0.07,0.06,0.04,0.07
1,0006_146.9,0.09,0.06,0.09,0.08,0.05,0.08,0.07,0.04,0.07
2,0006_149.9,0.12,0.09,0.16,0.11,0.08,0.14,0.09,0.07,0.13
3,0006_150.6,0.12,0.09,0.16,0.10,0.08,0.14,0.08,0.07,0.13
4,0006_152.6,0.14,0.10,0.19,0.13,0.09,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...
4222,WFRC_8457,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4223,WFRC_8461,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4224,WFRC_8466,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4225,WFRC_8473,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [101]:
df_Scenarios_toProcess

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
...,...,...,...,...,...,...,...,...
11,NB5042,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP42
12,UF32,2032 Needs-Based Scenario,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based Scenario,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


# Exports for WebApp

In [102]:
if export_results:
    
    df_Scenarios_json = df_Scenarios.copy()
    df_Scenarios_json = df_Scenarios_json[['ScenarioCode','ScenarioName']]
    df_Scenarios_json.columns = ('value','label')
    df_Scenarios_json['filename'] = "VC_" + df_Scenarios_json['value'] + '.json'
    
    df_VCGroup_Seasons_json = df_VCGroup_Seasons.copy()
    df_VCGroup_Seasons_json = df_VCGroup_Seasons_json[['SeasonCode','SeasonName','SeasonType']]
    df_VCGroup_Seasons_json.columns = ('value','label','SeasonType')
        
    df_VCGroup_DOWPkHr_json = df_VCGroup_DOWPkHr.copy()
    df_VCGroup_DOWPkHr_json = df_VCGroup_DOWPkHr_json[['DOWPkCode','DOWPkName']]
    df_VCGroup_DOWPkHr_json.columns = ('value','label')

    df_Scenarios_json.to_json(os.path.join(dirResults, r'WebAppData\scenarios.json'),orient='records')
    #df_Scenarios_json = df_Scenarios_json.reset_index()
    #df_Scenarios_json.to_json(os.path.join(dirResults, r'WebAppData\scenarios_test.json'))
    df_VCGroup_Seasons_json.to_json(os.path.join(dirResults, r'WebAppData\vcgroups_seasons.json'),orient='records')
    df_VCGroup_DOWPkHr_json.to_json(os.path.join(dirResults, r'WebAppData\vcgroups_dowpeaks.json'),orient='records')
    

In [103]:
df_Scenarios_json

,value,label,filename
ScenarioID,,,
0,Base19,2019 Base,VC_Base19.json
1,Base23,2023 Base,VC_Base23.json
2,TIP28,2028 TIP Projects Scenario,VC_TIP28.json
3,RTP32,2032 Fiscally-Constrained Scenario,VC_RTP32.json
4,RTP42,2042 Fiscally-Constrained Scenario,VC_RTP42.json
...,...,...,...
11,NB5042,2050 SE on a 2042 Network,VC_NB5042.json
12,UF32,2032 Needs-Based Scenario,VC_UF32.json
13,UF42,2042 Needs-Based Scenario,VC_UF42.json


In [104]:
if export_results:
    
    df_Scenarios_toExport = df_Scenarios_toProcess.reset_index()
    
    for index, df_VC_MaxDirFG_wVCGroups in enumerate(dfs_VC_MaxDirFG_wVCGroups):    

        filename = "WebAppData\VC_" + df_Scenarios_toExport['ScenarioCode'].iloc[index] + '.json'
        #display(filename)

        #df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.rename(columns={"SEGID": 'S'})

        #export to csv in data-processed
        df_VC_MaxDirFG_wVCGroups.to_json(os.path.join(dirResults, filename),orient='records')

        display('Done exporting ' + filename)

'Done exporting WebAppData\\VC_Base19.json'

'Done exporting WebAppData\\VC_Base23.json'

'Done exporting WebAppData\\VC_TIP28.json'

'Done exporting WebAppData\\VC_RTP32.json'

'Done exporting WebAppData\\VC_RTP42.json'

...

'Done exporting WebAppData\\VC_NB5042.json'

'Done exporting WebAppData\\VC_UF32.json'

'Done exporting WebAppData\\VC_UF42.json'

'Done exporting WebAppData\\VC_UF50.json'

'Done exporting WebAppData\\VC_UF50_MAG.json'

# Temp

In [105]:
df_Scenarios

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,Yes,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects Scenario,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained Scenario,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained Scenario,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
...,...,...,...,...,...,...,...,...
11,NB5042,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP42
12,UF32,2032 Needs-Based Scenario,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based Scenario,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


In [106]:
dfs_VC_MaxDirFG_wVCGroups =[]
df_TDMsf = []

display ('Scenario Codes:')

for (idx, row) in df_Scenarios.iterrows():    

    print(row.loc['ScenarioCode'])
    
    df_TDM = dfs_TDM[idx]
    
    df_TDMsf = df_TDM[['SEGID','FT']]
    
    df_TDMsfmax = df_TDMsf.groupby('SEGID', as_index=False).agg({'FT':[np.max]})
    df_TDMsfmax.columns = ['SEGID','FT']

    display(df_TDMsfmax)
    #df_TDMsf.append(df_TDMsf)
    
    df_VC_MaxDirFG_wVCGroups = pd.read_csv(os.path.join(dirResults, '3_VC_MaxDirFG_wVCGroups_' + row.loc['ScenarioCode'] + '.csv'))
    display(df_VC_MaxDirFG_wVCGroups)
    
    df_new = pd.DataFrame.merge(df_TDMsfmax,df_VC_MaxDirFG_wVCGroups,on=('SEGID'))
    display(df_new)
    
    filename = os.path.join(dirResults,r"temp\3_VC_MaxDirFG_wVCGroups_" + row.loc['ScenarioCode'] + '.csv')
    display('Exporting ' + filename)
    df_new.to_csv(filename,index=False)
    #dfs_VC_MaxDirFG_wVCGroups.append(dfs_Peak_VC_MaxDirFG_wVCGroups)


'Scenario Codes:'

Base19


,SEGID,FT
0,0006_141.0,2.0
1,0006_146.9,2.0
2,0006_149.9,2.0
3,0006_150.6,2.0
4,0006_152.6,3.0
...,...,...
4222,WFRC_8457,4.0
4223,WFRC_8461,5.0
4224,WFRC_8466,3.0
4225,WFRC_8473,5.0


,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.08,0.05,0.08,0.07,0.05,0.07,0.06,0.04,0.07
1,0006_146.9,0.09,0.06,0.09,0.08,0.05,0.08,0.07,0.04,0.07
2,0006_149.9,0.12,0.09,0.16,0.11,0.08,0.14,0.09,0.07,0.13
3,0006_150.6,0.12,0.09,0.16,0.10,0.08,0.14,0.08,0.07,0.13
4,0006_152.6,0.14,0.10,0.19,0.13,0.09,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...
4222,WFRC_8457,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4223,WFRC_8461,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4224,WFRC_8466,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4225,WFRC_8473,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


,SEGID,FT,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,2.0,0.08,0.05,0.08,0.07,0.05,0.07,0.06,0.04,0.07
1,0006_146.9,2.0,0.09,0.06,0.09,0.08,0.05,0.08,0.07,0.04,0.07
2,0006_149.9,2.0,0.12,0.09,0.16,0.11,0.08,0.14,0.09,0.07,0.13
3,0006_150.6,2.0,0.12,0.09,0.16,0.10,0.08,0.14,0.08,0.07,0.13
4,0006_152.6,3.0,0.14,0.10,0.19,0.13,0.09,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...,...
4222,WFRC_8457,4.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4223,WFRC_8461,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4224,WFRC_8466,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4225,WFRC_8473,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


...

UF50_MAG


,SEGID,FT
0,0006_141.0,2.0
1,0006_146.9,2.0
2,0006_149.9,2.0
3,0006_150.6,2.0
4,0006_152.6,3.0
...,...,...
5032,WFRC_8470,3.0
5033,WFRC_8471,4.0
5034,WFRC_8472,5.0
5035,WFRC_8473,5.0


,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.13,0.08,0.13,0.11,0.07,0.11,0.09,0.06,0.10
1,0006_146.9,0.14,0.09,0.13,0.13,0.08,0.12,0.10,0.07,0.11
2,0006_149.9,0.17,0.15,0.27,0.15,0.13,0.24,0.12,0.12,0.22
3,0006_150.6,0.17,0.15,0.28,0.15,0.14,0.24,0.13,0.12,0.23
4,0006_152.6,0.20,0.16,0.27,0.18,0.14,0.23,0.15,0.13,0.22
...,...,...,...,...,...,...,...,...,...,...
5032,WFRC_8470,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5033,WFRC_8471,0.64,0.55,0.80,0.59,0.51,0.73,0.49,0.44,0.66
5034,WFRC_8472,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5035,WFRC_8473,0.88,0.68,0.94,0.81,0.62,0.90,0.67,0.53,0.83


,SEGID,FT,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,2.0,0.13,0.08,0.13,0.11,0.07,0.11,0.09,0.06,0.10
1,0006_146.9,2.0,0.14,0.09,0.13,0.13,0.08,0.12,0.10,0.07,0.11
2,0006_149.9,2.0,0.17,0.15,0.27,0.15,0.13,0.24,0.12,0.12,0.22
3,0006_150.6,2.0,0.17,0.15,0.28,0.15,0.14,0.24,0.13,0.12,0.23
4,0006_152.6,3.0,0.20,0.16,0.27,0.18,0.14,0.23,0.15,0.13,0.22
...,...,...,...,...,...,...,...,...,...,...,...
5032,WFRC_8470,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5033,WFRC_8471,4.0,0.64,0.55,0.80,0.59,0.51,0.73,0.49,0.44,0.66
5034,WFRC_8472,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5035,WFRC_8473,5.0,0.88,0.68,0.94,0.81,0.62,0.90,0.67,0.53,0.83


'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\temp\\3_VC_MaxDirFG_wVCGroups_UF50_MAG.csv'

In [107]:
df_TDMsfmax = df_TDMsf.groupby('SEGID', as_index=False).agg({'FT':[np.max]})
df_TDMsfmax.columns = ['SEGID','FT']
df_TDMsfmax

,SEGID,FT
0,0006_141.0,2.0
1,0006_146.9,2.0
2,0006_149.9,2.0
3,0006_150.6,2.0
4,0006_152.6,3.0
...,...,...
5032,WFRC_8470,3.0
5033,WFRC_8471,4.0
5034,WFRC_8472,5.0
5035,WFRC_8473,5.0


In [108]:
df_VCGroup_Seasons

,SeasonCode,SeasonName,SEASONGROUP,SeasonType
0,Ann,Annual,S00-Ann,Season


In [109]:
df_TDMsfmax

,SEGID,FT
0,0006_141.0,2.0
1,0006_146.9,2.0
2,0006_149.9,2.0
3,0006_150.6,2.0
4,0006_152.6,3.0
...,...,...
5032,WFRC_8470,3.0
5033,WFRC_8471,4.0
5034,WFRC_8472,5.0
5035,WFRC_8473,5.0
